In [1]:
import pandas as pd
from WindPy import w
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [2]:
from datetime import date

# 新三板
问题：新三板2016.6.27起正式分层，所以之前转换交易方式的公司不能用创新层和基础层划分。
解决方式：不分层

正态化数值

In [27]:
#导入数据
path = 'C:/Users/84678/Desktop/做市商效率/新三板做市交易企业.xlsx'
df_n_m = pd.read_excel(path, sheet_name = "新三板做市")
df_n_m.drop(0,inplace = True)

In [28]:
#定义函数，用于给四个变量分组
def divide(df):
    bins = 3
    df['Segment_a'] = bins - pd.qcut(df['WGSD_ASSETS'], bins).cat.codes
    
    try:
        df['Segment_c'] = bins - pd.qcut(df['CLOSE_PER'], bins).cat.codes
    except:
        df['Segment_c'] = 0
        df['Segment_c'] = 2 - pd.qcut(df['CLOSE_PER'].loc[df['CLOSE_PER'] != 0],2).cat.codes
        df['Segment_c'].loc[df['CLOSE_PER'] == 0] = 0
        df['Segment_c'] += 1
        df['Segment_c'] = df['Segment_c'].astype(int)
    
    try:
        df['Segment_d'] = bins - pd.qcut(df['TRADE_DAYS_PER'], bins).cat.codes
    except:
        df['Segment_d'] = 0
        df['Segment_d'] = 2 - pd.qcut(df['TRADE_DAYS_PER'].loc[df['TRADE_DAYS_PER'] != 0],2).cat.codes
        df['Segment_d'].loc[df['TRADE_DAYS_PER'] == 0] = 0
        df['Segment_d'] += 1
        df['Segment_d'] = df['Segment_d'].astype(int)
        
    try:
        df['Segment_s'] = bins - pd.qcut(df['SHARE_TOTALTRADABLE'],bins).cat.codes
    except:
        df['Segment_s'] = 0
        df['Segment_s'] = 2 - pd.qcut(df['SHARE_TOTALTRADABLE'].loc[df['SHARE_TOTALTRADABLE'] != 0],2).cat.codes
        df['Segment_s'].loc[df['SHARE_TOTALTRADABLE'] == 0] = 0
        df['Segment_s'] += 1
        df['Segment_s'] = df['Segment_s'].astype(int)
    return df

In [29]:
#对做市商数据分组
divide(df_n_m)

C:\Users\84678\AppData\Local\Temp\ipykernel_15120\402152499.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Segment_s'].loc[df['SHARE_TOTALTRADABLE'] == 0] = 0


,代码,名称,做市首日,做市前100日,CLOSE_PER,TRADE_DAYS_PER,做市年份,匹配期,WGSD_ASSETS,SHARE_TOTALTRADABLE,Segment_a,Segment_c,Segment_d,Segment_s
1,430021.NQ,海鑫科金,2016-01-29 00:00:00,2015-10-21,16.50,58.0,2016.0,2015-12-31,1.169655e+09,135298934.0,1,1,1,2
2,430038.NQ,信维科技,2015-05-19 00:00:00,2015-02-08,13.18,20.0,2015.0,2014-12-31,8.448346e+07,13486082.0,3,1,1,3
3,430046.NQ,圣博润,2015-07-08 00:00:00,2015-03-30,10.74,27.0,2015.0,2014-12-31,7.873969e+07,6268656.0,3,2,1,3
4,430051.NQ,九恒星,2014-09-10 00:00:00,2014-06-02,6.82,42.0,2014.0,2013-12-31,2.749910e+08,40357180.0,1,2,1,2
5,430062.NQ,中科国信,2014-12-23 00:00:00,2014-09-14,17.60,31.0,2014.0,2013-12-31,6.788150e+07,3816226.0,3,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,872166.NQ,新烽光电,2018-12-26 00:00:00,2018-09-17,35.60,1.0,2018.0,2017-12-31,4.718734e+07,3849999.0,3,1,3,3
373,872169.NQ,雅琪生物,2021-08-13 00:00:00,2021-05-05,14.00,7.0,2021.0,2020-12-31,7.748684e+07,8621250.0,3,1,2,3
374,872219.NQ,联科云,2017-11-13 00:00:00,2017-08-05,27.50,9.0,2017.0,2016-12-31,1.998575e+07,0.0,3,1,2,1
375,873723.NQ,阅微基因,2022-06-27 00:00:00,2022-03-19,25.33,1.0,2022.0,2021-12-31,3.963585e+08,0.0,1,1,3,1


In [ ]:
#循环体尚未搞好
for i in range(df_n_m.shape()):
    #为每一个竞价交易的公司的四个变量赋值
    r = df_n_m.loc[i]
    date_b = r['做市前100日'].date()
    date_a = r['做市首日'].date()
    date_c = r['匹配期'].date()
    date_b = date_b.strftime('%Y%m%d')
    date_a = date_a.strftime('%Y%m%d')
    date_c = date_c.strftime('%Y%m%d')
    data_temp = w.wss("430003.NQ,430004.NQ,430005.NQ,430009.NQ,430010.NQ,430014.NQ,430017.NQ,430019.NQ,430020.NQ,430022.NQ,430025.NQ,430027.NQ,430028.NQ,430031.NQ,430033.NQ,430034.NQ,430041.NQ,430042.NQ,430044.NQ,430052.NQ,430053.NQ,430055.NQ,430056.NQ,430057.NQ,430061.NQ,430064.NQ,430066.NQ,430068.NQ,430072.NQ,430073.NQ,430076.NQ,430077.NQ,430078.NQ,430079.NQ,430081.NQ,430084.NQ,430085.NQ,430086.NQ,430089.NQ,430091.NQ,430093.NQ,430094.NQ,430096.NQ,430097.NQ,430099.NQ,430102.NQ,430103.NQ,430104.NQ,430105.NQ,430106.NQ,430107.NQ,430108.NQ,430109.NQ,430111.NQ,430112.NQ,430114.NQ,430116.NQ,430119.NQ,430120.NQ,430122.NQ,430124.NQ,430130.NQ,430131.NQ,430138.NQ,430143.NQ,430144.NQ,430145.NQ,430148.NQ,430152.NQ,430153.NQ,430156.NQ,430158.NQ,430159.NQ,430160.NQ,430161.NQ,430165.NQ,430168.NQ,430170.NQ,430171.NQ,430172.NQ,430176.NQ,430178.NQ,430179.NQ,430180.NQ,430182.NQ,430184.NQ,430185.NQ,430186.NQ,430188.NQ,430190.NQ,430191.NQ,430193.NQ,430197.NQ,430200.NQ,430203.NQ,430204.NQ,430215.NQ,430216.NQ,430217.NQ,430218.NQ,430219.NQ,430220.NQ,430224.NQ,430228.NQ,430229.NQ,430235.NQ,430239.NQ,430240.NQ,430241.NQ,430244.NQ,430246.NQ,430247.NQ,430249.NQ,430250.NQ,430252.NQ,430257.NQ,430258.NQ,430263.NQ,430264.NQ,430266.NQ,430268.NQ,430269.NQ,430271.NQ,430273.NQ,430274.NQ,430276.NQ,430281.NQ,430282.NQ,430283.NQ,430284.NQ,430285.NQ,430286.NQ,430288.NQ,430289.NQ,430292.NQ,430293.NQ,430296.NQ,430301.NQ,430304.NQ,430305.NQ,430314.NQ,430316.NQ,430319.NQ,430320.NQ,430321.NQ,430322.NQ,430324.NQ,430326.NQ,430331.NQ,430333.NQ,430336.NQ,430337.NQ,430338.NQ,430341.NQ,430343.NQ,430344.NQ,430346.NQ,430350.NQ,430359.NQ,430361.NQ,430362.NQ,430363.NQ,430366.NQ,430367.NQ,430368.NQ,430369.NQ,430370.NQ,430379.NQ,430380.NQ,430387.NQ,430388.NQ,430391.NQ,430392.NQ,430393.NQ,430395.NQ,430396.NQ,430397.NQ,430398.NQ,430403.NQ,430405.NQ,430409.NQ,430410.NQ,430412.NQ,430414.NQ,430420.NQ,430422.NQ,430423.NQ,430426.NQ,430427.NQ,430428.NQ,430429.NQ,430432.NQ,430434.NQ,430435.NQ,430437.NQ,430439.NQ,430441.NQ,430442.NQ,430443.NQ,430448.NQ,430451.NQ,430452.NQ,430453.NQ,430455.NQ,430457.NQ,430458.NQ,430459.NQ,430460.NQ,430461.NQ,430462.NQ,430463.NQ,430467.NQ,430470.NQ,430471.NQ,430473.NQ,430477.NQ,430479.NQ,430480.NQ,430482.NQ,430486.NQ,430487.NQ,430490.NQ,430491.NQ,430495.NQ,430497.NQ,430500.NQ,430502.NQ,430505.NQ,430509.NQ,430513.NQ,430514.NQ,430516.NQ,430517.NQ,430519.NQ,430520.NQ,430525.NQ,430528.NQ,430529.NQ,430530.NQ,430535.NQ,430537.NQ,430538.NQ,430539.NQ,430541.NQ,430542.NQ,430545.NQ,430549.NQ,430550.NQ,430552.NQ,430558.NQ,430560.NQ,430561.NQ,430562.NQ,430563.NQ,430566.NQ,430567.NQ,430570.NQ,430572.NQ,430573.NQ,430576.NQ,430577.NQ,430578.NQ,430579.NQ,430583.NQ,430584.NQ,430585.NQ,430586.NQ,430588.NQ,430589.NQ,430590.NQ,430592.NQ,430593.NQ,430594.NQ,430597.NQ,430601.NQ,430602.NQ,430607.NQ,430609.NQ,430610.NQ,430611.NQ,430615.NQ,430616.NQ,430619.NQ,430620.NQ,430626.NQ,430632.NQ,430635.NQ,430636.NQ,430638.NQ,430641.NQ,430644.NQ,430645.NQ,430646.NQ,430647.NQ,430649.NQ,430653.NQ,430655.NQ,430656.NQ,430657.NQ,430658.NQ,430659.NQ,430663.NQ,430664.NQ,430665.NQ,430666.NQ,430667.NQ,430669.NQ,430670.NQ,430673.NQ,430677.NQ,430683.NQ,430686.NQ,430687.NQ,430690.NQ,430692.NQ,430695.NQ,430697.NQ,430703.NQ,430704.NQ,430706.NQ,430709.NQ,430712.NQ,430716.NQ,430717.NQ,430719.NQ,430724.NQ,430725.NQ,430726.NQ,430728.NQ,430730.NQ,430732.NQ,430734.NQ,430736.NQ,430742.NQ,430743.NQ,430744.NQ,430745.NQ,430746.NQ,430748.NQ,430749.NQ,430753.NQ,430756.NQ,430758.NQ,430761.NQ,430762.NQ,830768.NQ,830769.NQ,830770.NQ,830772.NQ,830773.NQ,830774.NQ,830775.NQ,830776.NQ,830778.NQ,830779.NQ,830781.NQ,830782.NQ,830783.NQ,830784.NQ,830786.NQ,830787.NQ,830789.NQ,830790.NQ,830791.NQ,830792.NQ,830794.NQ,830796.NQ,830797.NQ,830798.NQ,830802.NQ,830803.NQ,830821.NQ,830824.NQ,830825.NQ,830828.NQ,830829.NQ,830830.NQ,830836.NQ,830840.NQ,830841.NQ,830844.NQ,830846.NQ,830848.NQ,830849.NQ,830851.NQ,830852.NQ,830856.NQ,830857.NQ,830859.NQ,830860.NQ,830861.NQ,830864.NQ,830867.NQ,830868.NQ,830870.NQ,830871.NQ,830872.NQ,830873.NQ,830874.NQ,830876.NQ,830877.NQ,830880.NQ,830883.NQ,830885.NQ,830886.NQ,830888.NQ,830889.NQ,830890.NQ,830891.NQ,830892.NQ,830894.NQ,830896.NQ,830898.NQ,830900.NQ,830902.NQ,830905.NQ,830913.NQ,830914.NQ,830915.NQ,830917.NQ,830918.NQ,830920.NQ,830922.NQ,830923.NQ,830924.NQ,830926.NQ,830927.NQ,830929.NQ,830930.NQ,830939.NQ,830941.NQ,830943.NQ,830949.NQ,830953.NQ,830955.NQ,830958.NQ,830961.NQ,830969.NQ,830971.NQ,830976.NQ,830980.NQ,830982.NQ,830983.NQ,830993.NQ,830994.NQ,830995.NQ,830996.NQ,831001.NQ,831002.NQ,831003.NQ,831004.NQ,831006.NQ,831013.NQ,831014.NQ,831021.NQ,831022.NQ,831025.NQ,831026.NQ,831032.NQ,831033.NQ,831034.NQ,831036.NQ,831040.NQ,831042.NQ,831047.NQ,831048.NQ,831053.NQ,831054.NQ,831057.NQ,831059.NQ,831063.NQ,831065.NQ,831066.NQ,831067.NQ,831070.NQ,831073.NQ,831078.NQ,831081.NQ,831083.NQ,831085.NQ,831091.NQ,831092.NQ,831095.NQ,831096.NQ,831097.NQ,831099.NQ,831101.NQ,831104.NQ,831107.NQ,831108.NQ,831110.NQ,831111.NQ,831114.NQ,831116.NQ,831119.NQ,831121.NQ,831122.NQ,831123.NQ,831124.NQ,831130.NQ,831131.NQ,831132.NQ,831133.NQ,831134.NQ,831137.NQ,831140.NQ,831142.NQ,831144.NQ,831149.NQ,831151.NQ,831154.NQ,831156.NQ,831162.NQ,831163.NQ,831165.NQ,831169.NQ,831175.NQ,831176.NQ,831179.NQ,831181.NQ,831183.NQ,831184.NQ,831185.NQ,831186.NQ,831187.NQ,831197.NQ,831199.NQ,831200.NQ,831201.NQ,831203.NQ,831208.NQ,831211.NQ,831216.NQ,831225.NQ,831227.NQ,831228.NQ,831229.NQ,831231.NQ,831232.NQ,831233.NQ,831234.NQ,831235.NQ,831236.NQ,831237.NQ,831239.NQ,831240.NQ,831244.NQ,831246.NQ,831252.NQ,831253.NQ,831254.NQ,831255.NQ,831259.NQ,831264.NQ,831266.NQ,831269.NQ,831270.NQ,831272.NQ,831273.NQ,831275.NQ,831276.NQ,831282.NQ,831284.NQ,831285.NQ,831288.NQ,831292.NQ,831293.NQ,831296.NQ,831297.NQ,831304.NQ,831308.NQ,831309.NQ,831312.NQ,831313.NQ,831317.NQ,831318.NQ,831321.NQ,831322.NQ,831323.NQ,831325.NQ,831330.NQ,831331.NQ,831332.NQ,831334.NQ,831336.NQ,831337.NQ,831338.NQ,831340.NQ,831343.NQ,831345.NQ,831346.NQ,831348.NQ,831350.NQ,831351.NQ,831355.NQ,831356.NQ,831357.NQ,831358.NQ,831361.NQ,831366.NQ,831367.NQ,831368.NQ,831369.NQ,831372.NQ,831373.NQ,831376.NQ,831380.NQ,831382.NQ,831384.NQ,831385.NQ,831386.NQ,831387.NQ,831389.NQ,831390.NQ,831391.NQ,831394.NQ,831395.NQ,831396.NQ,831397.NQ,831398.NQ,831399.NQ,831401.NQ,831402.NQ,831404.NQ,831405.NQ,831406.NQ,831408.NQ,831412.NQ,831414.NQ,831415.NQ,831416.NQ,831418.NQ,831419.NQ,831422.NQ,831423.NQ,831428.NQ,831433.NQ,831434.NQ,831439.NQ,831442.NQ,831446.NQ,831448.NQ,831449.NQ,831453.NQ,831459.NQ,831460.NQ,831462.NQ,831463.NQ,831464.NQ,831465.NQ,831467.NQ,831469.NQ,831470.NQ,831474.NQ,831476.NQ,831477.NQ,831478.NQ,831481.NQ,831482.NQ,831483.NQ,831485.NQ,831486.NQ,831490.NQ,831491.NQ,831492.NQ,831496.NQ,831501.NQ,831502.NQ,831510.NQ,831511.NQ,831514.NQ,831515.NQ,831516.NQ,831519.NQ,831521.NQ,831523.NQ,831532.NQ,831538.NQ,831539.NQ,831553.NQ,831554.NQ,831555.NQ,831556.NQ,831566.NQ,831569.NQ,831575.NQ,831577.NQ,831579.NQ,831581.NQ,831584.NQ,831587.NQ,831589.NQ,831590.NQ,831591.NQ,831593.NQ,831594.NQ,831598.NQ,831599.NQ,831605.NQ,831606.NQ,831608.NQ,831611.NQ,831614.NQ,831615.NQ,831619.NQ,831621.NQ,831622.NQ,831623.NQ,831625.NQ,831627.NQ,831629.NQ,831631.NQ,831633.NQ,831634.NQ,831638.NQ,831640.NQ,831642.NQ,831643.NQ,831646.NQ,831650.NQ,831654.NQ,831657.NQ,831658.NQ,831660.NQ,831661.NQ,831664.NQ,831668.NQ,831670.NQ,831672.NQ,831673.NQ,831674.NQ,831676.NQ,831677.NQ,831682.NQ,831685.NQ,831686.NQ,831690.NQ,831691.NQ,831693.NQ,831694.NQ,831701.NQ,831703.NQ,831706.NQ,831707.NQ,831708.NQ,831709.NQ,831714.NQ,831718.NQ,831719.NQ,831720.NQ,831721.NQ,831722.NQ,831724.NQ,831725.NQ,831727.NQ,831732.NQ,831734.NQ,831736.NQ,831738.NQ,831740.NQ,831742.NQ,831744.NQ,831747.NQ,831749.NQ,831750.NQ,831752.NQ,831753.NQ,831755.NQ,831756.NQ,831760.NQ,831765.NQ,831767.NQ,831769.NQ,831770.NQ,831773.NQ,831774.NQ,831778.NQ,831779.NQ,831783.NQ,831787.NQ,831793.NQ,831798.NQ,831799.NQ,831804.NQ,831805.NQ,831813.NQ,831814.NQ,831816.NQ,831817.NQ,831824.NQ,831827.NQ,831828.NQ,831830.NQ,831831.NQ,831833.NQ,831838.NQ,831841.NQ,831843.NQ,831850.NQ,831853.NQ,831854.NQ,831858.NQ,831860.NQ,831862.NQ,831865.NQ,831866.NQ,831869.NQ,831870.NQ,831874.NQ,831875.NQ,831877.NQ,831879.NQ,831880.NQ,831881.NQ,831882.NQ,831884.NQ,831885.NQ,831888.NQ,831892.NQ,831893.NQ,831894.NQ,831897.NQ,831904.NQ,831910.NQ,831912.NQ,831916.NQ,831917.NQ,831922.NQ,831924.NQ,831925.NQ,831929.NQ,831931.NQ,831934.NQ,831935.NQ,831936.NQ,831937.NQ,831942.NQ,831943.NQ,831944.NQ,831946.NQ,831948.NQ,831951.NQ,831952.NQ,831957.NQ,831958.NQ,831960.NQ,831962.NQ,831969.NQ,831970.NQ,831973.NQ,831974.NQ,831978.NQ,831985.NQ,831988.NQ,831989.NQ,831990.NQ,831991.NQ,831992.NQ,831994.NQ,831995.NQ,831999.NQ,832001.NQ,832002.NQ,832003.NQ,832006.NQ,832009.NQ,832010.NQ,832013.NQ,832014.NQ,832017.NQ,832018.NQ,832019.NQ,832021.NQ,832022.NQ,832025.NQ,832027.NQ,832033.NQ,832035.NQ,832036.NQ,832037.NQ,832040.NQ,832041.NQ,832042.NQ,832043.NQ,832045.NQ,832047.NQ,832048.NQ,832049.NQ,832050.NQ,832053.NQ,832054.NQ,832055.NQ,832058.NQ,832059.NQ,832061.NQ,832062.NQ,832063.NQ,832064.NQ,832067.NQ,832069.NQ,832075.NQ,832078.NQ,832081.NQ,832082.NQ,832084.NQ,832085.NQ,832088.NQ,832093.NQ,832094.NQ,832095.NQ,832100.NQ,832103.NQ,832105.NQ,832107.NQ,832108.NQ,832111.NQ,832113.NQ,832114.NQ,832116.NQ,832118.NQ,832119.NQ,832124.NQ,832126.NQ,832127.NQ,832129.NQ,832132.NQ,832133.NQ,832134.NQ,832136.NQ,832141.NQ,832146.NQ,832155.NQ,832163.NQ,832164.NQ,832173.NQ,832176.NQ,832178.NQ,832181.NQ,832185.NQ,832191.NQ,832196.NQ,832198.NQ,832205.NQ,832208.NQ,832209.NQ,832213.NQ,832214.NQ,832217.NQ,832221.NQ,832222.NQ,832223.NQ,832226.NQ,832229.NQ,832230.NQ,832232.NQ,832234.NQ,832239.NQ,832240.NQ,832241.NQ,832244.NQ,832246.NQ,832248.NQ,832249.NQ,832255.NQ,832256.NQ,832257.NQ,832258.NQ,832259.NQ,832263.NQ,832264.NQ,832265.NQ,832267.NQ,832270.NQ,832274.NQ,832276.NQ,832277.NQ,832281.NQ,832283.NQ,832289.NQ,832290.NQ,832291.NQ,832292.NQ,832293.NQ,832294.NQ,832296.NQ,832297.NQ,832298.NQ,832301.NQ,832302.NQ,832307.NQ,832309.NQ,832311.NQ,832314.NQ,832315.NQ,832318.NQ,832319.NQ,832320.NQ,832323.NQ,832326.NQ,832331.NQ,832334.NQ,832335.NQ,832337.NQ,832339.NQ,832344.NQ,832346.NQ,832347.NQ,832350.NQ,832351.NQ,832352.NQ,832354.NQ,832355.NQ,832357.NQ,832359.NQ,832360.NQ,832363.NQ,832364.NQ,832366.NQ,832368.NQ,832369.NQ,832372.NQ,832373.NQ,832377.NQ,832381.NQ,832383.NQ,832389.NQ,832391.NQ,832393.NQ,832394.NQ,832395.NQ,832398.NQ,832400.NQ,832402.NQ,832403.NQ,832405.NQ,832406.NQ,832409.NQ,832415.NQ,832417.NQ,832418.NQ,832423.NQ,832428.NQ,832430.NQ,832431.NQ,832432.NQ,832436.NQ,832437.NQ,832439.NQ,832441.NQ,832445.NQ,832447.NQ,832457.NQ,832459.NQ,832460.NQ,832465.NQ,832468.NQ,832469.NQ,832472.NQ,832476.NQ,832477.NQ,832479.NQ,832480.NQ,832484.NQ,832488.NQ,832492.NQ,832493.NQ,832494.NQ,832496.NQ,832502.NQ,832503.NQ,832505.NQ,832506.NQ,832508.NQ,832509.NQ,832510.NQ,832511.NQ,832513.NQ,832518.NQ,832519.NQ,832520.NQ,832521.NQ,832528.NQ,832531.NQ,832533.NQ,832534.NQ,832535.NQ,832538.NQ,832541.NQ,832545.NQ,832546.NQ,832547.NQ,832553.NQ,832554.NQ,832557.NQ,832558.NQ,832567.NQ,832568.NQ,832569.NQ,832570.NQ,832572.NQ,832573.NQ,832574.NQ,832575.NQ,832576.NQ,832583.NQ,832591.NQ,832592.NQ,832593.NQ,832594.NQ,832597.NQ,832601.NQ,832602.NQ,832603.NQ,832613.NQ,832614.NQ,832616.NQ,832620.NQ,832621.NQ,832623.NQ,832627.NQ,832629.NQ,832630.NQ,832634.NQ,832637.NQ,832638.NQ,832641.NQ,832642.NQ,832644.NQ,832646.NQ,832647.NQ,832649.NQ,832651.NQ,832652.NQ,832653.NQ,832654.NQ,832655.NQ,832656.NQ,832657.NQ,832658.NQ,832661.NQ,832664.NQ,832665.NQ,832670.NQ,832672.NQ,832673.NQ,832674.NQ,832676.NQ,832682.NQ,832684.NQ,832685.NQ,832687.NQ,832694.NQ,832695.NQ,832696.NQ,832702.NQ,832703.NQ,832705.NQ,832709.NQ,832711.NQ,832713.NQ,832714.NQ,832715.NQ,832718.NQ,832720.NQ,832721.NQ,832722.NQ,832723.NQ,832724.NQ,832725.NQ,832727.NQ,832728.NQ,832730.NQ,832734.NQ,832736.NQ,832737.NQ,832738.NQ,832740.NQ,832749.NQ,832750.NQ,832751.NQ,832753.NQ,832754.NQ,832757.NQ,832763.NQ,832764.NQ,832765.NQ,832767.NQ,832769.NQ,832770.NQ,832774.NQ,832777.NQ,832778.NQ,832780.NQ,832782.NQ,832783.NQ,832784.NQ,832787.NQ,832789.NQ,832790.NQ,832791.NQ,832792.NQ,832793.NQ,832798.NQ,832800.NQ,832807.NQ,832809.NQ,832813.NQ,832814.NQ,832816.NQ,832822.NQ,832825.NQ,832828.NQ,832830.NQ,832832.NQ,832834.NQ,832836.NQ,832839.NQ,832842.NQ,832848.NQ,832849.NQ,832850.NQ,832852.NQ,832853.NQ,832861.NQ,832862.NQ,832863.NQ,832864.NQ,832866.NQ,832867.NQ,832874.NQ,832875.NQ,832879.NQ,832881.NQ,832882.NQ,832888.NQ,832889.NQ,832890.NQ,832891.NQ,832894.NQ,832895.NQ,832902.NQ,832903.NQ,832905.NQ,832906.NQ,832911.NQ,832912.NQ,832914.NQ,832915.NQ,832917.NQ,832918.NQ,832920.NQ,832924.NQ,832925.NQ,832926.NQ,832927.NQ,832928.NQ,832929.NQ,832932.NQ,832935.NQ,832937.NQ,832943.NQ,832944.NQ,832947.NQ,832950.NQ,832952.NQ,832954.NQ,832955.NQ,832956.NQ,832958.NQ,832964.NQ,832966.NQ,832968.NQ,832970.NQ,832972.NQ,832973.NQ,832974.NQ,832978.NQ,832980.NQ,832982.NQ,832985.NQ,832986.NQ,832987.NQ,832988.NQ,832992.NQ,832994.NQ,832997.NQ,833000.NQ,833006.NQ,833009.NQ,833012.NQ,833014.NQ,833016.NQ,833022.NQ,833023.NQ,833024.NQ,833030.NQ,833033.NQ,833035.NQ,833037.NQ,833041.NQ,833042.NQ,833044.NQ,833046.NQ,833047.NQ,833050.NQ,833051.NQ,833052.NQ,833053.NQ,833058.NQ,833060.NQ,833063.NQ,833065.NQ,833068.NQ,833069.NQ,833070.NQ,833072.NQ,833073.NQ,833074.NQ,833077.NQ,833084.NQ,833088.NQ,833091.NQ,833096.NQ,833098.NQ,833099.NQ,833100.NQ,833105.NQ,833110.NQ,833111.NQ,833112.NQ,833115.NQ,833118.NQ,833119.NQ,833120.NQ,833124.NQ,833126.NQ,833127.NQ,833136.NQ,833137.NQ,833151.NQ,833153.NQ,833154.NQ,833158.NQ,833159.NQ,833160.NQ,833161.NQ,833162.NQ,833164.NQ,833165.NQ,833167.NQ,833168.NQ,833177.NQ,833179.NQ,833181.NQ,833189.NQ,833197.NQ,833199.NQ,833200.NQ,833201.NQ,833202.NQ,833204.NQ,833207.NQ,833208.NQ,833211.NQ,833212.NQ,833217.NQ,833218.NQ,833219.NQ,833222.NQ,833224.NQ,833225.NQ,833227.NQ,833228.NQ,833233.NQ,833239.NQ,833240.NQ,833242.NQ,833243.NQ,833244.NQ,833245.NQ,833246.NQ,833248.NQ,833251.NQ,833255.NQ,833256.NQ,833262.NQ,833263.NQ,833265.NQ,833267.NQ,833269.NQ,833277.NQ,833279.NQ,833281.NQ,833282.NQ,833284.NQ,833285.NQ,833286.NQ,833287.NQ,833298.NQ,833300.NQ,833301.NQ,833302.NQ,833303.NQ,833305.NQ,833306.NQ,833307.NQ,833308.NQ,833310.NQ,833316.NQ,833318.NQ,833319.NQ,833320.NQ,833324.NQ,833331.NQ,833332.NQ,833333.NQ,833334.NQ,833341.NQ,833345.NQ,833347.NQ,833349.NQ,833351.NQ,833355.NQ,833356.NQ,833369.NQ,833370.NQ,833374.NQ,833378.NQ,833381.NQ,833383.NQ,833385.NQ,833386.NQ,833390.NQ,833393.NQ,833394.NQ,833396.NQ,833399.NQ,833407.NQ,833408.NQ,833412.NQ,833420.NQ,833421.NQ,833422.NQ,833423.NQ,833426.NQ,833428.NQ,833430.NQ,833432.NQ,833434.NQ,833440.NQ,833443.NQ,833444.NQ,833453.NQ,833455.NQ,833457.NQ,833458.NQ,833462.NQ,833465.NQ,833467.NQ,833470.NQ,833472.NQ,833476.NQ,833477.NQ,833478.NQ,833479.NQ,833481.NQ,833482.NQ,833484.NQ,833485.NQ,833489.NQ,833491.NQ,833492.NQ,833493.NQ,833494.NQ,833495.NQ,833496.NQ,833499.NQ,833502.NQ,833504.NQ,833505.NQ,833507.NQ,833514.NQ,833515.NQ,833524.NQ,833526.NQ,833528.NQ,833532.NQ,833534.NQ,833535.NQ,833538.NQ,833539.NQ,833542.NQ,833543.NQ,833549.NQ,833552.NQ,833555.NQ,833556.NQ,833559.NQ,833562.NQ,833565.NQ,833567.NQ,833569.NQ,833570.NQ,833571.NQ,833572.NQ,833574.NQ,833576.NQ,833582.NQ,833585.NQ,833590.NQ,833594.NQ,833599.NQ,833603.NQ,833606.NQ,833608.NQ,833609.NQ,833611.NQ,833612.NQ,833613.NQ,833614.NQ,833617.NQ,833620.NQ,833622.NQ,833628.NQ,833629.NQ,833631.NQ,833636.NQ,833640.NQ,833648.NQ,833650.NQ,833651.NQ,833653.NQ,833654.NQ,833655.NQ,833658.NQ,833662.NQ,833663.NQ,833669.NQ,833672.NQ,833675.NQ,833680.NQ,833681.NQ,833682.NQ,833686.NQ,833687.NQ,833693.NQ,833696.NQ,833699.NQ,833700.NQ,833703.NQ,833706.NQ,833707.NQ,833711.NQ,833716.NQ,833717.NQ,833719.NQ,833723.NQ,833724.NQ,833726.NQ,833727.NQ,833735.NQ,833736.NQ,833737.NQ,833741.NQ,833746.NQ,833748.NQ,833751.NQ,833753.NQ,833755.NQ,833760.NQ,833763.NQ,833764.NQ,833765.NQ,833768.NQ,833772.NQ,833777.NQ,833778.NQ,833779.NQ,833783.NQ,833784.NQ,833785.NQ,833791.NQ,833796.NQ,833801.NQ,833802.NQ,833804.NQ,833809.NQ,833813.NQ,833814.NQ,833815.NQ,833816.NQ,833818.NQ,833821.NQ,833822.NQ,833824.NQ,833827.NQ,833830.NQ,833833.NQ,833835.NQ,833836.NQ,833838.NQ,833839.NQ,833844.NQ,833846.NQ,833848.NQ,833849.NQ,833855.NQ,833858.NQ,833859.NQ,833863.NQ,833864.NQ,833865.NQ,833866.NQ,833867.NQ,833876.NQ,833881.NQ,833884.NQ,833887.NQ,833893.NQ,833896.NQ,833898.NQ,833899.NQ,833901.NQ,833902.NQ,833903.NQ,833906.NQ,833907.NQ,833908.NQ,833909.NQ,833918.NQ,833921.NQ,833922.NQ,833923.NQ,833925.NQ,833928.NQ,833930.NQ,833934.NQ,833935.NQ,833936.NQ,833937.NQ,833939.NQ,833945.NQ,833954.NQ,833955.NQ,833962.NQ,833966.NQ,833967.NQ,833972.NQ,833976.NQ,833978.NQ,833979.NQ,833980.NQ,833981.NQ,833985.NQ,833988.NQ,833991.NQ,833997.NQ,833998.NQ,833999.NQ,834002.NQ,834005.NQ,834010.NQ,834012.NQ,834016.NQ,834018.NQ,834019.NQ,834026.NQ,834031.NQ,834037.NQ,834038.NQ,834040.NQ,834042.NQ,834043.NQ,834044.NQ,834045.NQ,834046.NQ,834048.NQ,834051.NQ,834054.NQ,834058.NQ,834063.NQ,834065.NQ,834069.NQ,834072.NQ,834073.NQ,834076.NQ,834080.NQ,834081.NQ,834087.NQ,834089.NQ,834092.NQ,834096.NQ,834101.NQ,834104.NQ,834105.NQ,834107.NQ,834110.NQ,834111.NQ,834113.NQ,834114.NQ,834118.NQ,834125.NQ,834126.NQ,834132.NQ,834134.NQ,834135.NQ,834143.NQ,834144.NQ,834146.NQ,834150.NQ,834151.NQ,834156.NQ,834162.NQ,834166.NQ,834167.NQ,834169.NQ,834170.NQ,834171.NQ,834179.NQ,834183.NQ,834184.NQ,834185.NQ,834187.NQ,834189.NQ,834193.NQ,834195.NQ,834196.NQ,834199.NQ,834202.NQ,834205.NQ,834209.NQ,834212.NQ,834213.NQ,834217.NQ,834218.NQ,834221.NQ,834222.NQ,834223.NQ,834233.NQ,834236.NQ,834239.NQ,834240.NQ,834243.NQ,834250.NQ,834253.NQ,834256.NQ,834258.NQ,834259.NQ,834260.NQ,834261.NQ,834263.NQ,834264.NQ,834267.NQ,834270.NQ,834274.NQ,834276.NQ,834277.NQ,834281.NQ,834282.NQ,834284.NQ,834288.NQ,834289.NQ,834290.NQ,834293.NQ,834295.NQ,834296.NQ,834298.NQ,834300.NQ,834306.NQ,834307.NQ,834308.NQ,834315.NQ,834316.NQ,834322.NQ,834323.NQ,834324.NQ,834325.NQ,834326.NQ,834327.NQ,834328.NQ,834329.NQ,834335.NQ,834342.NQ,834343.NQ,834345.NQ,834347.NQ,834348.NQ,834357.NQ,834359.NQ,834361.NQ,834365.NQ,834366.NQ,834367.NQ,834369.NQ,834370.NQ,834372.NQ,834375.NQ,834376.NQ,834377.NQ,834378.NQ,834387.NQ,834395.NQ,834397.NQ,834399.NQ,834401.NQ,834402.NQ,834404.NQ,834406.NQ,834408.NQ,834411.NQ,834412.NQ,834414.NQ,834418.NQ,834422.NQ,834424.NQ,834425.NQ,834426.NQ,834428.NQ,834429.NQ,834440.NQ,834443.NQ,834446.NQ,834465.NQ,834466.NQ,834468.NQ,834469.NQ,834470.NQ,834473.NQ,834476.NQ,834477.NQ,834482.NQ,834483.NQ,834486.NQ,834487.NQ,834489.NQ,834492.NQ,834495.NQ,834497.NQ,834498.NQ,834500.NQ,834503.NQ,834504.NQ,834510.NQ,834515.NQ,834517.NQ,834518.NQ,834519.NQ,834520.NQ,834529.NQ,834530.NQ,834536.NQ,834538.NQ,834539.NQ,834540.NQ,834542.NQ,834543.NQ,834546.NQ,834548.NQ,834549.NQ,834552.NQ,834557.NQ,834558.NQ,834559.NQ,834561.NQ,834562.NQ,834565.NQ,834569.NQ,834572.NQ,834577.NQ,834578.NQ,834579.NQ,834584.NQ,834585.NQ,834587.NQ,834588.NQ,834590.NQ,834591.NQ,834592.NQ,834593.NQ,834594.NQ,834596.NQ,834607.NQ,834609.NQ,834614.NQ,834616.NQ,834618.NQ,834620.NQ,834621.NQ,834623.NQ,834624.NQ,834627.NQ,834630.NQ,834631.NQ,834643.NQ,834644.NQ,834646.NQ,834648.NQ,834650.NQ,834657.NQ,834658.NQ,834667.NQ,834668.NQ,834670.NQ,834677.NQ,834678.NQ,834679.NQ,834683.NQ,834684.NQ,834685.NQ,834686.NQ,834687.NQ,834692.NQ,834696.NQ,834699.NQ,834700.NQ,834701.NQ,834702.NQ,834703.NQ,834706.NQ,834707.NQ,834712.NQ,834713.NQ,834720.NQ,834727.NQ,834728.NQ,834730.NQ,834734.NQ,834738.NQ,834740.NQ,834741.NQ,834743.NQ,834751.NQ,834752.NQ,834755.NQ,834756.NQ,834757.NQ,834759.NQ,834762.NQ,834763.NQ,834764.NQ,834766.NQ,834773.NQ,834774.NQ,834777.NQ,834780.NQ,834781.NQ,834782.NQ,834791.NQ,834792.NQ,834794.NQ,834795.NQ,834797.NQ,834800.NQ,834802.NQ,834803.NQ,834808.NQ,834811.NQ,834812.NQ,834815.NQ,834819.NQ,834822.NQ,834823.NQ,834826.NQ,834832.NQ,834833.NQ,834834.NQ,834843.NQ,834845.NQ,834849.NQ,834853.NQ,834857.NQ,834863.NQ,834866.NQ,834871.NQ,834872.NQ,834874.NQ,834879.NQ,834881.NQ,834882.NQ,834891.NQ,834892.NQ,834896.NQ,834898.NQ,834911.NQ,834912.NQ,834914.NQ,834915.NQ,834916.NQ,834919.NQ,834920.NQ,834921.NQ,834926.NQ,834927.NQ,834928.NQ,834935.NQ,834937.NQ,834938.NQ,834939.NQ,834948.NQ,834951.NQ,834952.NQ,834954.NQ,834956.NQ,834959.NQ,834960.NQ,834965.NQ,834966.NQ,834972.NQ,834974.NQ,834975.NQ,834976.NQ,834978.NQ,834982.NQ,834985.NQ,834990.NQ,834992.NQ,834997.NQ,835000.NQ,835003.NQ,835007.NQ,835011.NQ,835013.NQ,835015.NQ,835017.NQ,835020.NQ,835021.NQ,835026.NQ,835031.NQ,835034.NQ,835037.NQ,835045.NQ,835046.NQ,835047.NQ,835049.NQ,835050.NQ,835051.NQ,835052.NQ,835058.NQ,835061.NQ,835062.NQ,835063.NQ,835067.NQ,835068.NQ,835070.NQ,835071.NQ,835072.NQ,835074.NQ,835075.NQ,835080.NQ,835081.NQ,835084.NQ,835085.NQ,835086.NQ,835087.NQ,835091.NQ,835094.NQ,835101.NQ,835102.NQ,835104.NQ,835105.NQ,835106.NQ,835107.NQ,835111.NQ,835117.NQ,835120.NQ,835126.NQ,835128.NQ,835133.NQ,835135.NQ,835138.NQ,835141.NQ,835152.NQ,835156.NQ,835157.NQ,835159.NQ,835164.NQ,835168.NQ,835169.NQ,835171.NQ,835173.NQ,835175.NQ,835180.NQ,835181.NQ,835187.NQ,835191.NQ,835192.NQ,835193.NQ,835195.NQ,835196.NQ,835198.NQ,835203.NQ,835205.NQ,835208.NQ,835211.NQ,835217.NQ,835222.NQ,835223.NQ,835228.NQ,835230.NQ,835231.NQ,835234.NQ,835244.NQ,835245.NQ,835246.NQ,835250.NQ,835256.NQ,835257.NQ,835266.NQ,835269.NQ,835278.NQ,835281.NQ,835282.NQ,835283.NQ,835286.NQ,835287.NQ,835290.NQ,835291.NQ,835293.NQ,835295.NQ,835296.NQ,835297.NQ,835298.NQ,835300.NQ,835301.NQ,835307.NQ,835311.NQ,835312.NQ,835314.NQ,835316.NQ,835319.NQ,835322.NQ,835323.NQ,835328.NQ,835330.NQ,835333.NQ,835335.NQ,835338.NQ,835340.NQ,835342.NQ,835347.NQ,835352.NQ,835354.NQ,835359.NQ,835361.NQ,835363.NQ,835364.NQ,835365.NQ,835367.NQ,835370.NQ,835372.NQ,835373.NQ,835374.NQ,835375.NQ,835382.NQ,835383.NQ,835387.NQ,835393.NQ,835395.NQ,835401.NQ,835405.NQ,835409.NQ,835410.NQ,835414.NQ,835415.NQ,835416.NQ,835417.NQ,835421.NQ,835425.NQ,835426.NQ,835427.NQ,835429.NQ,835431.NQ,835435.NQ,835438.NQ,835441.NQ,835442.NQ,835445.NQ,835446.NQ,835447.NQ,835448.NQ,835450.NQ,835452.NQ,835453.NQ,835454.NQ,835461.NQ,835463.NQ,835471.NQ,835473.NQ,835474.NQ,835475.NQ,835476.NQ,835487.NQ,835488.NQ,835492.NQ,835495.NQ,835496.NQ,835500.NQ,835505.NQ,835511.NQ,835512.NQ,835513.NQ,835517.NQ,835521.NQ,835525.NQ,835526.NQ,835527.NQ,835532.NQ,835533.NQ,835534.NQ,835536.NQ,835538.NQ,835540.NQ,835541.NQ,835542.NQ,835544.NQ,835545.NQ,835553.NQ,835555.NQ,835558.NQ,835563.NQ,835565.NQ,835567.NQ,835578.NQ,835579.NQ,835582.NQ,835584.NQ,835591.NQ,835595.NQ,835596.NQ,835597.NQ,835599.NQ,835600.NQ,835602.NQ,835605.NQ,835608.NQ,835609.NQ,835616.NQ,835620.NQ,835631.NQ,835632.NQ,835634.NQ,835635.NQ,835636.NQ,835637.NQ,835642.NQ,835647.NQ,835654.NQ,835655.NQ,835659.NQ,835665.NQ,835667.NQ,835669.NQ,835675.NQ,835678.NQ,835679.NQ,835680.NQ,835682.NQ,835684.NQ,835685.NQ,835688.NQ,835690.NQ,835695.NQ,835697.NQ,835698.NQ,835702.NQ,835705.NQ,835708.NQ,835709.NQ,835714.NQ,835718.NQ,835719.NQ,835721.NQ,835723.NQ,835725.NQ,835728.NQ,835729.NQ,835735.NQ,835746.NQ,835751.NQ,835755.NQ,835756.NQ,835759.NQ,835762.NQ,835763.NQ,835764.NQ,835765.NQ,835773.NQ,835775.NQ,835778.NQ,835779.NQ,835785.NQ,835787.NQ,835788.NQ,835791.NQ,835792.NQ,835798.NQ,835800.NQ,835801.NQ,835804.NQ,835805.NQ,835806.NQ,835808.NQ,835812.NQ,835817.NQ,835818.NQ,835827.NQ,835833.NQ,835834.NQ,835839.NQ,835841.NQ,835843.NQ,835847.NQ,835848.NQ,835849.NQ,835850.NQ,835851.NQ,835852.NQ,835853.NQ,835856.NQ,835860.NQ,835861.NQ,835862.NQ,835872.NQ,835877.NQ,835881.NQ,835883.NQ,835888.NQ,835889.NQ,835891.NQ,835894.NQ,835898.NQ,835899.NQ,835901.NQ,835906.NQ,835907.NQ,835909.NQ,835910.NQ,835911.NQ,835912.NQ,835913.NQ,835914.NQ,835924.NQ,835925.NQ,835927.NQ,835936.NQ,835940.NQ,835947.NQ,835948.NQ,835957.NQ,835959.NQ,835960.NQ,835961.NQ,835963.NQ,835968.NQ,835969.NQ,835970.NQ,835979.NQ,835982.NQ,835983.NQ,835987.NQ,835993.NQ,835995.NQ,835996.NQ,836002.NQ,836004.NQ,836005.NQ,836008.NQ,836011.NQ,836012.NQ,836014.NQ,836015.NQ,836024.NQ,836027.NQ,836028.NQ,836029.NQ,836030.NQ,836032.NQ,836033.NQ,836036.NQ,836041.NQ,836047.NQ,836050.NQ,836056.NQ,836058.NQ,836059.NQ,836062.NQ,836070.NQ,836072.NQ,836073.NQ,836075.NQ,836076.NQ,836078.NQ,836081.NQ,836089.NQ,836093.NQ,836098.NQ,836100.NQ,836106.NQ,836108.NQ,836109.NQ,836113.NQ,836115.NQ,836117.NQ,836118.NQ,836122.NQ,836123.NQ,836125.NQ,836126.NQ,836129.NQ,836136.NQ,836138.NQ,836139.NQ,836145.NQ,836153.NQ,836156.NQ,836159.NQ,836160.NQ,836161.NQ,836162.NQ,836163.NQ,836164.NQ,836166.NQ,836170.NQ,836171.NQ,836179.NQ,836183.NQ,836185.NQ,836186.NQ,836189.NQ,836190.NQ,836191.NQ,836193.NQ,836195.NQ,836196.NQ,836205.NQ,836206.NQ,836208.NQ,836209.NQ,836210.NQ,836212.NQ,836215.NQ,836216.NQ,836217.NQ,836221.NQ,836225.NQ,836228.NQ,836230.NQ,836234.NQ,836235.NQ,836236.NQ,836237.NQ,836241.NQ,836248.NQ,836249.NQ,836253.NQ,836254.NQ,836255.NQ,836256.NQ,836259.NQ,836261.NQ,836264.NQ,836266.NQ,836274.NQ,836275.NQ,836277.NQ,836278.NQ,836281.NQ,836284.NQ,836286.NQ,836287.NQ,836291.NQ,836297.NQ,836300.NQ,836303.NQ,836305.NQ,836312.NQ,836314.NQ,836315.NQ,836316.NQ,836318.NQ,836319.NQ,836321.NQ,836325.NQ,836329.NQ,836331.NQ,836332.NQ,836333.NQ,836337.NQ,836339.NQ,836342.NQ,836344.NQ,836347.NQ,836348.NQ,836357.NQ,836362.NQ,836364.NQ,836371.NQ,836373.NQ,836374.NQ,836377.NQ,836378.NQ,836381.NQ,836384.NQ,836386.NQ,836388.NQ,836389.NQ,836390.NQ,836392.NQ,836396.NQ,836399.NQ,836405.NQ,836409.NQ,836415.NQ,836417.NQ,836418.NQ,836419.NQ,836421.NQ,836425.NQ,836432.NQ,836437.NQ,836439.NQ,836440.NQ,836442.NQ,836443.NQ,836450.NQ,836451.NQ,836452.NQ,836453.NQ,836455.NQ,836456.NQ,836458.NQ,836459.NQ,836460.NQ,836464.NQ,836468.NQ,836472.NQ,836475.NQ,836476.NQ,836477.NQ,836479.NQ,836482.NQ,836491.NQ,836503.NQ,836504.NQ,836507.NQ,836508.NQ,836509.NQ,836514.NQ,836521.NQ,836523.NQ,836530.NQ,836531.NQ,836533.NQ,836537.NQ,836542.NQ,836545.NQ,836547.NQ,836550.NQ,836552.NQ,836556.NQ,836557.NQ,836560.NQ,836562.NQ,836572.NQ,836575.NQ,836577.NQ,836579.NQ,836580.NQ,836583.NQ,836584.NQ,836589.NQ,836590.NQ,836592.NQ,836595.NQ,836596.NQ,836597.NQ,836602.NQ,836603.NQ,836605.NQ,836609.NQ,836610.NQ,836613.NQ,836617.NQ,836618.NQ,836619.NQ,836624.NQ,836625.NQ,836632.NQ,836635.NQ,836636.NQ,836639.NQ,836641.NQ,836642.NQ,836645.NQ,836647.NQ,836649.NQ,836650.NQ,836652.NQ,836657.NQ,836658.NQ,836660.NQ,836663.NQ,836665.NQ,836666.NQ,836667.NQ,836668.NQ,836669.NQ,836670.NQ,836674.NQ,836678.NQ,836679.NQ,836683.NQ,836688.NQ,836690.NQ,836691.NQ,836692.NQ,836696.NQ,836699.NQ,836703.NQ,836708.NQ,836715.NQ,836717.NQ,836719.NQ,836727.NQ,836733.NQ,836734.NQ,836735.NQ,836741.NQ,836742.NQ,836743.NQ,836744.NQ,836745.NQ,836746.NQ,836753.NQ,836755.NQ,836758.NQ,836761.NQ,836770.NQ,836772.NQ,836774.NQ,836775.NQ,836777.NQ,836779.NQ,836780.NQ,836782.NQ,836783.NQ,836786.NQ,836790.NQ,836792.NQ,836793.NQ,836797.NQ,836803.NQ,836805.NQ,836809.NQ,836810.NQ,836811.NQ,836816.NQ,836818.NQ,836820.NQ,836823.NQ,836825.NQ,836827.NQ,836829.NQ,836830.NQ,836831.NQ,836834.NQ,836836.NQ,836837.NQ,836843.NQ,836846.NQ,836847.NQ,836851.NQ,836858.NQ,836861.NQ,836863.NQ,836864.NQ,836866.NQ,836867.NQ,836870.NQ,836872.NQ,836874.NQ,836876.NQ,836877.NQ,836879.NQ,836881.NQ,836883.NQ,836885.NQ,836888.NQ,836890.NQ,836893.NQ,836898.NQ,836903.NQ,836907.NQ,836908.NQ,836909.NQ,836910.NQ,836911.NQ,836913.NQ,836915.NQ,836924.NQ,836930.NQ,836932.NQ,836935.NQ,836939.NQ,836941.NQ,836944.NQ,836946.NQ,836949.NQ,836954.NQ,836955.NQ,836961.NQ,836967.NQ,836972.NQ,836974.NQ,836976.NQ,836977.NQ,836978.NQ,836979.NQ,836981.NQ,836984.NQ,836989.NQ,836991.NQ,836994.NQ,836998.NQ,837000.NQ,837003.NQ,837006.NQ,837008.NQ,837009.NQ,837011.NQ,837012.NQ,837014.NQ,837017.NQ,837018.NQ,837019.NQ,837021.NQ,837023.NQ,837033.NQ,837034.NQ,837035.NQ,837036.NQ,837040.NQ,837042.NQ,837044.NQ,837051.NQ,837052.NQ,837053.NQ,837058.NQ,837062.NQ,837063.NQ,837065.NQ,837067.NQ,837070.NQ,837072.NQ,837073.NQ,837074.NQ,837078.NQ,837081.NQ,837083.NQ,837087.NQ,837089.NQ,837090.NQ,837094.NQ,837095.NQ,837098.NQ,837105.NQ,837106.NQ,837108.NQ,837125.NQ,837126.NQ,837129.NQ,837133.NQ,837134.NQ,837135.NQ,837136.NQ,837137.NQ,837138.NQ,837143.NQ,837146.NQ,837148.NQ,837150.NQ,837153.NQ,837157.NQ,837173.NQ,837174.NQ,837177.NQ,837178.NQ,837180.NQ,837181.NQ,837185.NQ,837187.NQ,837189.NQ,837193.NQ,837200.NQ,837201.NQ,837207.NQ,837210.NQ,837211.NQ,837215.NQ,837217.NQ,837220.NQ,837222.NQ,837224.NQ,837225.NQ,837226.NQ,837227.NQ,837228.NQ,837231.NQ,837232.NQ,837236.NQ,837251.NQ,837253.NQ,837254.NQ,837256.NQ,837261.NQ,837263.NQ,837264.NQ,837269.NQ,837273.NQ,837274.NQ,837276.NQ,837283.NQ,837284.NQ,837287.NQ,837288.NQ,837289.NQ,837291.NQ,837298.NQ,837299.NQ,837301.NQ,837302.NQ,837303.NQ,837304.NQ,837305.NQ,837308.NQ,837312.NQ,837314.NQ,837317.NQ,837318.NQ,837319.NQ,837320.NQ,837323.NQ,837324.NQ,837326.NQ,837327.NQ,837334.NQ,837336.NQ,837340.NQ,837341.NQ,837345.NQ,837347.NQ,837348.NQ,837350.NQ,837351.NQ,837353.NQ,837355.NQ,837363.NQ,837368.NQ,837370.NQ,837371.NQ,837372.NQ,837373.NQ,837375.NQ,837377.NQ,837378.NQ,837383.NQ,837384.NQ,837385.NQ,837388.NQ,837392.NQ,837393.NQ,837396.NQ,837399.NQ,837402.NQ,837403.NQ,837404.NQ,837406.NQ,837408.NQ,837424.NQ,837429.NQ,837430.NQ,837432.NQ,837441.NQ,837443.NQ,837445.NQ,837447.NQ,837450.NQ,837453.NQ,837455.NQ,837459.NQ,837461.NQ,837477.NQ,837479.NQ,837487.NQ,837491.NQ,837492.NQ,837494.NQ,837497.NQ,837506.NQ,837509.NQ,837510.NQ,837511.NQ,837512.NQ,837519.NQ,837522.NQ,837523.NQ,837524.NQ,837525.NQ,837526.NQ,837527.NQ,837528.NQ,837530.NQ,837531.NQ,837535.NQ,837549.NQ,837550.NQ,837552.NQ,837555.NQ,837557.NQ,837558.NQ,837560.NQ,837561.NQ,837566.NQ,837567.NQ,837569.NQ,837574.NQ,837576.NQ,837577.NQ,837579.NQ,837580.NQ,837581.NQ,837585.NQ,837588.NQ,837591.NQ,837592.NQ,837595.NQ,837601.NQ,837607.NQ,837608.NQ,837609.NQ,837611.NQ,837614.NQ,837617.NQ,837619.NQ,837620.NQ,837630.NQ,837631.NQ,837633.NQ,837637.NQ,837641.NQ,837644.NQ,837646.NQ,837651.NQ,837653.NQ,837654.NQ,837656.NQ,837661.NQ,837662.NQ,837664.NQ,837666.NQ,837667.NQ,837668.NQ,837670.NQ,837674.NQ,837680.NQ,837681.NQ,837685.NQ,837687.NQ,837690.NQ,837691.NQ,837694.NQ,837695.NQ,837697.NQ,837698.NQ,837699.NQ,837701.NQ,837704.NQ,837707.NQ,837710.NQ,837711.NQ,837718.NQ,837719.NQ,837720.NQ,837721.NQ,837726.NQ,837728.NQ,837729.NQ,837733.NQ,837734.NQ,837741.NQ,837742.NQ,837743.NQ,837745.NQ,837747.NQ,837748.NQ,837749.NQ,837758.NQ,837761.NQ,837766.NQ,837768.NQ,837773.NQ,837777.NQ,837779.NQ,837782.NQ,837784.NQ,837785.NQ,837793.NQ,837796.NQ,837798.NQ,837809.NQ,837810.NQ,837814.NQ,837815.NQ,837816.NQ,837819.NQ,837823.NQ,837826.NQ,837831.NQ,837833.NQ,837835.NQ,837836.NQ,837837.NQ,837838.NQ,837840.NQ,837845.NQ,837851.NQ,837852.NQ,837856.NQ,837857.NQ,837863.NQ,837864.NQ,837866.NQ,837867.NQ,837868.NQ,837869.NQ,837870.NQ,837874.NQ,837877.NQ,837879.NQ,837880.NQ,837881.NQ,837887.NQ,837889.NQ,837891.NQ,837892.NQ,837893.NQ,837894.NQ,837899.NQ,837907.NQ,837912.NQ,837918.NQ,837919.NQ,837920.NQ,837922.NQ,837925.NQ,837927.NQ,837929.NQ,837931.NQ,837932.NQ,837934.NQ,837935.NQ,837936.NQ,837940.NQ,837941.NQ,837949.NQ,837950.NQ,837952.NQ,837953.NQ,837955.NQ,837961.NQ,837965.NQ,837967.NQ,837968.NQ,837971.NQ,837977.NQ,837979.NQ,837980.NQ,837982.NQ,837983.NQ,837986.NQ,837988.NQ,837989.NQ,837991.NQ,837993.NQ,837995.NQ,837997.NQ,837998.NQ,838010.NQ,838012.NQ,838017.NQ,838020.NQ,838028.NQ,838029.NQ,838031.NQ,838032.NQ,838033.NQ,838036.NQ,838041.NQ,838042.NQ,838046.NQ,838049.NQ,838054.NQ,838060.NQ,838064.NQ,838067.NQ,838070.NQ,838071.NQ,838072.NQ,838075.NQ,838078.NQ,838082.NQ,838089.NQ,838091.NQ,838092.NQ,838093.NQ,838094.NQ,838095.NQ,838097.NQ,838099.NQ,838102.NQ,838103.NQ,838104.NQ,838109.NQ,838114.NQ,838115.NQ,838116.NQ,838118.NQ,838122.NQ,838128.NQ,838129.NQ,838131.NQ,838137.NQ,838140.NQ,838141.NQ,838146.NQ,838149.NQ,838150.NQ,838152.NQ,838153.NQ,838158.NQ,838161.NQ,838162.NQ,838164.NQ,838168.NQ,838172.NQ,838174.NQ,838176.NQ,838182.NQ,838183.NQ,838184.NQ,838187.NQ,838190.NQ,838194.NQ,838196.NQ,838197.NQ,838203.NQ,838204.NQ,838209.NQ,838211.NQ,838213.NQ,838214.NQ,838221.NQ,838222.NQ,838236.NQ,838238.NQ,838239.NQ,838240.NQ,838241.NQ,838242.NQ,838243.NQ,838246.NQ,838247.NQ,838248.NQ,838250.NQ,838251.NQ,838257.NQ,838260.NQ,838264.NQ,838268.NQ,838269.NQ,838273.NQ,838274.NQ,838279.NQ,838282.NQ,838283.NQ,838284.NQ,838288.NQ,838293.NQ,838296.NQ,838297.NQ,838300.NQ,838309.NQ,838310.NQ,838312.NQ,838313.NQ,838314.NQ,838316.NQ,838319.NQ,838322.NQ,838324.NQ,838327.NQ,838328.NQ,838330.NQ,838331.NQ,838332.NQ,838334.NQ,838335.NQ,838337.NQ,838344.NQ,838351.NQ,838354.NQ,838355.NQ,838357.NQ,838362.NQ,838363.NQ,838364.NQ,838365.NQ,838366.NQ,838369.NQ,838372.NQ,838374.NQ,838379.NQ,838381.NQ,838384.NQ,838386.NQ,838388.NQ,838392.NQ,838394.NQ,838395.NQ,838399.NQ,838404.NQ,838408.NQ,838409.NQ,838410.NQ,838411.NQ,838413.NQ,838414.NQ,838415.NQ,838418.NQ,838419.NQ,838425.NQ,838426.NQ,838428.NQ,838430.NQ,838432.NQ,838440.NQ,838443.NQ,838448.NQ,838449.NQ,838451.NQ,838453.NQ,838456.NQ,838457.NQ,838458.NQ,838460.NQ,838463.NQ,838464.NQ,838469.NQ,838470.NQ,838471.NQ,838474.NQ,838476.NQ,838477.NQ,838478.NQ,838479.NQ,838481.NQ,838482.NQ,838483.NQ,838484.NQ,838486.NQ,838491.NQ,838497.NQ,838498.NQ,838501.NQ,838502.NQ,838504.NQ,838508.NQ,838511.NQ,838512.NQ,838514.NQ,838518.NQ,838519.NQ,838522.NQ,838523.NQ,838526.NQ,838529.NQ,838531.NQ,838533.NQ,838535.NQ,838536.NQ,838537.NQ,838541.NQ,838542.NQ,838547.NQ,838549.NQ,838550.NQ,838553.NQ,838558.NQ,838559.NQ,838561.NQ,838563.NQ,838564.NQ,838566.NQ,838569.NQ,838570.NQ,838571.NQ,838573.NQ,838574.NQ,838576.NQ,838578.NQ,838582.NQ,838584.NQ,838587.NQ,838589.NQ,838590.NQ,838593.NQ,838595.NQ,838597.NQ,838598.NQ,838602.NQ,838606.NQ,838615.NQ,838616.NQ,838618.NQ,838623.NQ,838624.NQ,838626.NQ,838629.NQ,838632.NQ,838635.NQ,838637.NQ,838644.NQ,838646.NQ,838647.NQ,838649.NQ,838651.NQ,838654.NQ,838655.NQ,838656.NQ,838659.NQ,838661.NQ,838664.NQ,838665.NQ,838667.NQ,838669.NQ,838671.NQ,838672.NQ,838675.NQ,838686.NQ,838687.NQ,838691.NQ,838693.NQ,838694.NQ,838696.NQ,838701.NQ,838704.NQ,838707.NQ,838708.NQ,838712.NQ,838714.NQ,838721.NQ,838723.NQ,838726.NQ,838729.NQ,838736.NQ,838737.NQ,838740.NQ,838741.NQ,838744.NQ,838749.NQ,838752.NQ,838753.NQ,838754.NQ,838756.NQ,838757.NQ,838758.NQ,838760.NQ,838762.NQ,838766.NQ,838767.NQ,838768.NQ,838770.NQ,838775.NQ,838784.NQ,838786.NQ,838790.NQ,838791.NQ,838793.NQ,838795.NQ,838796.NQ,838798.NQ,838799.NQ,838800.NQ,838801.NQ,838802.NQ,838803.NQ,838804.NQ,838807.NQ,838809.NQ,838811.NQ,838812.NQ,838813.NQ,838814.NQ,838823.NQ,838826.NQ,838829.NQ,838834.NQ,838835.NQ,838837.NQ,838840.NQ,838841.NQ,838843.NQ,838850.NQ,838854.NQ,838856.NQ,838857.NQ,838858.NQ,838861.NQ,838865.NQ,838866.NQ,838868.NQ,838875.NQ,838878.NQ,838879.NQ,838883.NQ,838893.NQ,838894.NQ,838897.NQ,838898.NQ,838899.NQ,838900.NQ,838901.NQ,838903.NQ,838906.NQ,838909.NQ,838910.NQ,838911.NQ,838915.NQ,838916.NQ,838917.NQ,838925.NQ,838927.NQ,838928.NQ,838933.NQ,838934.NQ,838939.NQ,838940.NQ,838941.NQ,838943.NQ,838944.NQ,838949.NQ,838951.NQ,838952.NQ,838955.NQ,838956.NQ,838957.NQ,838959.NQ,838961.NQ,838970.NQ,838978.NQ,838980.NQ,838981.NQ,838982.NQ,838983.NQ,838984.NQ,838987.NQ,838989.NQ,838990.NQ,838992.NQ,838994.NQ,838998.NQ,839003.NQ,839005.NQ,839009.NQ,839010.NQ,839012.NQ,839013.NQ,839018.NQ,839024.NQ,839028.NQ,839029.NQ,839030.NQ,839031.NQ,839032.NQ,839033.NQ,839035.NQ,839036.NQ,839037.NQ,839038.NQ,839041.NQ,839044.NQ,839045.NQ,839047.NQ,839054.NQ,839056.NQ,839058.NQ,839067.NQ,839071.NQ,839073.NQ,839074.NQ,839076.NQ,839080.NQ,839081.NQ,839084.NQ,839085.NQ,839086.NQ,839089.NQ,839093.NQ,839099.NQ,839101.NQ,839109.NQ,839110.NQ,839113.NQ,839114.NQ,839118.NQ,839120.NQ,839121.NQ,839123.NQ,839125.NQ,839131.NQ,839132.NQ,839133.NQ,839137.NQ,839139.NQ,839140.NQ,839142.NQ,839143.NQ,839145.NQ,839146.NQ,839148.NQ,839152.NQ,839153.NQ,839154.NQ,839155.NQ,839156.NQ,839161.NQ,839163.NQ,839164.NQ,839173.NQ,839174.NQ,839175.NQ,839178.NQ,839180.NQ,839181.NQ,839182.NQ,839187.NQ,839188.NQ,839189.NQ,839190.NQ,839192.NQ,839193.NQ,839196.NQ,839198.NQ,839199.NQ,839201.NQ,839202.NQ,839204.NQ,839205.NQ,839206.NQ,839207.NQ,839209.NQ,839211.NQ,839213.NQ,839216.NQ,839218.NQ,839220.NQ,839222.NQ,839223.NQ,839224.NQ,839226.NQ,839228.NQ,839229.NQ,839231.NQ,839232.NQ,839233.NQ,839237.NQ,839239.NQ,839241.NQ,839244.NQ,839253.NQ,839254.NQ,839255.NQ,839258.NQ,839260.NQ,839266.NQ,839270.NQ,839275.NQ,839277.NQ,839278.NQ,839279.NQ,839281.NQ,839284.NQ,839285.NQ,839289.NQ,839291.NQ,839292.NQ,839295.NQ,839296.NQ,839302.NQ,839305.NQ,839306.NQ,839308.NQ,839309.NQ,839310.NQ,839312.NQ,839315.NQ,839319.NQ,839321.NQ,839322.NQ,839324.NQ,839326.NQ,839327.NQ,839330.NQ,839332.NQ,839335.NQ,839339.NQ,839340.NQ,839341.NQ,839342.NQ,839350.NQ,839351.NQ,839354.NQ,839355.NQ,839356.NQ,839357.NQ,839362.NQ,839364.NQ,839365.NQ,839367.NQ,839370.NQ,839373.NQ,839376.NQ,839379.NQ,839381.NQ,839385.NQ,839387.NQ,839389.NQ,839391.NQ,839392.NQ,839395.NQ,839406.NQ,839409.NQ,839410.NQ,839411.NQ,839416.NQ,839418.NQ,839421.NQ,839432.NQ,839433.NQ,839435.NQ,839438.NQ,839443.NQ,839444.NQ,839446.NQ,839447.NQ,839452.NQ,839453.NQ,839454.NQ,839458.NQ,839461.NQ,839463.NQ,839466.NQ,839468.NQ,839470.NQ,839472.NQ,839476.NQ,839477.NQ,839481.NQ,839482.NQ,839484.NQ,839485.NQ,839489.NQ,839490.NQ,839491.NQ,839493.NQ,839497.NQ,839499.NQ,839501.NQ,839502.NQ,839503.NQ,839506.NQ,839507.NQ,839509.NQ,839511.NQ,839513.NQ,839514.NQ,839515.NQ,839517.NQ,839520.NQ,839523.NQ,839531.NQ,839532.NQ,839535.NQ,839537.NQ,839539.NQ,839540.NQ,839545.NQ,839546.NQ,839547.NQ,839549.NQ,839553.NQ,839555.NQ,839558.NQ,839559.NQ,839560.NQ,839561.NQ,839563.NQ,839567.NQ,839568.NQ,839572.NQ,839574.NQ,839581.NQ,839582.NQ,839583.NQ,839587.NQ,839589.NQ,839593.NQ,839596.NQ,839599.NQ,839605.NQ,839606.NQ,839609.NQ,839610.NQ,839612.NQ,839613.NQ,839618.NQ,839621.NQ,839622.NQ,839625.NQ,839629.NQ,839630.NQ,839631.NQ,839633.NQ,839635.NQ,839640.NQ,839642.NQ,839643.NQ,839646.NQ,839649.NQ,839657.NQ,839659.NQ,839666.NQ,839667.NQ,839668.NQ,839672.NQ,839675.NQ,839679.NQ,839681.NQ,839683.NQ,839684.NQ,839686.NQ,839692.NQ,839694.NQ,839696.NQ,839697.NQ,839698.NQ,839707.NQ,839710.NQ,839713.NQ,839717.NQ,839718.NQ,839719.NQ,839720.NQ,839723.NQ,839726.NQ,839731.NQ,839732.NQ,839733.NQ,839740.NQ,839742.NQ,839745.NQ,839746.NQ,839750.NQ,839751.NQ,839754.NQ,839755.NQ,839756.NQ,839759.NQ,839762.NQ,839765.NQ,839768.NQ,839769.NQ,839770.NQ,839773.NQ,839774.NQ,839775.NQ,839777.NQ,839778.NQ,839780.NQ,839782.NQ,839783.NQ,839785.NQ,839786.NQ,839788.NQ,839792.NQ,839796.NQ,839799.NQ,839801.NQ,839805.NQ,839806.NQ,839808.NQ,839809.NQ,839813.NQ,839817.NQ,839820.NQ,839821.NQ,839822.NQ,839825.NQ,839826.NQ,839830.NQ,839833.NQ,839834.NQ,839836.NQ,839837.NQ,839838.NQ,839839.NQ,839840.NQ,839841.NQ,839842.NQ,839845.NQ,839847.NQ,839853.NQ,839856.NQ,839857.NQ,839865.NQ,839867.NQ,839870.NQ,839871.NQ,839874.NQ,839879.NQ,839880.NQ,839881.NQ,839884.NQ,839888.NQ,839890.NQ,839893.NQ,839894.NQ,839898.NQ,839901.NQ,839904.NQ,839907.NQ,839909.NQ,839911.NQ,839913.NQ,839914.NQ,839915.NQ,839917.NQ,839920.NQ,839922.NQ,839926.NQ,839929.NQ,839934.NQ,839938.NQ,839939.NQ,839942.NQ,839943.NQ,839944.NQ,839951.NQ,839953.NQ,839954.NQ,839962.NQ,839964.NQ,839965.NQ,839966.NQ,839967.NQ,839969.NQ,839977.NQ,839979.NQ,839980.NQ,839981.NQ,839982.NQ,839983.NQ,839984.NQ,839985.NQ,839986.NQ,839987.NQ,839990.NQ,839993.NQ,839994.NQ,839998.NQ,839999.NQ,870000.NQ,870001.NQ,870003.NQ,870005.NQ,870009.NQ,870010.NQ,870014.NQ,870016.NQ,870020.NQ,870021.NQ,870024.NQ,870026.NQ,870029.NQ,870030.NQ,870031.NQ,870037.NQ,870044.NQ,870046.NQ,870049.NQ,870050.NQ,870052.NQ,870053.NQ,870055.NQ,870058.NQ,870061.NQ,870065.NQ,870066.NQ,870070.NQ,870071.NQ,870072.NQ,870073.NQ,870079.NQ,870080.NQ,870086.NQ,870092.NQ,870095.NQ,870096.NQ,870098.NQ,870099.NQ,870100.NQ,870101.NQ,870102.NQ,870104.NQ,870110.NQ,870111.NQ,870115.NQ,870116.NQ,870119.NQ,870122.NQ,870123.NQ,870125.NQ,870126.NQ,870130.NQ,870131.NQ,870134.NQ,870139.NQ,870140.NQ,870141.NQ,870144.NQ,870145.NQ,870149.NQ,870150.NQ,870154.NQ,870157.NQ,870158.NQ,870160.NQ,870164.NQ,870166.NQ,870167.NQ,870170.NQ,870172.NQ,870175.NQ,870176.NQ,870177.NQ,870178.NQ,870182.NQ,870183.NQ,870186.NQ,870188.NQ,870191.NQ,870195.NQ,870198.NQ,870202.NQ,870208.NQ,870212.NQ,870213.NQ,870215.NQ,870219.NQ,870220.NQ,870221.NQ,870223.NQ,870225.NQ,870227.NQ,870229.NQ,870231.NQ,870232.NQ,870233.NQ,870235.NQ,870236.NQ,870238.NQ,870250.NQ,870251.NQ,870253.NQ,870256.NQ,870259.NQ,870260.NQ,870262.NQ,870263.NQ,870264.NQ,870268.NQ,870270.NQ,870271.NQ,870275.NQ,870277.NQ,870278.NQ,870281.NQ,870282.NQ,870283.NQ,870284.NQ,870286.NQ,870287.NQ,870288.NQ,870292.NQ,870293.NQ,870294.NQ,870296.NQ,870298.NQ,870301.NQ,870302.NQ,870304.NQ,870305.NQ,870307.NQ,870308.NQ,870309.NQ,870316.NQ,870317.NQ,870318.NQ,870319.NQ,870323.NQ,870324.NQ,870328.NQ,870331.NQ,870334.NQ,870335.NQ,870336.NQ,870337.NQ,870343.NQ,870345.NQ,870350.NQ,870353.NQ,870355.NQ,870356.NQ,870358.NQ,870359.NQ,870361.NQ,870362.NQ,870364.NQ,870365.NQ,870366.NQ,870368.NQ,870369.NQ,870370.NQ,870371.NQ,870372.NQ,870374.NQ,870376.NQ,870382.NQ,870383.NQ,870386.NQ,870388.NQ,870389.NQ,870390.NQ,870395.NQ,870398.NQ,870405.NQ,870406.NQ,870409.NQ,870411.NQ,870412.NQ,870415.NQ,870417.NQ,870418.NQ,870419.NQ,870420.NQ,870421.NQ,870426.NQ,870427.NQ,870428.NQ,870429.NQ,870431.NQ,870432.NQ,870437.NQ,870438.NQ,870440.NQ,870441.NQ,870443.NQ,870446.NQ,870448.NQ,870450.NQ,870451.NQ,870452.NQ,870453.NQ,870457.NQ,870459.NQ,870470.NQ,870472.NQ,870475.NQ,870480.NQ,870482.NQ,870483.NQ,870487.NQ,870488.NQ,870489.NQ,870491.NQ,870492.NQ,870494.NQ,870496.NQ,870500.NQ,870502.NQ,870505.NQ,870506.NQ,870507.NQ,870511.NQ,870512.NQ,870519.NQ,870523.NQ,870524.NQ,870529.NQ,870530.NQ,870531.NQ,870532.NQ,870534.NQ,870535.NQ,870536.NQ,870538.NQ,870539.NQ,870542.NQ,870543.NQ,870544.NQ,870548.NQ,870549.NQ,870550.NQ,870551.NQ,870552.NQ,870553.NQ,870555.NQ,870556.NQ,870557.NQ,870558.NQ,870559.NQ,870566.NQ,870570.NQ,870571.NQ,870572.NQ,870573.NQ,870579.NQ,870581.NQ,870583.NQ,870586.NQ,870592.NQ,870593.NQ,870600.NQ,870601.NQ,870603.NQ,870605.NQ,870608.NQ,870614.NQ,870615.NQ,870618.NQ,870619.NQ,870620.NQ,870621.NQ,870622.NQ,870623.NQ,870624.NQ,870625.NQ,870626.NQ,870627.NQ,870629.NQ,870632.NQ,870633.NQ,870635.NQ,870637.NQ,870643.NQ,870646.NQ,870647.NQ,870652.NQ,870654.NQ,870656.NQ,870659.NQ,870662.NQ,870667.NQ,870669.NQ,870678.NQ,870679.NQ,870682.NQ,870686.NQ,870689.NQ,870690.NQ,870691.NQ,870693.NQ,870696.NQ,870697.NQ,870702.NQ,870706.NQ,870707.NQ,870710.NQ,870711.NQ,870712.NQ,870714.NQ,870716.NQ,870717.NQ,870719.NQ,870724.NQ,870725.NQ,870726.NQ,870727.NQ,870730.NQ,870733.NQ,870736.NQ,870737.NQ,870746.NQ,870749.NQ,870752.NQ,870753.NQ,870754.NQ,870756.NQ,870757.NQ,870760.NQ,870762.NQ,870764.NQ,870768.NQ,870772.NQ,870774.NQ,870775.NQ,870777.NQ,870779.NQ,870780.NQ,870781.NQ,870783.NQ,870784.NQ,870787.NQ,870788.NQ,870790.NQ,870792.NQ,870796.NQ,870797.NQ,870799.NQ,870800.NQ,870801.NQ,870802.NQ,870804.NQ,870805.NQ,870806.NQ,870810.NQ,870811.NQ,870812.NQ,870814.NQ,870815.NQ,870818.NQ,870819.NQ,870820.NQ,870821.NQ,870822.NQ,870825.NQ,870832.NQ,870834.NQ,870838.NQ,870840.NQ,870841.NQ,870844.NQ,870847.NQ,870848.NQ,870849.NQ,870851.NQ,870853.NQ,870855.NQ,870856.NQ,870859.NQ,870861.NQ,870869.NQ,870870.NQ,870874.NQ,870875.NQ,870876.NQ,870877.NQ,870879.NQ,870880.NQ,870881.NQ,870884.NQ,870886.NQ,870887.NQ,870888.NQ,870889.NQ,870890.NQ,870891.NQ,870892.NQ,870893.NQ,870895.NQ,870897.NQ,870899.NQ,870906.NQ,870909.NQ,870911.NQ,870912.NQ,870914.NQ,870916.NQ,870917.NQ,870924.NQ,870925.NQ,870931.NQ,870935.NQ,870936.NQ,870952.NQ,870953.NQ,870955.NQ,870957.NQ,870958.NQ,870959.NQ,870961.NQ,870962.NQ,870966.NQ,870968.NQ,870970.NQ,870972.NQ,870976.NQ,870978.NQ,870979.NQ,870981.NQ,870984.NQ,870986.NQ,870990.NQ,870991.NQ,870992.NQ,870996.NQ,870997.NQ,871002.NQ,871003.NQ,871005.NQ,871008.NQ,871012.NQ,871013.NQ,871014.NQ,871019.NQ,871022.NQ,871023.NQ,871028.NQ,871037.NQ,871039.NQ,871044.NQ,871047.NQ,871048.NQ,871056.NQ,871060.NQ,871064.NQ,871073.NQ,871074.NQ,871075.NQ,871076.NQ,871080.NQ,871081.NQ,871082.NQ,871084.NQ,871086.NQ,871088.NQ,871090.NQ,871091.NQ,871093.NQ,871095.NQ,871097.NQ,871098.NQ,871101.NQ,871102.NQ,871107.NQ,871108.NQ,871109.NQ,871122.NQ,871124.NQ,871125.NQ,871126.NQ,871132.NQ,871135.NQ,871136.NQ,871137.NQ,871138.NQ,871140.NQ,871147.NQ,871148.NQ,871149.NQ,871158.NQ,871163.NQ,871166.NQ,871167.NQ,871168.NQ,871169.NQ,871171.NQ,871172.NQ,871174.NQ,871176.NQ,871177.NQ,871178.NQ,871181.NQ,871183.NQ,871185.NQ,871186.NQ,871190.NQ,871192.NQ,871194.NQ,871195.NQ,871197.NQ,871198.NQ,871200.NQ,871201.NQ,871206.NQ,871215.NQ,871222.NQ,871223.NQ,871225.NQ,871226.NQ,871228.NQ,871229.NQ,871230.NQ,871231.NQ,871232.NQ,871235.NQ,871239.NQ,871241.NQ,871242.NQ,871244.NQ,871248.NQ,871252.NQ,871255.NQ,871258.NQ,871260.NQ,871261.NQ,871262.NQ,871263.NQ,871264.NQ,871271.NQ,871272.NQ,871274.NQ,871275.NQ,871278.NQ,871280.NQ,871281.NQ,871282.NQ,871283.NQ,871289.NQ,871291.NQ,871294.NQ,871295.NQ,871296.NQ,871297.NQ,871298.NQ,871301.NQ,871304.NQ,871305.NQ,871308.NQ,871310.NQ,871311.NQ,871312.NQ,871313.NQ,871314.NQ,871315.NQ,871318.NQ,871320.NQ,871323.NQ,871324.NQ,871325.NQ,871329.NQ,871330.NQ,871332.NQ,871334.NQ,871335.NQ,871336.NQ,871338.NQ,871339.NQ,871340.NQ,871342.NQ,871343.NQ,871346.NQ,871348.NQ,871349.NQ,871350.NQ,871351.NQ,871352.NQ,871356.NQ,871357.NQ,871358.NQ,871359.NQ,871360.NQ,871361.NQ,871364.NQ,871365.NQ,871366.NQ,871368.NQ,871369.NQ,871371.NQ,871373.NQ,871380.NQ,871381.NQ,871382.NQ,871386.NQ,871392.NQ,871394.NQ,871395.NQ,871398.NQ,871402.NQ,871403.NQ,871404.NQ,871406.NQ,871408.NQ,871409.NQ,871410.NQ,871412.NQ,871413.NQ,871415.NQ,871417.NQ,871418.NQ,871421.NQ,871424.NQ,871425.NQ,871427.NQ,871429.NQ,871430.NQ,871433.NQ,871436.NQ,871444.NQ,871446.NQ,871447.NQ,871449.NQ,871450.NQ,871451.NQ,871453.NQ,871455.NQ,871457.NQ,871458.NQ,871459.NQ,871461.NQ,871465.NQ,871466.NQ,871469.NQ,871470.NQ,871471.NQ,871472.NQ,871475.NQ,871478.NQ,871480.NQ,871481.NQ,871482.NQ,871484.NQ,871485.NQ,871486.NQ,871487.NQ,871491.NQ,871494.NQ,871495.NQ,871500.NQ,871501.NQ,871504.NQ,871507.NQ,871508.NQ,871511.NQ,871512.NQ,871515.NQ,871516.NQ,871518.NQ,871519.NQ,871522.NQ,871527.NQ,871528.NQ,871529.NQ,871536.NQ,871539.NQ,871540.NQ,871547.NQ,871548.NQ,871549.NQ,871550.NQ,871551.NQ,871554.NQ,871555.NQ,871556.NQ,871559.NQ,871561.NQ,871562.NQ,871563.NQ,871564.NQ,871566.NQ,871567.NQ,871572.NQ,871574.NQ,871575.NQ,871580.NQ,871582.NQ,871583.NQ,871584.NQ,871589.NQ,871592.NQ,871594.NQ,871597.NQ,871598.NQ,871599.NQ,871601.NQ,871604.NQ,871605.NQ,871607.NQ,871613.NQ,871615.NQ,871617.NQ,871618.NQ,871620.NQ,871621.NQ,871626.NQ,871629.NQ,871633.NQ,871635.NQ,871636.NQ,871641.NQ,871643.NQ,871644.NQ,871645.NQ,871646.NQ,871647.NQ,871648.NQ,871651.NQ,871652.NQ,871653.NQ,871654.NQ,871655.NQ,871656.NQ,871664.NQ,871670.NQ,871672.NQ,871674.NQ,871675.NQ,871676.NQ,871677.NQ,871685.NQ,871688.NQ,871690.NQ,871691.NQ,871692.NQ,871693.NQ,871694.NQ,871695.NQ,871696.NQ,871699.NQ,871700.NQ,871701.NQ,871702.NQ,871703.NQ,871705.NQ,871707.NQ,871710.NQ,871713.NQ,871714.NQ,871715.NQ,871717.NQ,871721.NQ,871725.NQ,871727.NQ,871728.NQ,871729.NQ,871732.NQ,871733.NQ,871734.NQ,871736.NQ,871738.NQ,871739.NQ,871740.NQ,871741.NQ,871744.NQ,871748.NQ,871751.NQ,871756.NQ,871757.NQ,871760.NQ,871761.NQ,871765.NQ,871767.NQ,871768.NQ,871769.NQ,871771.NQ,871772.NQ,871774.NQ,871775.NQ,871781.NQ,871782.NQ,871783.NQ,871787.NQ,871791.NQ,871792.NQ,871794.NQ,871797.NQ,871799.NQ,871803.NQ,871805.NQ,871806.NQ,871807.NQ,871811.NQ,871813.NQ,871816.NQ,871819.NQ,871821.NQ,871822.NQ,871823.NQ,871824.NQ,871827.NQ,871828.NQ,871830.NQ,871833.NQ,871834.NQ,871835.NQ,871837.NQ,871839.NQ,871840.NQ,871841.NQ,871842.NQ,871843.NQ,871845.NQ,871846.NQ,871847.NQ,871848.NQ,871849.NQ,871855.NQ,871856.NQ,871858.NQ,871859.NQ,871862.NQ,871866.NQ,871868.NQ,871870.NQ,871874.NQ,871875.NQ,871878.NQ,871879.NQ,871880.NQ,871881.NQ,871882.NQ,871883.NQ,871884.NQ,871885.NQ,871888.NQ,871892.NQ,871894.NQ,871897.NQ,871902.NQ,871907.NQ,871908.NQ,871909.NQ,871910.NQ,871912.NQ,871920.NQ,871923.NQ,871927.NQ,871930.NQ,871931.NQ,871932.NQ,871933.NQ,871934.NQ,871935.NQ,871936.NQ,871937.NQ,871939.NQ,871941.NQ,871943.NQ,871945.NQ,871948.NQ,871950.NQ,871951.NQ,871952.NQ,871953.NQ,871954.NQ,871955.NQ,871957.NQ,871958.NQ,871962.NQ,871968.NQ,871972.NQ,871975.NQ,871979.NQ,871982.NQ,871986.NQ,871987.NQ,871988.NQ,871990.NQ,871992.NQ,871994.NQ,871995.NQ,872001.NQ,872002.NQ,872006.NQ,872007.NQ,872009.NQ,872013.NQ,872014.NQ,872017.NQ,872018.NQ,872019.NQ,872021.NQ,872022.NQ,872024.NQ,872032.NQ,872037.NQ,872038.NQ,872042.NQ,872044.NQ,872049.NQ,872050.NQ,872055.NQ,872058.NQ,872061.NQ,872063.NQ,872065.NQ,872066.NQ,872068.NQ,872069.NQ,872070.NQ,872076.NQ,872077.NQ,872082.NQ,872085.NQ,872089.NQ,872090.NQ,872092.NQ,872093.NQ,872094.NQ,872098.NQ,872099.NQ,872100.NQ,872101.NQ,872102.NQ,872103.NQ,872104.NQ,872106.NQ,872107.NQ,872109.NQ,872111.NQ,872112.NQ,872117.NQ,872118.NQ,872121.NQ,872122.NQ,872123.NQ,872124.NQ,872127.NQ,872128.NQ,872129.NQ,872131.NQ,872135.NQ,872137.NQ,872138.NQ,872139.NQ,872143.NQ,872144.NQ,872145.NQ,872149.NQ,872150.NQ,872152.NQ,872153.NQ,872154.NQ,872155.NQ,872156.NQ,872161.NQ,872162.NQ,872164.NQ,872165.NQ,872167.NQ,872168.NQ,872170.NQ,872172.NQ,872174.NQ,872175.NQ,872178.NQ,872180.NQ,872183.NQ,872184.NQ,872186.NQ,872188.NQ,872189.NQ,872191.NQ,872192.NQ,872197.NQ,872198.NQ,872199.NQ,872200.NQ,872202.NQ,872203.NQ,872204.NQ,872205.NQ,872206.NQ,872209.NQ,872211.NQ,872212.NQ,872213.NQ,872216.NQ,872217.NQ,872220.NQ,872223.NQ,872224.NQ,872226.NQ,872227.NQ,872230.NQ,872232.NQ,872233.NQ,872235.NQ,872238.NQ,872240.NQ,872242.NQ,872245.NQ,872247.NQ,872248.NQ,872249.NQ,872250.NQ,872255.NQ,872256.NQ,872258.NQ,872259.NQ,872260.NQ,872261.NQ,872262.NQ,872264.NQ,872266.NQ,872267.NQ,872271.NQ,872272.NQ,872279.NQ,872281.NQ,872284.NQ,872285.NQ,872286.NQ,872287.NQ,872288.NQ,872289.NQ,872290.NQ,872291.NQ,872293.NQ,872294.NQ,872297.NQ,872298.NQ,872299.NQ,872300.NQ,872301.NQ,872303.NQ,872304.NQ,872305.NQ,872307.NQ,872308.NQ,872310.NQ,872313.NQ,872315.NQ,872318.NQ,872319.NQ,872320.NQ,872322.NQ,872324.NQ,872325.NQ,872327.NQ,872328.NQ,872330.NQ,872333.NQ,872334.NQ,872336.NQ,872337.NQ,872338.NQ,872339.NQ,872340.NQ,872341.NQ,872342.NQ,872344.NQ,872345.NQ,872346.NQ,872347.NQ,872349.NQ,872352.NQ,872355.NQ,872357.NQ,872358.NQ,872361.NQ,872362.NQ,872363.NQ,872369.NQ,872371.NQ,872373.NQ,872375.NQ,872376.NQ,872377.NQ,872379.NQ,872380.NQ,872381.NQ,872383.NQ,872386.NQ,872388.NQ,872389.NQ,872391.NQ,872393.NQ,872396.NQ,872397.NQ,872398.NQ,872399.NQ,872400.NQ,872401.NQ,872402.NQ,872404.NQ,872405.NQ,872406.NQ,872408.NQ,872409.NQ,872411.NQ,872415.NQ,872416.NQ,872417.NQ,872418.NQ,872419.NQ,872420.NQ,872421.NQ,872422.NQ,872423.NQ,872424.NQ,872425.NQ,872426.NQ,872430.NQ,872432.NQ,872434.NQ,872435.NQ,872439.NQ,872440.NQ,872443.NQ,872444.NQ,872449.NQ,872450.NQ,872451.NQ,872453.NQ,872458.NQ,872460.NQ,872461.NQ,872464.NQ,872471.NQ,872473.NQ,872475.NQ,872476.NQ,872477.NQ,872478.NQ,872480.NQ,872481.NQ,872482.NQ,872483.NQ,872486.NQ,872488.NQ,872489.NQ,872490.NQ,872491.NQ,872492.NQ,872493.NQ,872494.NQ,872495.NQ,872496.NQ,872497.NQ,872498.NQ,872500.NQ,872502.NQ,872504.NQ,872505.NQ,872506.NQ,872507.NQ,872510.NQ,872511.NQ,872512.NQ,872515.NQ,872516.NQ,872518.NQ,872519.NQ,872520.NQ,872521.NQ,872522.NQ,872523.NQ,872525.NQ,872528.NQ,872530.NQ,872532.NQ,872534.NQ,872536.NQ,872538.NQ,872539.NQ,872544.NQ,872545.NQ,872546.NQ,872547.NQ,872549.NQ,872550.NQ,872551.NQ,872552.NQ,872553.NQ,872554.NQ,872555.NQ,872556.NQ,872558.NQ,872559.NQ,872560.NQ,872562.NQ,872564.NQ,872565.NQ,872567.NQ,872568.NQ,872570.NQ,872572.NQ,872573.NQ,872574.NQ,872575.NQ,872577.NQ,872578.NQ,872579.NQ,872580.NQ,872581.NQ,872582.NQ,872583.NQ,872584.NQ,872585.NQ,872586.NQ,872588.NQ,872593.NQ,872594.NQ,872595.NQ,872597.NQ,872598.NQ,872599.NQ,872602.NQ,872606.NQ,872609.NQ,872610.NQ,872611.NQ,872612.NQ,872616.NQ,872617.NQ,872618.NQ,872621.NQ,872622.NQ,872623.NQ,872624.NQ,872626.NQ,872628.NQ,872629.NQ,872631.NQ,872632.NQ,872633.NQ,872634.NQ,872635.NQ,872636.NQ,872638.NQ,872640.NQ,872641.NQ,872642.NQ,872643.NQ,872644.NQ,872645.NQ,872646.NQ,872655.NQ,872656.NQ,872658.NQ,872660.NQ,872662.NQ,872663.NQ,872665.NQ,872666.NQ,872667.NQ,872668.NQ,872669.NQ,872670.NQ,872672.NQ,872673.NQ,872675.NQ,872676.NQ,872677.NQ,872678.NQ,872679.NQ,872680.NQ,872681.NQ,872682.NQ,872684.NQ,872685.NQ,872687.NQ,872689.NQ,872691.NQ,872696.NQ,872698.NQ,872702.NQ,872704.NQ,872706.NQ,872707.NQ,872709.NQ,872710.NQ,872711.NQ,872712.NQ,872714.NQ,872715.NQ,872716.NQ,872720.NQ,872721.NQ,872723.NQ,872727.NQ,872728.NQ,872729.NQ,872732.NQ,872733.NQ,872735.NQ,872739.NQ,872740.NQ,872741.NQ,872742.NQ,872744.NQ,872745.NQ,872747.NQ,872748.NQ,872750.NQ,872752.NQ,872753.NQ,872757.NQ,872758.NQ,872759.NQ,872760.NQ,872762.NQ,872765.NQ,872766.NQ,872767.NQ,872773.NQ,872775.NQ,872776.NQ,872779.NQ,872780.NQ,872782.NQ,872783.NQ,872784.NQ,872787.NQ,872788.NQ,872791.NQ,872792.NQ,872793.NQ,872799.NQ,872801.NQ,872802.NQ,872804.NQ,872806.NQ,872807.NQ,872809.NQ,872810.NQ,872811.NQ,872813.NQ,872816.NQ,872817.NQ,872818.NQ,872819.NQ,872820.NQ,872821.NQ,872822.NQ,872823.NQ,872827.NQ,872828.NQ,872829.NQ,872831.NQ,872833.NQ,872836.NQ,872839.NQ,872840.NQ,872842.NQ,872843.NQ,872844.NQ,872845.NQ,872850.NQ,872851.NQ,872856.NQ,872857.NQ,872858.NQ,872860.NQ,872861.NQ,872862.NQ,872863.NQ,872864.NQ,872865.NQ,872868.NQ,872869.NQ,872870.NQ,872873.NQ,872874.NQ,872875.NQ,872878.NQ,872879.NQ,872880.NQ,872881.NQ,872882.NQ,872883.NQ,872884.NQ,872886.NQ,872887.NQ,872888.NQ,872890.NQ,872891.NQ,872892.NQ,872893.NQ,872894.NQ,872895.NQ,872896.NQ,872899.NQ,872900.NQ,872901.NQ,872902.NQ,872906.NQ,872907.NQ,872908.NQ,872910.NQ,872911.NQ,872915.NQ,872916.NQ,872917.NQ,872918.NQ,872919.NQ,872920.NQ,872921.NQ,872922.NQ,872926.NQ,872928.NQ,872930.NQ,872931.NQ,872935.NQ,872936.NQ,872937.NQ,872940.NQ,872941.NQ,872946.NQ,872948.NQ,872950.NQ,872951.NQ,872952.NQ,872953.NQ,872955.NQ,872956.NQ,872957.NQ,872958.NQ,872960.NQ,872961.NQ,872962.NQ,872963.NQ,872965.NQ,872967.NQ,872968.NQ,872969.NQ,872970.NQ,872971.NQ,872975.NQ,872978.NQ,872980.NQ,872981.NQ,872982.NQ,872984.NQ,872985.NQ,872986.NQ,872987.NQ,872988.NQ,872990.NQ,872991.NQ,872992.NQ,872996.NQ,872997.NQ,872998.NQ,872999.NQ,873000.NQ,873002.NQ,873003.NQ,873004.NQ,873005.NQ,873006.NQ,873007.NQ,873008.NQ,873009.NQ,873010.NQ,873011.NQ,873015.NQ,873016.NQ,873017.NQ,873018.NQ,873019.NQ,873020.NQ,873021.NQ,873025.NQ,873026.NQ,873027.NQ,873030.NQ,873031.NQ,873033.NQ,873036.NQ,873037.NQ,873038.NQ,873041.NQ,873045.NQ,873046.NQ,873047.NQ,873048.NQ,873049.NQ,873050.NQ,873051.NQ,873052.NQ,873053.NQ,873055.NQ,873056.NQ,873057.NQ,873058.NQ,873059.NQ,873060.NQ,873061.NQ,873062.NQ,873063.NQ,873065.NQ,873069.NQ,873070.NQ,873071.NQ,873073.NQ,873078.NQ,873079.NQ,873081.NQ,873082.NQ,873083.NQ,873085.NQ,873088.NQ,873089.NQ,873092.NQ,873093.NQ,873095.NQ,873097.NQ,873098.NQ,873099.NQ,873100.NQ,873101.NQ,873102.NQ,873107.NQ,873108.NQ,873110.NQ,873111.NQ,873114.NQ,873116.NQ,873117.NQ,873118.NQ,873119.NQ,873120.NQ,873125.NQ,873126.NQ,873128.NQ,873129.NQ,873130.NQ,873131.NQ,873132.NQ,873133.NQ,873135.NQ,873136.NQ,873137.NQ,873138.NQ,873139.NQ,873140.NQ,873141.NQ,873142.NQ,873143.NQ,873145.NQ,873146.NQ,873147.NQ,873148.NQ,873150.NQ,873151.NQ,873153.NQ,873155.NQ,873157.NQ,873161.NQ,873162.NQ,873163.NQ,873165.NQ,873170.NQ,873172.NQ,873173.NQ,873175.NQ,873176.NQ,873177.NQ,873180.NQ,873182.NQ,873183.NQ,873184.NQ,873185.NQ,873186.NQ,873187.NQ,873188.NQ,873189.NQ,873190.NQ,873192.NQ,873193.NQ,873195.NQ,873196.NQ,873197.NQ,873198.NQ,873199.NQ,873202.NQ,873203.NQ,873205.NQ,873206.NQ,873207.NQ,873208.NQ,873210.NQ,873212.NQ,873215.NQ,873217.NQ,873218.NQ,873219.NQ,873222.NQ,873224.NQ,873225.NQ,873226.NQ,873227.NQ,873228.NQ,873229.NQ,873230.NQ,873231.NQ,873232.NQ,873233.NQ,873234.NQ,873235.NQ,873236.NQ,873237.NQ,873238.NQ,873239.NQ,873240.NQ,873241.NQ,873243.NQ,873245.NQ,873246.NQ,873247.NQ,873248.NQ,873249.NQ,873250.NQ,873251.NQ,873252.NQ,873253.NQ,873255.NQ,873256.NQ,873258.NQ,873260.NQ,873261.NQ,873263.NQ,873266.NQ,873267.NQ,873268.NQ,873269.NQ,873270.NQ,873272.NQ,873273.NQ,873275.NQ,873277.NQ,873278.NQ,873279.NQ,873280.NQ,873283.NQ,873285.NQ,873286.NQ,873288.NQ,873289.NQ,873290.NQ,873291.NQ,873293.NQ,873295.NQ,873296.NQ,873297.NQ,873300.NQ,873301.NQ,873302.NQ,873306.NQ,873307.NQ,873308.NQ,873309.NQ,873311.NQ,873312.NQ,873316.NQ,873318.NQ,873319.NQ,873320.NQ,873321.NQ,873322.NQ,873323.NQ,873324.NQ,873325.NQ,873326.NQ,873327.NQ,873328.NQ,873329.NQ,873330.NQ,873331.NQ,873332.NQ,873333.NQ,873334.NQ,873335.NQ,873336.NQ,873337.NQ,873338.NQ,873340.NQ,873341.NQ,873343.NQ,873344.NQ,873346.NQ,873347.NQ,873348.NQ,873349.NQ,873350.NQ,873351.NQ,873352.NQ,873353.NQ,873355.NQ,873356.NQ,873357.NQ,873358.NQ,873359.NQ,873360.NQ,873361.NQ,873362.NQ,873363.NQ,873366.NQ,873367.NQ,873369.NQ,873370.NQ,873373.NQ,873375.NQ,873376.NQ,873377.NQ,873378.NQ,873380.NQ,873381.NQ,873382.NQ,873383.NQ,873384.NQ,873385.NQ,873386.NQ,873387.NQ,873389.NQ,873390.NQ,873391.NQ,873392.NQ,873393.NQ,873395.NQ,873396.NQ,873397.NQ,873398.NQ,873399.NQ,873400.NQ,873401.NQ,873402.NQ,873403.NQ,873406.NQ,873407.NQ,873408.NQ,873410.NQ,873411.NQ,873412.NQ,873415.NQ,873416.NQ,873417.NQ,873418.NQ,873419.NQ,873420.NQ,873421.NQ,873423.NQ,873425.NQ,873426.NQ,873427.NQ,873428.NQ,873429.NQ,873430.NQ,873432.NQ,873433.NQ,873434.NQ,873435.NQ,873436.NQ,873437.NQ,873438.NQ,873440.NQ,873441.NQ,873442.NQ,873443.NQ,873444.NQ,873446.NQ,873447.NQ,873448.NQ,873449.NQ,873450.NQ,873451.NQ,873452.NQ,873453.NQ,873455.NQ,873456.NQ,873458.NQ,873459.NQ,873460.NQ,873461.NQ,873462.NQ,873463.NQ,873465.NQ,873467.NQ,873468.NQ,873469.NQ,873470.NQ,873471.NQ,873472.NQ,873473.NQ,873475.NQ,873476.NQ,873477.NQ,873478.NQ,873479.NQ,873480.NQ,873481.NQ,873482.NQ,873483.NQ,873485.NQ,873486.NQ,873487.NQ,873488.NQ,873490.NQ,873491.NQ,873492.NQ,873493.NQ,873495.NQ,873496.NQ,873498.NQ,873499.NQ,873500.NQ,873501.NQ,873502.NQ,873503.NQ,873505.NQ,873506.NQ,873507.NQ,873508.NQ,873509.NQ,873510.NQ,873511.NQ,873512.NQ,873513.NQ,873515.NQ,873516.NQ,873517.NQ,873518.NQ,873519.NQ,873520.NQ,873521.NQ,873522.NQ,873523.NQ,873525.NQ,873526.NQ,873528.NQ,873529.NQ,873530.NQ,873531.NQ,873533.NQ,873535.NQ,873536.NQ,873537.NQ,873538.NQ,873539.NQ,873540.NQ,873541.NQ,873542.NQ,873543.NQ,873546.NQ,873547.NQ,873548.NQ,873549.NQ,873550.NQ,873551.NQ,873552.NQ,873553.NQ,873554.NQ,873556.NQ,873557.NQ,873559.NQ,873560.NQ,873561.NQ,873563.NQ,873564.NQ,873565.NQ,873566.NQ,873567.NQ,873568.NQ,873569.NQ,873570.NQ,873571.NQ,873572.NQ,873573.NQ,873575.NQ,873576.NQ,873577.NQ,873579.NQ,873580.NQ,873581.NQ,873583.NQ,873585.NQ,873586.NQ,873587.NQ,873588.NQ,873589.NQ,873590.NQ,873591.NQ,873592.NQ,873593.NQ,873595.NQ,873596.NQ,873597.NQ,873598.NQ,873599.NQ,873600.NQ,873601.NQ,873602.NQ,873603.NQ,873605.NQ,873606.NQ,873607.NQ,873608.NQ,873609.NQ,873610.NQ,873611.NQ,873612.NQ,873613.NQ,873615.NQ,873616.NQ,873617.NQ,873618.NQ,873619.NQ,873620.NQ,873621.NQ,873623.NQ,873625.NQ,873626.NQ,873627.NQ,873628.NQ,873629.NQ,873630.NQ,873631.NQ,873632.NQ,873633.NQ,873635.NQ,873636.NQ,873637.NQ,873638.NQ,873639.NQ,873640.NQ,873641.NQ,873642.NQ,873643.NQ,873644.NQ,873645.NQ,873646.NQ,873647.NQ,873648.NQ,873649.NQ,873650.NQ,873651.NQ,873652.NQ,873653.NQ,873655.NQ,873656.NQ,873657.NQ,873658.NQ,873659.NQ,873660.NQ,873661.NQ,873662.NQ,873663.NQ,873664.NQ,873665.NQ,873666.NQ,873667.NQ,873668.NQ,873669.NQ,873670.NQ,873671.NQ,873672.NQ,873673.NQ,873675.NQ,873676.NQ,873678.NQ,873679.NQ,873680.NQ,873681.NQ,873682.NQ,873683.NQ,873684.NQ,873685.NQ,873686.NQ,873687.NQ,873688.NQ,873689.NQ,873690.NQ,873691.NQ,873692.NQ,873693.NQ,873694.NQ,873695.NQ,873696.NQ,873697.NQ,873698.NQ,873699.NQ,873700.NQ,873701.NQ,873702.NQ,873703.NQ,873705.NQ,873706.NQ,873707.NQ,873708.NQ,873709.NQ,873710.NQ,873711.NQ,873712.NQ,873713.NQ,873715.NQ,873716.NQ,873717.NQ,873718.NQ,873719.NQ,873720.NQ,873721.NQ,873722.NQ,873725.NQ,873726.NQ,873727.NQ,873728.NQ,873729.NQ,873730.NQ,873731.NQ,873732.NQ,873733.NQ,873735.NQ,873736.NQ,873737.NQ,873738.NQ,873739.NQ,873740.NQ,873741.NQ,873743.NQ,873744.NQ,873745.NQ,873746.NQ,873747.NQ,873748.NQ,873749.NQ,873750.NQ,873751.NQ,873752.NQ,873753.NQ,873755.NQ,873756.NQ,873757.NQ,873758.NQ,873759.NQ,873760.NQ,873762.NQ,873763.NQ,873765.NQ,873766.NQ,873767.NQ,873768.NQ,873769.NQ,873770.NQ,873771.NQ,873772.NQ,873773.NQ,873775.NQ,873776.NQ,873777.NQ,873778.NQ,873779.NQ,873780.NQ,873781.NQ,873782.NQ,873783.NQ,873784.NQ,873785.NQ,873786.NQ,873787.NQ,873788.NQ,873789.NQ,873790.NQ,873791.NQ,873792.NQ,873793.NQ,873795.NQ,873796.NQ,873797.NQ,873798.NQ,873799.NQ,873800.NQ,873801.NQ,873802.NQ,873803.NQ,873804.NQ,873805.NQ,873806.NQ,873807.NQ,873808.NQ,873809.NQ,873810.NQ,873811.NQ,873812.NQ,873813.NQ,873814.NQ,873815.NQ,873816.NQ,873817.NQ,873818.NQ,873820.NQ,873821.NQ,873822.NQ,873823.NQ,873825.NQ,873826.NQ,873827.NQ,873828.NQ,873829.NQ,873830.NQ,873831.NQ,873832.NQ,873833.NQ,873834.NQ,873835.NQ,873836.NQ,873837.NQ,873838.NQ,873839.NQ,873840.NQ,873841.NQ,873842.NQ,873843.NQ,873845.NQ,873847.NQ,873848.NQ,873849.NQ,873850.NQ,873851.NQ,873852.NQ,873853.NQ,873855.NQ,873856.NQ,873857.NQ,873858.NQ,873859.NQ,873860.NQ,873861.NQ,873862.NQ,873863.NQ,873864.NQ,873865.NQ,873866.NQ,873867.NQ,873868.NQ,873869.NQ,873870.NQ,873871.NQ,873872.NQ,873873.NQ,873874.NQ,873875.NQ,873876.NQ,873877.NQ,873878.NQ,873879.NQ,873880.NQ,873881.NQ,873882.NQ,873884.NQ,873885.NQ,873886.NQ,873887.NQ,873888.NQ,873889.NQ,873890.NQ,873891.NQ,873892.NQ,873893.NQ,873895.NQ,873896.NQ,873898.NQ,873899.NQ,873901.NQ,873902.NQ,873903.NQ,873905.NQ,873906.NQ,873907.NQ,873908.NQ,873909.NQ,873910.NQ,873911.NQ,873912.NQ,873913.NQ,873914.NQ,873915.NQ,873916.NQ,873917.NQ,873918.NQ,873919.NQ,873920.NQ,873921.NQ,873922.NQ,873923.NQ,873926.NQ,873927.NQ,873928.NQ,873929.NQ,873930.NQ,873931.NQ,873932.NQ,873933.NQ,873935.NQ,873936.NQ,873937.NQ,873938.NQ,873939.NQ,873943.NQ,873945.NQ,873946.NQ,873947.NQ,873948.NQ,873949.NQ,873950.NQ,873951.NQ,873953.NQ,873955.NQ,873956.NQ,873957.NQ,873958.NQ,873959.NQ,873960.NQ,873961.NQ,873962.NQ,873963.NQ,873964.NQ,873965.NQ,873966.NQ,873967.NQ,873968.NQ,873969.NQ,873970.NQ,873971.NQ,873972.NQ,873973.NQ,873975.NQ,873976.NQ,873977.NQ,873978.NQ,873979.NQ,873980.NQ,873981.NQ,873982.NQ,873983.NQ,873985.NQ,873987.NQ,873988.NQ,873989.NQ,873990.NQ,873991.NQ,873992.NQ,873993.NQ,873995.NQ,873996.NQ,873997.NQ,873998.NQ,873999.NQ,874000.NQ,874001.NQ,874002.NQ,874003.NQ,874005.NQ,874006.NQ,874007.NQ,874009.NQ,874010.NQ,874011.NQ,874016.NQ,874017.NQ,874018.NQ,874019.NQ,874020.NQ,874021.NQ,874022.NQ,874025.NQ,874026.NQ,874027.NQ,874028.NQ,874030.NQ,874031.NQ,874033.NQ,874035.NQ,874036.NQ,874037.NQ,874038.NQ,874039.NQ,874040.NQ,874041.NQ,874047.NQ,874048.NQ,874050.NQ,874055.NQ,874056.NQ,874057.NQ,874058.NQ,874062.NQ,874068.NQ,874070.NQ,874075.NQ,874095.NQ,874107.NQ,874109.NQ,874110.NQ", "close_per,trade_days_per,share_totaltradable,wgsd_assets","startDate="+date_b+";endDate="+date_a+";priceAdj=U;unit=1;tradeDate="+date_c+";rptDate="+date_c+";rptType=1;currencyType=")
    #将数据类型转换为dataframe
    df_n_c = pd.DataFrame(data1.Data,index = data.Fields,columns = data.Codes)
    #转置
    df_n_c = df_n_c.T
    #删除四个变量全为空的数据
    df_n_c.dropna(how = 'all',inplace = True)
    #将其他为空的变量用0填充
    df_n_c.fillna(0)
    #分组
    divide(df_n_c)

In [31]:
r1 = df_n_m.loc[1]
date_b = r1['做市前100日'].date()
date_a = r1['做市首日'].date()
date_c = r1['匹配期'].date()

In [32]:
date_b = date_b.strftime('%Y%m%d')
date_a = date_a.strftime('%Y%m%d')
date_c = date_c.strftime('%Y%m%d')

In [33]:
data1 = w.wss("430003.NQ,430004.NQ,430005.NQ,430009.NQ,430010.NQ,430014.NQ,430017.NQ,430019.NQ,430020.NQ,430022.NQ,430025.NQ,430027.NQ,430028.NQ,430031.NQ,430033.NQ,430034.NQ,430041.NQ,430042.NQ,430044.NQ,430052.NQ,430053.NQ,430055.NQ,430056.NQ,430057.NQ,430061.NQ,430064.NQ,430066.NQ,430068.NQ,430072.NQ,430073.NQ,430076.NQ,430077.NQ,430078.NQ,430079.NQ,430081.NQ,430084.NQ,430085.NQ,430086.NQ,430089.NQ,430091.NQ,430093.NQ,430094.NQ,430096.NQ,430097.NQ,430099.NQ,430102.NQ,430103.NQ,430104.NQ,430105.NQ,430106.NQ,430107.NQ,430108.NQ,430109.NQ,430111.NQ,430112.NQ,430114.NQ,430116.NQ,430119.NQ,430120.NQ,430122.NQ,430124.NQ,430130.NQ,430131.NQ,430138.NQ,430143.NQ,430144.NQ,430145.NQ,430148.NQ,430152.NQ,430153.NQ,430156.NQ,430158.NQ,430159.NQ,430160.NQ,430161.NQ,430165.NQ,430168.NQ,430170.NQ,430171.NQ,430172.NQ,430176.NQ,430178.NQ,430179.NQ,430180.NQ,430182.NQ,430184.NQ,430185.NQ,430186.NQ,430188.NQ,430190.NQ,430191.NQ,430193.NQ,430197.NQ,430200.NQ,430203.NQ,430204.NQ,430215.NQ,430216.NQ,430217.NQ,430218.NQ,430219.NQ,430220.NQ,430224.NQ,430228.NQ,430229.NQ,430235.NQ,430239.NQ,430240.NQ,430241.NQ,430244.NQ,430246.NQ,430247.NQ,430249.NQ,430250.NQ,430252.NQ,430257.NQ,430258.NQ,430263.NQ,430264.NQ,430266.NQ,430268.NQ,430269.NQ,430271.NQ,430273.NQ,430274.NQ,430276.NQ,430281.NQ,430282.NQ,430283.NQ,430284.NQ,430285.NQ,430286.NQ,430288.NQ,430289.NQ,430292.NQ,430293.NQ,430296.NQ,430301.NQ,430304.NQ,430305.NQ,430314.NQ,430316.NQ,430319.NQ,430320.NQ,430321.NQ,430322.NQ,430324.NQ,430326.NQ,430331.NQ,430333.NQ,430336.NQ,430337.NQ,430338.NQ,430341.NQ,430343.NQ,430344.NQ,430346.NQ,430350.NQ,430359.NQ,430361.NQ,430362.NQ,430363.NQ,430366.NQ,430367.NQ,430368.NQ,430369.NQ,430370.NQ,430379.NQ,430380.NQ,430387.NQ,430388.NQ,430391.NQ,430392.NQ,430393.NQ,430395.NQ,430396.NQ,430397.NQ,430398.NQ,430403.NQ,430405.NQ,430409.NQ,430410.NQ,430412.NQ,430414.NQ,430420.NQ,430422.NQ,430423.NQ,430426.NQ,430427.NQ,430428.NQ,430429.NQ,430432.NQ,430434.NQ,430435.NQ,430437.NQ,430439.NQ,430441.NQ,430442.NQ,430443.NQ,430448.NQ,430451.NQ,430452.NQ,430453.NQ,430455.NQ,430457.NQ,430458.NQ,430459.NQ,430460.NQ,430461.NQ,430462.NQ,430463.NQ,430467.NQ,430470.NQ,430471.NQ,430473.NQ,430477.NQ,430479.NQ,430480.NQ,430482.NQ,430486.NQ,430487.NQ,430490.NQ,430491.NQ,430495.NQ,430497.NQ,430500.NQ,430502.NQ,430505.NQ,430509.NQ,430513.NQ,430514.NQ,430516.NQ,430517.NQ,430519.NQ,430520.NQ,430525.NQ,430528.NQ,430529.NQ,430530.NQ,430535.NQ,430537.NQ,430538.NQ,430539.NQ,430541.NQ,430542.NQ,430545.NQ,430549.NQ,430550.NQ,430552.NQ,430558.NQ,430560.NQ,430561.NQ,430562.NQ,430563.NQ,430566.NQ,430567.NQ,430570.NQ,430572.NQ,430573.NQ,430576.NQ,430577.NQ,430578.NQ,430579.NQ,430583.NQ,430584.NQ,430585.NQ,430586.NQ,430588.NQ,430589.NQ,430590.NQ,430592.NQ,430593.NQ,430594.NQ,430597.NQ,430601.NQ,430602.NQ,430607.NQ,430609.NQ,430610.NQ,430611.NQ,430615.NQ,430616.NQ,430619.NQ,430620.NQ,430626.NQ,430632.NQ,430635.NQ,430636.NQ,430638.NQ,430641.NQ,430644.NQ,430645.NQ,430646.NQ,430647.NQ,430649.NQ,430653.NQ,430655.NQ,430656.NQ,430657.NQ,430658.NQ,430659.NQ,430663.NQ,430664.NQ,430665.NQ,430666.NQ,430667.NQ,430669.NQ,430670.NQ,430673.NQ,430677.NQ,430683.NQ,430686.NQ,430687.NQ,430690.NQ,430692.NQ,430695.NQ,430697.NQ,430703.NQ,430704.NQ,430706.NQ,430709.NQ,430712.NQ,430716.NQ,430717.NQ,430719.NQ,430724.NQ,430725.NQ,430726.NQ,430728.NQ,430730.NQ,430732.NQ,430734.NQ,430736.NQ,430742.NQ,430743.NQ,430744.NQ,430745.NQ,430746.NQ,430748.NQ,430749.NQ,430753.NQ,430756.NQ,430758.NQ,430761.NQ,430762.NQ,830768.NQ,830769.NQ,830770.NQ,830772.NQ,830773.NQ,830774.NQ,830775.NQ,830776.NQ,830778.NQ,830779.NQ,830781.NQ,830782.NQ,830783.NQ,830784.NQ,830786.NQ,830787.NQ,830789.NQ,830790.NQ,830791.NQ,830792.NQ,830794.NQ,830796.NQ,830797.NQ,830798.NQ,830802.NQ,830803.NQ,830821.NQ,830824.NQ,830825.NQ,830828.NQ,830829.NQ,830830.NQ,830836.NQ,830840.NQ,830841.NQ,830844.NQ,830846.NQ,830848.NQ,830849.NQ,830851.NQ,830852.NQ,830856.NQ,830857.NQ,830859.NQ,830860.NQ,830861.NQ,830864.NQ,830867.NQ,830868.NQ,830870.NQ,830871.NQ,830872.NQ,830873.NQ,830874.NQ,830876.NQ,830877.NQ,830880.NQ,830883.NQ,830885.NQ,830886.NQ,830888.NQ,830889.NQ,830890.NQ,830891.NQ,830892.NQ,830894.NQ,830896.NQ,830898.NQ,830900.NQ,830902.NQ,830905.NQ,830913.NQ,830914.NQ,830915.NQ,830917.NQ,830918.NQ,830920.NQ,830922.NQ,830923.NQ,830924.NQ,830926.NQ,830927.NQ,830929.NQ,830930.NQ,830939.NQ,830941.NQ,830943.NQ,830949.NQ,830953.NQ,830955.NQ,830958.NQ,830961.NQ,830969.NQ,830971.NQ,830976.NQ,830980.NQ,830982.NQ,830983.NQ,830993.NQ,830994.NQ,830995.NQ,830996.NQ,831001.NQ,831002.NQ,831003.NQ,831004.NQ,831006.NQ,831013.NQ,831014.NQ,831021.NQ,831022.NQ,831025.NQ,831026.NQ,831032.NQ,831033.NQ,831034.NQ,831036.NQ,831040.NQ,831042.NQ,831047.NQ,831048.NQ,831053.NQ,831054.NQ,831057.NQ,831059.NQ,831063.NQ,831065.NQ,831066.NQ,831067.NQ,831070.NQ,831073.NQ,831078.NQ,831081.NQ,831083.NQ,831085.NQ,831091.NQ,831092.NQ,831095.NQ,831096.NQ,831097.NQ,831099.NQ,831101.NQ,831104.NQ,831107.NQ,831108.NQ,831110.NQ,831111.NQ,831114.NQ,831116.NQ,831119.NQ,831121.NQ,831122.NQ,831123.NQ,831124.NQ,831130.NQ,831131.NQ,831132.NQ,831133.NQ,831134.NQ,831137.NQ,831140.NQ,831142.NQ,831144.NQ,831149.NQ,831151.NQ,831154.NQ,831156.NQ,831162.NQ,831163.NQ,831165.NQ,831169.NQ,831175.NQ,831176.NQ,831179.NQ,831181.NQ,831183.NQ,831184.NQ,831185.NQ,831186.NQ,831187.NQ,831197.NQ,831199.NQ,831200.NQ,831201.NQ,831203.NQ,831208.NQ,831211.NQ,831216.NQ,831225.NQ,831227.NQ,831228.NQ,831229.NQ,831231.NQ,831232.NQ,831233.NQ,831234.NQ,831235.NQ,831236.NQ,831237.NQ,831239.NQ,831240.NQ,831244.NQ,831246.NQ,831252.NQ,831253.NQ,831254.NQ,831255.NQ,831259.NQ,831264.NQ,831266.NQ,831269.NQ,831270.NQ,831272.NQ,831273.NQ,831275.NQ,831276.NQ,831282.NQ,831284.NQ,831285.NQ,831288.NQ,831292.NQ,831293.NQ,831296.NQ,831297.NQ,831304.NQ,831308.NQ,831309.NQ,831312.NQ,831313.NQ,831317.NQ,831318.NQ,831321.NQ,831322.NQ,831323.NQ,831325.NQ,831330.NQ,831331.NQ,831332.NQ,831334.NQ,831336.NQ,831337.NQ,831338.NQ,831340.NQ,831343.NQ,831345.NQ,831346.NQ,831348.NQ,831350.NQ,831351.NQ,831355.NQ,831356.NQ,831357.NQ,831358.NQ,831361.NQ,831366.NQ,831367.NQ,831368.NQ,831369.NQ,831372.NQ,831373.NQ,831376.NQ,831380.NQ,831382.NQ,831384.NQ,831385.NQ,831386.NQ,831387.NQ,831389.NQ,831390.NQ,831391.NQ,831394.NQ,831395.NQ,831396.NQ,831397.NQ,831398.NQ,831399.NQ,831401.NQ,831402.NQ,831404.NQ,831405.NQ,831406.NQ,831408.NQ,831412.NQ,831414.NQ,831415.NQ,831416.NQ,831418.NQ,831419.NQ,831422.NQ,831423.NQ,831428.NQ,831433.NQ,831434.NQ,831439.NQ,831442.NQ,831446.NQ,831448.NQ,831449.NQ,831453.NQ,831459.NQ,831460.NQ,831462.NQ,831463.NQ,831464.NQ,831465.NQ,831467.NQ,831469.NQ,831470.NQ,831474.NQ,831476.NQ,831477.NQ,831478.NQ,831481.NQ,831482.NQ,831483.NQ,831485.NQ,831486.NQ,831490.NQ,831491.NQ,831492.NQ,831496.NQ,831501.NQ,831502.NQ,831510.NQ,831511.NQ,831514.NQ,831515.NQ,831516.NQ,831519.NQ,831521.NQ,831523.NQ,831532.NQ,831538.NQ,831539.NQ,831553.NQ,831554.NQ,831555.NQ,831556.NQ,831566.NQ,831569.NQ,831575.NQ,831577.NQ,831579.NQ,831581.NQ,831584.NQ,831587.NQ,831589.NQ,831590.NQ,831591.NQ,831593.NQ,831594.NQ,831598.NQ,831599.NQ,831605.NQ,831606.NQ,831608.NQ,831611.NQ,831614.NQ,831615.NQ,831619.NQ,831621.NQ,831622.NQ,831623.NQ,831625.NQ,831627.NQ,831629.NQ,831631.NQ,831633.NQ,831634.NQ,831638.NQ,831640.NQ,831642.NQ,831643.NQ,831646.NQ,831650.NQ,831654.NQ,831657.NQ,831658.NQ,831660.NQ,831661.NQ,831664.NQ,831668.NQ,831670.NQ,831672.NQ,831673.NQ,831674.NQ,831676.NQ,831677.NQ,831682.NQ,831685.NQ,831686.NQ,831690.NQ,831691.NQ,831693.NQ,831694.NQ,831701.NQ,831703.NQ,831706.NQ,831707.NQ,831708.NQ,831709.NQ,831714.NQ,831718.NQ,831719.NQ,831720.NQ,831721.NQ,831722.NQ,831724.NQ,831725.NQ,831727.NQ,831732.NQ,831734.NQ,831736.NQ,831738.NQ,831740.NQ,831742.NQ,831744.NQ,831747.NQ,831749.NQ,831750.NQ,831752.NQ,831753.NQ,831755.NQ,831756.NQ,831760.NQ,831765.NQ,831767.NQ,831769.NQ,831770.NQ,831773.NQ,831774.NQ,831778.NQ,831779.NQ,831783.NQ,831787.NQ,831793.NQ,831798.NQ,831799.NQ,831804.NQ,831805.NQ,831813.NQ,831814.NQ,831816.NQ,831817.NQ,831824.NQ,831827.NQ,831828.NQ,831830.NQ,831831.NQ,831833.NQ,831838.NQ,831841.NQ,831843.NQ,831850.NQ,831853.NQ,831854.NQ,831858.NQ,831860.NQ,831862.NQ,831865.NQ,831866.NQ,831869.NQ,831870.NQ,831874.NQ,831875.NQ,831877.NQ,831879.NQ,831880.NQ,831881.NQ,831882.NQ,831884.NQ,831885.NQ,831888.NQ,831892.NQ,831893.NQ,831894.NQ,831897.NQ,831904.NQ,831910.NQ,831912.NQ,831916.NQ,831917.NQ,831922.NQ,831924.NQ,831925.NQ,831929.NQ,831931.NQ,831934.NQ,831935.NQ,831936.NQ,831937.NQ,831942.NQ,831943.NQ,831944.NQ,831946.NQ,831948.NQ,831951.NQ,831952.NQ,831957.NQ,831958.NQ,831960.NQ,831962.NQ,831969.NQ,831970.NQ,831973.NQ,831974.NQ,831978.NQ,831985.NQ,831988.NQ,831989.NQ,831990.NQ,831991.NQ,831992.NQ,831994.NQ,831995.NQ,831999.NQ,832001.NQ,832002.NQ,832003.NQ,832006.NQ,832009.NQ,832010.NQ,832013.NQ,832014.NQ,832017.NQ,832018.NQ,832019.NQ,832021.NQ,832022.NQ,832025.NQ,832027.NQ,832033.NQ,832035.NQ,832036.NQ,832037.NQ,832040.NQ,832041.NQ,832042.NQ,832043.NQ,832045.NQ,832047.NQ,832048.NQ,832049.NQ,832050.NQ,832053.NQ,832054.NQ,832055.NQ,832058.NQ,832059.NQ,832061.NQ,832062.NQ,832063.NQ,832064.NQ,832067.NQ,832069.NQ,832075.NQ,832078.NQ,832081.NQ,832082.NQ,832084.NQ,832085.NQ,832088.NQ,832093.NQ,832094.NQ,832095.NQ,832100.NQ,832103.NQ,832105.NQ,832107.NQ,832108.NQ,832111.NQ,832113.NQ,832114.NQ,832116.NQ,832118.NQ,832119.NQ,832124.NQ,832126.NQ,832127.NQ,832129.NQ,832132.NQ,832133.NQ,832134.NQ,832136.NQ,832141.NQ,832146.NQ,832155.NQ,832163.NQ,832164.NQ,832173.NQ,832176.NQ,832178.NQ,832181.NQ,832185.NQ,832191.NQ,832196.NQ,832198.NQ,832205.NQ,832208.NQ,832209.NQ,832213.NQ,832214.NQ,832217.NQ,832221.NQ,832222.NQ,832223.NQ,832226.NQ,832229.NQ,832230.NQ,832232.NQ,832234.NQ,832239.NQ,832240.NQ,832241.NQ,832244.NQ,832246.NQ,832248.NQ,832249.NQ,832255.NQ,832256.NQ,832257.NQ,832258.NQ,832259.NQ,832263.NQ,832264.NQ,832265.NQ,832267.NQ,832270.NQ,832274.NQ,832276.NQ,832277.NQ,832281.NQ,832283.NQ,832289.NQ,832290.NQ,832291.NQ,832292.NQ,832293.NQ,832294.NQ,832296.NQ,832297.NQ,832298.NQ,832301.NQ,832302.NQ,832307.NQ,832309.NQ,832311.NQ,832314.NQ,832315.NQ,832318.NQ,832319.NQ,832320.NQ,832323.NQ,832326.NQ,832331.NQ,832334.NQ,832335.NQ,832337.NQ,832339.NQ,832344.NQ,832346.NQ,832347.NQ,832350.NQ,832351.NQ,832352.NQ,832354.NQ,832355.NQ,832357.NQ,832359.NQ,832360.NQ,832363.NQ,832364.NQ,832366.NQ,832368.NQ,832369.NQ,832372.NQ,832373.NQ,832377.NQ,832381.NQ,832383.NQ,832389.NQ,832391.NQ,832393.NQ,832394.NQ,832395.NQ,832398.NQ,832400.NQ,832402.NQ,832403.NQ,832405.NQ,832406.NQ,832409.NQ,832415.NQ,832417.NQ,832418.NQ,832423.NQ,832428.NQ,832430.NQ,832431.NQ,832432.NQ,832436.NQ,832437.NQ,832439.NQ,832441.NQ,832445.NQ,832447.NQ,832457.NQ,832459.NQ,832460.NQ,832465.NQ,832468.NQ,832469.NQ,832472.NQ,832476.NQ,832477.NQ,832479.NQ,832480.NQ,832484.NQ,832488.NQ,832492.NQ,832493.NQ,832494.NQ,832496.NQ,832502.NQ,832503.NQ,832505.NQ,832506.NQ,832508.NQ,832509.NQ,832510.NQ,832511.NQ,832513.NQ,832518.NQ,832519.NQ,832520.NQ,832521.NQ,832528.NQ,832531.NQ,832533.NQ,832534.NQ,832535.NQ,832538.NQ,832541.NQ,832545.NQ,832546.NQ,832547.NQ,832553.NQ,832554.NQ,832557.NQ,832558.NQ,832567.NQ,832568.NQ,832569.NQ,832570.NQ,832572.NQ,832573.NQ,832574.NQ,832575.NQ,832576.NQ,832583.NQ,832591.NQ,832592.NQ,832593.NQ,832594.NQ,832597.NQ,832601.NQ,832602.NQ,832603.NQ,832613.NQ,832614.NQ,832616.NQ,832620.NQ,832621.NQ,832623.NQ,832627.NQ,832629.NQ,832630.NQ,832634.NQ,832637.NQ,832638.NQ,832641.NQ,832642.NQ,832644.NQ,832646.NQ,832647.NQ,832649.NQ,832651.NQ,832652.NQ,832653.NQ,832654.NQ,832655.NQ,832656.NQ,832657.NQ,832658.NQ,832661.NQ,832664.NQ,832665.NQ,832670.NQ,832672.NQ,832673.NQ,832674.NQ,832676.NQ,832682.NQ,832684.NQ,832685.NQ,832687.NQ,832694.NQ,832695.NQ,832696.NQ,832702.NQ,832703.NQ,832705.NQ,832709.NQ,832711.NQ,832713.NQ,832714.NQ,832715.NQ,832718.NQ,832720.NQ,832721.NQ,832722.NQ,832723.NQ,832724.NQ,832725.NQ,832727.NQ,832728.NQ,832730.NQ,832734.NQ,832736.NQ,832737.NQ,832738.NQ,832740.NQ,832749.NQ,832750.NQ,832751.NQ,832753.NQ,832754.NQ,832757.NQ,832763.NQ,832764.NQ,832765.NQ,832767.NQ,832769.NQ,832770.NQ,832774.NQ,832777.NQ,832778.NQ,832780.NQ,832782.NQ,832783.NQ,832784.NQ,832787.NQ,832789.NQ,832790.NQ,832791.NQ,832792.NQ,832793.NQ,832798.NQ,832800.NQ,832807.NQ,832809.NQ,832813.NQ,832814.NQ,832816.NQ,832822.NQ,832825.NQ,832828.NQ,832830.NQ,832832.NQ,832834.NQ,832836.NQ,832839.NQ,832842.NQ,832848.NQ,832849.NQ,832850.NQ,832852.NQ,832853.NQ,832861.NQ,832862.NQ,832863.NQ,832864.NQ,832866.NQ,832867.NQ,832874.NQ,832875.NQ,832879.NQ,832881.NQ,832882.NQ,832888.NQ,832889.NQ,832890.NQ,832891.NQ,832894.NQ,832895.NQ,832902.NQ,832903.NQ,832905.NQ,832906.NQ,832911.NQ,832912.NQ,832914.NQ,832915.NQ,832917.NQ,832918.NQ,832920.NQ,832924.NQ,832925.NQ,832926.NQ,832927.NQ,832928.NQ,832929.NQ,832932.NQ,832935.NQ,832937.NQ,832943.NQ,832944.NQ,832947.NQ,832950.NQ,832952.NQ,832954.NQ,832955.NQ,832956.NQ,832958.NQ,832964.NQ,832966.NQ,832968.NQ,832970.NQ,832972.NQ,832973.NQ,832974.NQ,832978.NQ,832980.NQ,832982.NQ,832985.NQ,832986.NQ,832987.NQ,832988.NQ,832992.NQ,832994.NQ,832997.NQ,833000.NQ,833006.NQ,833009.NQ,833012.NQ,833014.NQ,833016.NQ,833022.NQ,833023.NQ,833024.NQ,833030.NQ,833033.NQ,833035.NQ,833037.NQ,833041.NQ,833042.NQ,833044.NQ,833046.NQ,833047.NQ,833050.NQ,833051.NQ,833052.NQ,833053.NQ,833058.NQ,833060.NQ,833063.NQ,833065.NQ,833068.NQ,833069.NQ,833070.NQ,833072.NQ,833073.NQ,833074.NQ,833077.NQ,833084.NQ,833088.NQ,833091.NQ,833096.NQ,833098.NQ,833099.NQ,833100.NQ,833105.NQ,833110.NQ,833111.NQ,833112.NQ,833115.NQ,833118.NQ,833119.NQ,833120.NQ,833124.NQ,833126.NQ,833127.NQ,833136.NQ,833137.NQ,833151.NQ,833153.NQ,833154.NQ,833158.NQ,833159.NQ,833160.NQ,833161.NQ,833162.NQ,833164.NQ,833165.NQ,833167.NQ,833168.NQ,833177.NQ,833179.NQ,833181.NQ,833189.NQ,833197.NQ,833199.NQ,833200.NQ,833201.NQ,833202.NQ,833204.NQ,833207.NQ,833208.NQ,833211.NQ,833212.NQ,833217.NQ,833218.NQ,833219.NQ,833222.NQ,833224.NQ,833225.NQ,833227.NQ,833228.NQ,833233.NQ,833239.NQ,833240.NQ,833242.NQ,833243.NQ,833244.NQ,833245.NQ,833246.NQ,833248.NQ,833251.NQ,833255.NQ,833256.NQ,833262.NQ,833263.NQ,833265.NQ,833267.NQ,833269.NQ,833277.NQ,833279.NQ,833281.NQ,833282.NQ,833284.NQ,833285.NQ,833286.NQ,833287.NQ,833298.NQ,833300.NQ,833301.NQ,833302.NQ,833303.NQ,833305.NQ,833306.NQ,833307.NQ,833308.NQ,833310.NQ,833316.NQ,833318.NQ,833319.NQ,833320.NQ,833324.NQ,833331.NQ,833332.NQ,833333.NQ,833334.NQ,833341.NQ,833345.NQ,833347.NQ,833349.NQ,833351.NQ,833355.NQ,833356.NQ,833369.NQ,833370.NQ,833374.NQ,833378.NQ,833381.NQ,833383.NQ,833385.NQ,833386.NQ,833390.NQ,833393.NQ,833394.NQ,833396.NQ,833399.NQ,833407.NQ,833408.NQ,833412.NQ,833420.NQ,833421.NQ,833422.NQ,833423.NQ,833426.NQ,833428.NQ,833430.NQ,833432.NQ,833434.NQ,833440.NQ,833443.NQ,833444.NQ,833453.NQ,833455.NQ,833457.NQ,833458.NQ,833462.NQ,833465.NQ,833467.NQ,833470.NQ,833472.NQ,833476.NQ,833477.NQ,833478.NQ,833479.NQ,833481.NQ,833482.NQ,833484.NQ,833485.NQ,833489.NQ,833491.NQ,833492.NQ,833493.NQ,833494.NQ,833495.NQ,833496.NQ,833499.NQ,833502.NQ,833504.NQ,833505.NQ,833507.NQ,833514.NQ,833515.NQ,833524.NQ,833526.NQ,833528.NQ,833532.NQ,833534.NQ,833535.NQ,833538.NQ,833539.NQ,833542.NQ,833543.NQ,833549.NQ,833552.NQ,833555.NQ,833556.NQ,833559.NQ,833562.NQ,833565.NQ,833567.NQ,833569.NQ,833570.NQ,833571.NQ,833572.NQ,833574.NQ,833576.NQ,833582.NQ,833585.NQ,833590.NQ,833594.NQ,833599.NQ,833603.NQ,833606.NQ,833608.NQ,833609.NQ,833611.NQ,833612.NQ,833613.NQ,833614.NQ,833617.NQ,833620.NQ,833622.NQ,833628.NQ,833629.NQ,833631.NQ,833636.NQ,833640.NQ,833648.NQ,833650.NQ,833651.NQ,833653.NQ,833654.NQ,833655.NQ,833658.NQ,833662.NQ,833663.NQ,833669.NQ,833672.NQ,833675.NQ,833680.NQ,833681.NQ,833682.NQ,833686.NQ,833687.NQ,833693.NQ,833696.NQ,833699.NQ,833700.NQ,833703.NQ,833706.NQ,833707.NQ,833711.NQ,833716.NQ,833717.NQ,833719.NQ,833723.NQ,833724.NQ,833726.NQ,833727.NQ,833735.NQ,833736.NQ,833737.NQ,833741.NQ,833746.NQ,833748.NQ,833751.NQ,833753.NQ,833755.NQ,833760.NQ,833763.NQ,833764.NQ,833765.NQ,833768.NQ,833772.NQ,833777.NQ,833778.NQ,833779.NQ,833783.NQ,833784.NQ,833785.NQ,833791.NQ,833796.NQ,833801.NQ,833802.NQ,833804.NQ,833809.NQ,833813.NQ,833814.NQ,833815.NQ,833816.NQ,833818.NQ,833821.NQ,833822.NQ,833824.NQ,833827.NQ,833830.NQ,833833.NQ,833835.NQ,833836.NQ,833838.NQ,833839.NQ,833844.NQ,833846.NQ,833848.NQ,833849.NQ,833855.NQ,833858.NQ,833859.NQ,833863.NQ,833864.NQ,833865.NQ,833866.NQ,833867.NQ,833876.NQ,833881.NQ,833884.NQ,833887.NQ,833893.NQ,833896.NQ,833898.NQ,833899.NQ,833901.NQ,833902.NQ,833903.NQ,833906.NQ,833907.NQ,833908.NQ,833909.NQ,833918.NQ,833921.NQ,833922.NQ,833923.NQ,833925.NQ,833928.NQ,833930.NQ,833934.NQ,833935.NQ,833936.NQ,833937.NQ,833939.NQ,833945.NQ,833954.NQ,833955.NQ,833962.NQ,833966.NQ,833967.NQ,833972.NQ,833976.NQ,833978.NQ,833979.NQ,833980.NQ,833981.NQ,833985.NQ,833988.NQ,833991.NQ,833997.NQ,833998.NQ,833999.NQ,834002.NQ,834005.NQ,834010.NQ,834012.NQ,834016.NQ,834018.NQ,834019.NQ,834026.NQ,834031.NQ,834037.NQ,834038.NQ,834040.NQ,834042.NQ,834043.NQ,834044.NQ,834045.NQ,834046.NQ,834048.NQ,834051.NQ,834054.NQ,834058.NQ,834063.NQ,834065.NQ,834069.NQ,834072.NQ,834073.NQ,834076.NQ,834080.NQ,834081.NQ,834087.NQ,834089.NQ,834092.NQ,834096.NQ,834101.NQ,834104.NQ,834105.NQ,834107.NQ,834110.NQ,834111.NQ,834113.NQ,834114.NQ,834118.NQ,834125.NQ,834126.NQ,834132.NQ,834134.NQ,834135.NQ,834143.NQ,834144.NQ,834146.NQ,834150.NQ,834151.NQ,834156.NQ,834162.NQ,834166.NQ,834167.NQ,834169.NQ,834170.NQ,834171.NQ,834179.NQ,834183.NQ,834184.NQ,834185.NQ,834187.NQ,834189.NQ,834193.NQ,834195.NQ,834196.NQ,834199.NQ,834202.NQ,834205.NQ,834209.NQ,834212.NQ,834213.NQ,834217.NQ,834218.NQ,834221.NQ,834222.NQ,834223.NQ,834233.NQ,834236.NQ,834239.NQ,834240.NQ,834243.NQ,834250.NQ,834253.NQ,834256.NQ,834258.NQ,834259.NQ,834260.NQ,834261.NQ,834263.NQ,834264.NQ,834267.NQ,834270.NQ,834274.NQ,834276.NQ,834277.NQ,834281.NQ,834282.NQ,834284.NQ,834288.NQ,834289.NQ,834290.NQ,834293.NQ,834295.NQ,834296.NQ,834298.NQ,834300.NQ,834306.NQ,834307.NQ,834308.NQ,834315.NQ,834316.NQ,834322.NQ,834323.NQ,834324.NQ,834325.NQ,834326.NQ,834327.NQ,834328.NQ,834329.NQ,834335.NQ,834342.NQ,834343.NQ,834345.NQ,834347.NQ,834348.NQ,834357.NQ,834359.NQ,834361.NQ,834365.NQ,834366.NQ,834367.NQ,834369.NQ,834370.NQ,834372.NQ,834375.NQ,834376.NQ,834377.NQ,834378.NQ,834387.NQ,834395.NQ,834397.NQ,834399.NQ,834401.NQ,834402.NQ,834404.NQ,834406.NQ,834408.NQ,834411.NQ,834412.NQ,834414.NQ,834418.NQ,834422.NQ,834424.NQ,834425.NQ,834426.NQ,834428.NQ,834429.NQ,834440.NQ,834443.NQ,834446.NQ,834465.NQ,834466.NQ,834468.NQ,834469.NQ,834470.NQ,834473.NQ,834476.NQ,834477.NQ,834482.NQ,834483.NQ,834486.NQ,834487.NQ,834489.NQ,834492.NQ,834495.NQ,834497.NQ,834498.NQ,834500.NQ,834503.NQ,834504.NQ,834510.NQ,834515.NQ,834517.NQ,834518.NQ,834519.NQ,834520.NQ,834529.NQ,834530.NQ,834536.NQ,834538.NQ,834539.NQ,834540.NQ,834542.NQ,834543.NQ,834546.NQ,834548.NQ,834549.NQ,834552.NQ,834557.NQ,834558.NQ,834559.NQ,834561.NQ,834562.NQ,834565.NQ,834569.NQ,834572.NQ,834577.NQ,834578.NQ,834579.NQ,834584.NQ,834585.NQ,834587.NQ,834588.NQ,834590.NQ,834591.NQ,834592.NQ,834593.NQ,834594.NQ,834596.NQ,834607.NQ,834609.NQ,834614.NQ,834616.NQ,834618.NQ,834620.NQ,834621.NQ,834623.NQ,834624.NQ,834627.NQ,834630.NQ,834631.NQ,834643.NQ,834644.NQ,834646.NQ,834648.NQ,834650.NQ,834657.NQ,834658.NQ,834667.NQ,834668.NQ,834670.NQ,834677.NQ,834678.NQ,834679.NQ,834683.NQ,834684.NQ,834685.NQ,834686.NQ,834687.NQ,834692.NQ,834696.NQ,834699.NQ,834700.NQ,834701.NQ,834702.NQ,834703.NQ,834706.NQ,834707.NQ,834712.NQ,834713.NQ,834720.NQ,834727.NQ,834728.NQ,834730.NQ,834734.NQ,834738.NQ,834740.NQ,834741.NQ,834743.NQ,834751.NQ,834752.NQ,834755.NQ,834756.NQ,834757.NQ,834759.NQ,834762.NQ,834763.NQ,834764.NQ,834766.NQ,834773.NQ,834774.NQ,834777.NQ,834780.NQ,834781.NQ,834782.NQ,834791.NQ,834792.NQ,834794.NQ,834795.NQ,834797.NQ,834800.NQ,834802.NQ,834803.NQ,834808.NQ,834811.NQ,834812.NQ,834815.NQ,834819.NQ,834822.NQ,834823.NQ,834826.NQ,834832.NQ,834833.NQ,834834.NQ,834843.NQ,834845.NQ,834849.NQ,834853.NQ,834857.NQ,834863.NQ,834866.NQ,834871.NQ,834872.NQ,834874.NQ,834879.NQ,834881.NQ,834882.NQ,834891.NQ,834892.NQ,834896.NQ,834898.NQ,834911.NQ,834912.NQ,834914.NQ,834915.NQ,834916.NQ,834919.NQ,834920.NQ,834921.NQ,834926.NQ,834927.NQ,834928.NQ,834935.NQ,834937.NQ,834938.NQ,834939.NQ,834948.NQ,834951.NQ,834952.NQ,834954.NQ,834956.NQ,834959.NQ,834960.NQ,834965.NQ,834966.NQ,834972.NQ,834974.NQ,834975.NQ,834976.NQ,834978.NQ,834982.NQ,834985.NQ,834990.NQ,834992.NQ,834997.NQ,835000.NQ,835003.NQ,835007.NQ,835011.NQ,835013.NQ,835015.NQ,835017.NQ,835020.NQ,835021.NQ,835026.NQ,835031.NQ,835034.NQ,835037.NQ,835045.NQ,835046.NQ,835047.NQ,835049.NQ,835050.NQ,835051.NQ,835052.NQ,835058.NQ,835061.NQ,835062.NQ,835063.NQ,835067.NQ,835068.NQ,835070.NQ,835071.NQ,835072.NQ,835074.NQ,835075.NQ,835080.NQ,835081.NQ,835084.NQ,835085.NQ,835086.NQ,835087.NQ,835091.NQ,835094.NQ,835101.NQ,835102.NQ,835104.NQ,835105.NQ,835106.NQ,835107.NQ,835111.NQ,835117.NQ,835120.NQ,835126.NQ,835128.NQ,835133.NQ,835135.NQ,835138.NQ,835141.NQ,835152.NQ,835156.NQ,835157.NQ,835159.NQ,835164.NQ,835168.NQ,835169.NQ,835171.NQ,835173.NQ,835175.NQ,835180.NQ,835181.NQ,835187.NQ,835191.NQ,835192.NQ,835193.NQ,835195.NQ,835196.NQ,835198.NQ,835203.NQ,835205.NQ,835208.NQ,835211.NQ,835217.NQ,835222.NQ,835223.NQ,835228.NQ,835230.NQ,835231.NQ,835234.NQ,835244.NQ,835245.NQ,835246.NQ,835250.NQ,835256.NQ,835257.NQ,835266.NQ,835269.NQ,835278.NQ,835281.NQ,835282.NQ,835283.NQ,835286.NQ,835287.NQ,835290.NQ,835291.NQ,835293.NQ,835295.NQ,835296.NQ,835297.NQ,835298.NQ,835300.NQ,835301.NQ,835307.NQ,835311.NQ,835312.NQ,835314.NQ,835316.NQ,835319.NQ,835322.NQ,835323.NQ,835328.NQ,835330.NQ,835333.NQ,835335.NQ,835338.NQ,835340.NQ,835342.NQ,835347.NQ,835352.NQ,835354.NQ,835359.NQ,835361.NQ,835363.NQ,835364.NQ,835365.NQ,835367.NQ,835370.NQ,835372.NQ,835373.NQ,835374.NQ,835375.NQ,835382.NQ,835383.NQ,835387.NQ,835393.NQ,835395.NQ,835401.NQ,835405.NQ,835409.NQ,835410.NQ,835414.NQ,835415.NQ,835416.NQ,835417.NQ,835421.NQ,835425.NQ,835426.NQ,835427.NQ,835429.NQ,835431.NQ,835435.NQ,835438.NQ,835441.NQ,835442.NQ,835445.NQ,835446.NQ,835447.NQ,835448.NQ,835450.NQ,835452.NQ,835453.NQ,835454.NQ,835461.NQ,835463.NQ,835471.NQ,835473.NQ,835474.NQ,835475.NQ,835476.NQ,835487.NQ,835488.NQ,835492.NQ,835495.NQ,835496.NQ,835500.NQ,835505.NQ,835511.NQ,835512.NQ,835513.NQ,835517.NQ,835521.NQ,835525.NQ,835526.NQ,835527.NQ,835532.NQ,835533.NQ,835534.NQ,835536.NQ,835538.NQ,835540.NQ,835541.NQ,835542.NQ,835544.NQ,835545.NQ,835553.NQ,835555.NQ,835558.NQ,835563.NQ,835565.NQ,835567.NQ,835578.NQ,835579.NQ,835582.NQ,835584.NQ,835591.NQ,835595.NQ,835596.NQ,835597.NQ,835599.NQ,835600.NQ,835602.NQ,835605.NQ,835608.NQ,835609.NQ,835616.NQ,835620.NQ,835631.NQ,835632.NQ,835634.NQ,835635.NQ,835636.NQ,835637.NQ,835642.NQ,835647.NQ,835654.NQ,835655.NQ,835659.NQ,835665.NQ,835667.NQ,835669.NQ,835675.NQ,835678.NQ,835679.NQ,835680.NQ,835682.NQ,835684.NQ,835685.NQ,835688.NQ,835690.NQ,835695.NQ,835697.NQ,835698.NQ,835702.NQ,835705.NQ,835708.NQ,835709.NQ,835714.NQ,835718.NQ,835719.NQ,835721.NQ,835723.NQ,835725.NQ,835728.NQ,835729.NQ,835735.NQ,835746.NQ,835751.NQ,835755.NQ,835756.NQ,835759.NQ,835762.NQ,835763.NQ,835764.NQ,835765.NQ,835773.NQ,835775.NQ,835778.NQ,835779.NQ,835785.NQ,835787.NQ,835788.NQ,835791.NQ,835792.NQ,835798.NQ,835800.NQ,835801.NQ,835804.NQ,835805.NQ,835806.NQ,835808.NQ,835812.NQ,835817.NQ,835818.NQ,835827.NQ,835833.NQ,835834.NQ,835839.NQ,835841.NQ,835843.NQ,835847.NQ,835848.NQ,835849.NQ,835850.NQ,835851.NQ,835852.NQ,835853.NQ,835856.NQ,835860.NQ,835861.NQ,835862.NQ,835872.NQ,835877.NQ,835881.NQ,835883.NQ,835888.NQ,835889.NQ,835891.NQ,835894.NQ,835898.NQ,835899.NQ,835901.NQ,835906.NQ,835907.NQ,835909.NQ,835910.NQ,835911.NQ,835912.NQ,835913.NQ,835914.NQ,835924.NQ,835925.NQ,835927.NQ,835936.NQ,835940.NQ,835947.NQ,835948.NQ,835957.NQ,835959.NQ,835960.NQ,835961.NQ,835963.NQ,835968.NQ,835969.NQ,835970.NQ,835979.NQ,835982.NQ,835983.NQ,835987.NQ,835993.NQ,835995.NQ,835996.NQ,836002.NQ,836004.NQ,836005.NQ,836008.NQ,836011.NQ,836012.NQ,836014.NQ,836015.NQ,836024.NQ,836027.NQ,836028.NQ,836029.NQ,836030.NQ,836032.NQ,836033.NQ,836036.NQ,836041.NQ,836047.NQ,836050.NQ,836056.NQ,836058.NQ,836059.NQ,836062.NQ,836070.NQ,836072.NQ,836073.NQ,836075.NQ,836076.NQ,836078.NQ,836081.NQ,836089.NQ,836093.NQ,836098.NQ,836100.NQ,836106.NQ,836108.NQ,836109.NQ,836113.NQ,836115.NQ,836117.NQ,836118.NQ,836122.NQ,836123.NQ,836125.NQ,836126.NQ,836129.NQ,836136.NQ,836138.NQ,836139.NQ,836145.NQ,836153.NQ,836156.NQ,836159.NQ,836160.NQ,836161.NQ,836162.NQ,836163.NQ,836164.NQ,836166.NQ,836170.NQ,836171.NQ,836179.NQ,836183.NQ,836185.NQ,836186.NQ,836189.NQ,836190.NQ,836191.NQ,836193.NQ,836195.NQ,836196.NQ,836205.NQ,836206.NQ,836208.NQ,836209.NQ,836210.NQ,836212.NQ,836215.NQ,836216.NQ,836217.NQ,836221.NQ,836225.NQ,836228.NQ,836230.NQ,836234.NQ,836235.NQ,836236.NQ,836237.NQ,836241.NQ,836248.NQ,836249.NQ,836253.NQ,836254.NQ,836255.NQ,836256.NQ,836259.NQ,836261.NQ,836264.NQ,836266.NQ,836274.NQ,836275.NQ,836277.NQ,836278.NQ,836281.NQ,836284.NQ,836286.NQ,836287.NQ,836291.NQ,836297.NQ,836300.NQ,836303.NQ,836305.NQ,836312.NQ,836314.NQ,836315.NQ,836316.NQ,836318.NQ,836319.NQ,836321.NQ,836325.NQ,836329.NQ,836331.NQ,836332.NQ,836333.NQ,836337.NQ,836339.NQ,836342.NQ,836344.NQ,836347.NQ,836348.NQ,836357.NQ,836362.NQ,836364.NQ,836371.NQ,836373.NQ,836374.NQ,836377.NQ,836378.NQ,836381.NQ,836384.NQ,836386.NQ,836388.NQ,836389.NQ,836390.NQ,836392.NQ,836396.NQ,836399.NQ,836405.NQ,836409.NQ,836415.NQ,836417.NQ,836418.NQ,836419.NQ,836421.NQ,836425.NQ,836432.NQ,836437.NQ,836439.NQ,836440.NQ,836442.NQ,836443.NQ,836450.NQ,836451.NQ,836452.NQ,836453.NQ,836455.NQ,836456.NQ,836458.NQ,836459.NQ,836460.NQ,836464.NQ,836468.NQ,836472.NQ,836475.NQ,836476.NQ,836477.NQ,836479.NQ,836482.NQ,836491.NQ,836503.NQ,836504.NQ,836507.NQ,836508.NQ,836509.NQ,836514.NQ,836521.NQ,836523.NQ,836530.NQ,836531.NQ,836533.NQ,836537.NQ,836542.NQ,836545.NQ,836547.NQ,836550.NQ,836552.NQ,836556.NQ,836557.NQ,836560.NQ,836562.NQ,836572.NQ,836575.NQ,836577.NQ,836579.NQ,836580.NQ,836583.NQ,836584.NQ,836589.NQ,836590.NQ,836592.NQ,836595.NQ,836596.NQ,836597.NQ,836602.NQ,836603.NQ,836605.NQ,836609.NQ,836610.NQ,836613.NQ,836617.NQ,836618.NQ,836619.NQ,836624.NQ,836625.NQ,836632.NQ,836635.NQ,836636.NQ,836639.NQ,836641.NQ,836642.NQ,836645.NQ,836647.NQ,836649.NQ,836650.NQ,836652.NQ,836657.NQ,836658.NQ,836660.NQ,836663.NQ,836665.NQ,836666.NQ,836667.NQ,836668.NQ,836669.NQ,836670.NQ,836674.NQ,836678.NQ,836679.NQ,836683.NQ,836688.NQ,836690.NQ,836691.NQ,836692.NQ,836696.NQ,836699.NQ,836703.NQ,836708.NQ,836715.NQ,836717.NQ,836719.NQ,836727.NQ,836733.NQ,836734.NQ,836735.NQ,836741.NQ,836742.NQ,836743.NQ,836744.NQ,836745.NQ,836746.NQ,836753.NQ,836755.NQ,836758.NQ,836761.NQ,836770.NQ,836772.NQ,836774.NQ,836775.NQ,836777.NQ,836779.NQ,836780.NQ,836782.NQ,836783.NQ,836786.NQ,836790.NQ,836792.NQ,836793.NQ,836797.NQ,836803.NQ,836805.NQ,836809.NQ,836810.NQ,836811.NQ,836816.NQ,836818.NQ,836820.NQ,836823.NQ,836825.NQ,836827.NQ,836829.NQ,836830.NQ,836831.NQ,836834.NQ,836836.NQ,836837.NQ,836843.NQ,836846.NQ,836847.NQ,836851.NQ,836858.NQ,836861.NQ,836863.NQ,836864.NQ,836866.NQ,836867.NQ,836870.NQ,836872.NQ,836874.NQ,836876.NQ,836877.NQ,836879.NQ,836881.NQ,836883.NQ,836885.NQ,836888.NQ,836890.NQ,836893.NQ,836898.NQ,836903.NQ,836907.NQ,836908.NQ,836909.NQ,836910.NQ,836911.NQ,836913.NQ,836915.NQ,836924.NQ,836930.NQ,836932.NQ,836935.NQ,836939.NQ,836941.NQ,836944.NQ,836946.NQ,836949.NQ,836954.NQ,836955.NQ,836961.NQ,836967.NQ,836972.NQ,836974.NQ,836976.NQ,836977.NQ,836978.NQ,836979.NQ,836981.NQ,836984.NQ,836989.NQ,836991.NQ,836994.NQ,836998.NQ,837000.NQ,837003.NQ,837006.NQ,837008.NQ,837009.NQ,837011.NQ,837012.NQ,837014.NQ,837017.NQ,837018.NQ,837019.NQ,837021.NQ,837023.NQ,837033.NQ,837034.NQ,837035.NQ,837036.NQ,837040.NQ,837042.NQ,837044.NQ,837051.NQ,837052.NQ,837053.NQ,837058.NQ,837062.NQ,837063.NQ,837065.NQ,837067.NQ,837070.NQ,837072.NQ,837073.NQ,837074.NQ,837078.NQ,837081.NQ,837083.NQ,837087.NQ,837089.NQ,837090.NQ,837094.NQ,837095.NQ,837098.NQ,837105.NQ,837106.NQ,837108.NQ,837125.NQ,837126.NQ,837129.NQ,837133.NQ,837134.NQ,837135.NQ,837136.NQ,837137.NQ,837138.NQ,837143.NQ,837146.NQ,837148.NQ,837150.NQ,837153.NQ,837157.NQ,837173.NQ,837174.NQ,837177.NQ,837178.NQ,837180.NQ,837181.NQ,837185.NQ,837187.NQ,837189.NQ,837193.NQ,837200.NQ,837201.NQ,837207.NQ,837210.NQ,837211.NQ,837215.NQ,837217.NQ,837220.NQ,837222.NQ,837224.NQ,837225.NQ,837226.NQ,837227.NQ,837228.NQ,837231.NQ,837232.NQ,837236.NQ,837251.NQ,837253.NQ,837254.NQ,837256.NQ,837261.NQ,837263.NQ,837264.NQ,837269.NQ,837273.NQ,837274.NQ,837276.NQ,837283.NQ,837284.NQ,837287.NQ,837288.NQ,837289.NQ,837291.NQ,837298.NQ,837299.NQ,837301.NQ,837302.NQ,837303.NQ,837304.NQ,837305.NQ,837308.NQ,837312.NQ,837314.NQ,837317.NQ,837318.NQ,837319.NQ,837320.NQ,837323.NQ,837324.NQ,837326.NQ,837327.NQ,837334.NQ,837336.NQ,837340.NQ,837341.NQ,837345.NQ,837347.NQ,837348.NQ,837350.NQ,837351.NQ,837353.NQ,837355.NQ,837363.NQ,837368.NQ,837370.NQ,837371.NQ,837372.NQ,837373.NQ,837375.NQ,837377.NQ,837378.NQ,837383.NQ,837384.NQ,837385.NQ,837388.NQ,837392.NQ,837393.NQ,837396.NQ,837399.NQ,837402.NQ,837403.NQ,837404.NQ,837406.NQ,837408.NQ,837424.NQ,837429.NQ,837430.NQ,837432.NQ,837441.NQ,837443.NQ,837445.NQ,837447.NQ,837450.NQ,837453.NQ,837455.NQ,837459.NQ,837461.NQ,837477.NQ,837479.NQ,837487.NQ,837491.NQ,837492.NQ,837494.NQ,837497.NQ,837506.NQ,837509.NQ,837510.NQ,837511.NQ,837512.NQ,837519.NQ,837522.NQ,837523.NQ,837524.NQ,837525.NQ,837526.NQ,837527.NQ,837528.NQ,837530.NQ,837531.NQ,837535.NQ,837549.NQ,837550.NQ,837552.NQ,837555.NQ,837557.NQ,837558.NQ,837560.NQ,837561.NQ,837566.NQ,837567.NQ,837569.NQ,837574.NQ,837576.NQ,837577.NQ,837579.NQ,837580.NQ,837581.NQ,837585.NQ,837588.NQ,837591.NQ,837592.NQ,837595.NQ,837601.NQ,837607.NQ,837608.NQ,837609.NQ,837611.NQ,837614.NQ,837617.NQ,837619.NQ,837620.NQ,837630.NQ,837631.NQ,837633.NQ,837637.NQ,837641.NQ,837644.NQ,837646.NQ,837651.NQ,837653.NQ,837654.NQ,837656.NQ,837661.NQ,837662.NQ,837664.NQ,837666.NQ,837667.NQ,837668.NQ,837670.NQ,837674.NQ,837680.NQ,837681.NQ,837685.NQ,837687.NQ,837690.NQ,837691.NQ,837694.NQ,837695.NQ,837697.NQ,837698.NQ,837699.NQ,837701.NQ,837704.NQ,837707.NQ,837710.NQ,837711.NQ,837718.NQ,837719.NQ,837720.NQ,837721.NQ,837726.NQ,837728.NQ,837729.NQ,837733.NQ,837734.NQ,837741.NQ,837742.NQ,837743.NQ,837745.NQ,837747.NQ,837748.NQ,837749.NQ,837758.NQ,837761.NQ,837766.NQ,837768.NQ,837773.NQ,837777.NQ,837779.NQ,837782.NQ,837784.NQ,837785.NQ,837793.NQ,837796.NQ,837798.NQ,837809.NQ,837810.NQ,837814.NQ,837815.NQ,837816.NQ,837819.NQ,837823.NQ,837826.NQ,837831.NQ,837833.NQ,837835.NQ,837836.NQ,837837.NQ,837838.NQ,837840.NQ,837845.NQ,837851.NQ,837852.NQ,837856.NQ,837857.NQ,837863.NQ,837864.NQ,837866.NQ,837867.NQ,837868.NQ,837869.NQ,837870.NQ,837874.NQ,837877.NQ,837879.NQ,837880.NQ,837881.NQ,837887.NQ,837889.NQ,837891.NQ,837892.NQ,837893.NQ,837894.NQ,837899.NQ,837907.NQ,837912.NQ,837918.NQ,837919.NQ,837920.NQ,837922.NQ,837925.NQ,837927.NQ,837929.NQ,837931.NQ,837932.NQ,837934.NQ,837935.NQ,837936.NQ,837940.NQ,837941.NQ,837949.NQ,837950.NQ,837952.NQ,837953.NQ,837955.NQ,837961.NQ,837965.NQ,837967.NQ,837968.NQ,837971.NQ,837977.NQ,837979.NQ,837980.NQ,837982.NQ,837983.NQ,837986.NQ,837988.NQ,837989.NQ,837991.NQ,837993.NQ,837995.NQ,837997.NQ,837998.NQ,838010.NQ,838012.NQ,838017.NQ,838020.NQ,838028.NQ,838029.NQ,838031.NQ,838032.NQ,838033.NQ,838036.NQ,838041.NQ,838042.NQ,838046.NQ,838049.NQ,838054.NQ,838060.NQ,838064.NQ,838067.NQ,838070.NQ,838071.NQ,838072.NQ,838075.NQ,838078.NQ,838082.NQ,838089.NQ,838091.NQ,838092.NQ,838093.NQ,838094.NQ,838095.NQ,838097.NQ,838099.NQ,838102.NQ,838103.NQ,838104.NQ,838109.NQ,838114.NQ,838115.NQ,838116.NQ,838118.NQ,838122.NQ,838128.NQ,838129.NQ,838131.NQ,838137.NQ,838140.NQ,838141.NQ,838146.NQ,838149.NQ,838150.NQ,838152.NQ,838153.NQ,838158.NQ,838161.NQ,838162.NQ,838164.NQ,838168.NQ,838172.NQ,838174.NQ,838176.NQ,838182.NQ,838183.NQ,838184.NQ,838187.NQ,838190.NQ,838194.NQ,838196.NQ,838197.NQ,838203.NQ,838204.NQ,838209.NQ,838211.NQ,838213.NQ,838214.NQ,838221.NQ,838222.NQ,838236.NQ,838238.NQ,838239.NQ,838240.NQ,838241.NQ,838242.NQ,838243.NQ,838246.NQ,838247.NQ,838248.NQ,838250.NQ,838251.NQ,838257.NQ,838260.NQ,838264.NQ,838268.NQ,838269.NQ,838273.NQ,838274.NQ,838279.NQ,838282.NQ,838283.NQ,838284.NQ,838288.NQ,838293.NQ,838296.NQ,838297.NQ,838300.NQ,838309.NQ,838310.NQ,838312.NQ,838313.NQ,838314.NQ,838316.NQ,838319.NQ,838322.NQ,838324.NQ,838327.NQ,838328.NQ,838330.NQ,838331.NQ,838332.NQ,838334.NQ,838335.NQ,838337.NQ,838344.NQ,838351.NQ,838354.NQ,838355.NQ,838357.NQ,838362.NQ,838363.NQ,838364.NQ,838365.NQ,838366.NQ,838369.NQ,838372.NQ,838374.NQ,838379.NQ,838381.NQ,838384.NQ,838386.NQ,838388.NQ,838392.NQ,838394.NQ,838395.NQ,838399.NQ,838404.NQ,838408.NQ,838409.NQ,838410.NQ,838411.NQ,838413.NQ,838414.NQ,838415.NQ,838418.NQ,838419.NQ,838425.NQ,838426.NQ,838428.NQ,838430.NQ,838432.NQ,838440.NQ,838443.NQ,838448.NQ,838449.NQ,838451.NQ,838453.NQ,838456.NQ,838457.NQ,838458.NQ,838460.NQ,838463.NQ,838464.NQ,838469.NQ,838470.NQ,838471.NQ,838474.NQ,838476.NQ,838477.NQ,838478.NQ,838479.NQ,838481.NQ,838482.NQ,838483.NQ,838484.NQ,838486.NQ,838491.NQ,838497.NQ,838498.NQ,838501.NQ,838502.NQ,838504.NQ,838508.NQ,838511.NQ,838512.NQ,838514.NQ,838518.NQ,838519.NQ,838522.NQ,838523.NQ,838526.NQ,838529.NQ,838531.NQ,838533.NQ,838535.NQ,838536.NQ,838537.NQ,838541.NQ,838542.NQ,838547.NQ,838549.NQ,838550.NQ,838553.NQ,838558.NQ,838559.NQ,838561.NQ,838563.NQ,838564.NQ,838566.NQ,838569.NQ,838570.NQ,838571.NQ,838573.NQ,838574.NQ,838576.NQ,838578.NQ,838582.NQ,838584.NQ,838587.NQ,838589.NQ,838590.NQ,838593.NQ,838595.NQ,838597.NQ,838598.NQ,838602.NQ,838606.NQ,838615.NQ,838616.NQ,838618.NQ,838623.NQ,838624.NQ,838626.NQ,838629.NQ,838632.NQ,838635.NQ,838637.NQ,838644.NQ,838646.NQ,838647.NQ,838649.NQ,838651.NQ,838654.NQ,838655.NQ,838656.NQ,838659.NQ,838661.NQ,838664.NQ,838665.NQ,838667.NQ,838669.NQ,838671.NQ,838672.NQ,838675.NQ,838686.NQ,838687.NQ,838691.NQ,838693.NQ,838694.NQ,838696.NQ,838701.NQ,838704.NQ,838707.NQ,838708.NQ,838712.NQ,838714.NQ,838721.NQ,838723.NQ,838726.NQ,838729.NQ,838736.NQ,838737.NQ,838740.NQ,838741.NQ,838744.NQ,838749.NQ,838752.NQ,838753.NQ,838754.NQ,838756.NQ,838757.NQ,838758.NQ,838760.NQ,838762.NQ,838766.NQ,838767.NQ,838768.NQ,838770.NQ,838775.NQ,838784.NQ,838786.NQ,838790.NQ,838791.NQ,838793.NQ,838795.NQ,838796.NQ,838798.NQ,838799.NQ,838800.NQ,838801.NQ,838802.NQ,838803.NQ,838804.NQ,838807.NQ,838809.NQ,838811.NQ,838812.NQ,838813.NQ,838814.NQ,838823.NQ,838826.NQ,838829.NQ,838834.NQ,838835.NQ,838837.NQ,838840.NQ,838841.NQ,838843.NQ,838850.NQ,838854.NQ,838856.NQ,838857.NQ,838858.NQ,838861.NQ,838865.NQ,838866.NQ,838868.NQ,838875.NQ,838878.NQ,838879.NQ,838883.NQ,838893.NQ,838894.NQ,838897.NQ,838898.NQ,838899.NQ,838900.NQ,838901.NQ,838903.NQ,838906.NQ,838909.NQ,838910.NQ,838911.NQ,838915.NQ,838916.NQ,838917.NQ,838925.NQ,838927.NQ,838928.NQ,838933.NQ,838934.NQ,838939.NQ,838940.NQ,838941.NQ,838943.NQ,838944.NQ,838949.NQ,838951.NQ,838952.NQ,838955.NQ,838956.NQ,838957.NQ,838959.NQ,838961.NQ,838970.NQ,838978.NQ,838980.NQ,838981.NQ,838982.NQ,838983.NQ,838984.NQ,838987.NQ,838989.NQ,838990.NQ,838992.NQ,838994.NQ,838998.NQ,839003.NQ,839005.NQ,839009.NQ,839010.NQ,839012.NQ,839013.NQ,839018.NQ,839024.NQ,839028.NQ,839029.NQ,839030.NQ,839031.NQ,839032.NQ,839033.NQ,839035.NQ,839036.NQ,839037.NQ,839038.NQ,839041.NQ,839044.NQ,839045.NQ,839047.NQ,839054.NQ,839056.NQ,839058.NQ,839067.NQ,839071.NQ,839073.NQ,839074.NQ,839076.NQ,839080.NQ,839081.NQ,839084.NQ,839085.NQ,839086.NQ,839089.NQ,839093.NQ,839099.NQ,839101.NQ,839109.NQ,839110.NQ,839113.NQ,839114.NQ,839118.NQ,839120.NQ,839121.NQ,839123.NQ,839125.NQ,839131.NQ,839132.NQ,839133.NQ,839137.NQ,839139.NQ,839140.NQ,839142.NQ,839143.NQ,839145.NQ,839146.NQ,839148.NQ,839152.NQ,839153.NQ,839154.NQ,839155.NQ,839156.NQ,839161.NQ,839163.NQ,839164.NQ,839173.NQ,839174.NQ,839175.NQ,839178.NQ,839180.NQ,839181.NQ,839182.NQ,839187.NQ,839188.NQ,839189.NQ,839190.NQ,839192.NQ,839193.NQ,839196.NQ,839198.NQ,839199.NQ,839201.NQ,839202.NQ,839204.NQ,839205.NQ,839206.NQ,839207.NQ,839209.NQ,839211.NQ,839213.NQ,839216.NQ,839218.NQ,839220.NQ,839222.NQ,839223.NQ,839224.NQ,839226.NQ,839228.NQ,839229.NQ,839231.NQ,839232.NQ,839233.NQ,839237.NQ,839239.NQ,839241.NQ,839244.NQ,839253.NQ,839254.NQ,839255.NQ,839258.NQ,839260.NQ,839266.NQ,839270.NQ,839275.NQ,839277.NQ,839278.NQ,839279.NQ,839281.NQ,839284.NQ,839285.NQ,839289.NQ,839291.NQ,839292.NQ,839295.NQ,839296.NQ,839302.NQ,839305.NQ,839306.NQ,839308.NQ,839309.NQ,839310.NQ,839312.NQ,839315.NQ,839319.NQ,839321.NQ,839322.NQ,839324.NQ,839326.NQ,839327.NQ,839330.NQ,839332.NQ,839335.NQ,839339.NQ,839340.NQ,839341.NQ,839342.NQ,839350.NQ,839351.NQ,839354.NQ,839355.NQ,839356.NQ,839357.NQ,839362.NQ,839364.NQ,839365.NQ,839367.NQ,839370.NQ,839373.NQ,839376.NQ,839379.NQ,839381.NQ,839385.NQ,839387.NQ,839389.NQ,839391.NQ,839392.NQ,839395.NQ,839406.NQ,839409.NQ,839410.NQ,839411.NQ,839416.NQ,839418.NQ,839421.NQ,839432.NQ,839433.NQ,839435.NQ,839438.NQ,839443.NQ,839444.NQ,839446.NQ,839447.NQ,839452.NQ,839453.NQ,839454.NQ,839458.NQ,839461.NQ,839463.NQ,839466.NQ,839468.NQ,839470.NQ,839472.NQ,839476.NQ,839477.NQ,839481.NQ,839482.NQ,839484.NQ,839485.NQ,839489.NQ,839490.NQ,839491.NQ,839493.NQ,839497.NQ,839499.NQ,839501.NQ,839502.NQ,839503.NQ,839506.NQ,839507.NQ,839509.NQ,839511.NQ,839513.NQ,839514.NQ,839515.NQ,839517.NQ,839520.NQ,839523.NQ,839531.NQ,839532.NQ,839535.NQ,839537.NQ,839539.NQ,839540.NQ,839545.NQ,839546.NQ,839547.NQ,839549.NQ,839553.NQ,839555.NQ,839558.NQ,839559.NQ,839560.NQ,839561.NQ,839563.NQ,839567.NQ,839568.NQ,839572.NQ,839574.NQ,839581.NQ,839582.NQ,839583.NQ,839587.NQ,839589.NQ,839593.NQ,839596.NQ,839599.NQ,839605.NQ,839606.NQ,839609.NQ,839610.NQ,839612.NQ,839613.NQ,839618.NQ,839621.NQ,839622.NQ,839625.NQ,839629.NQ,839630.NQ,839631.NQ,839633.NQ,839635.NQ,839640.NQ,839642.NQ,839643.NQ,839646.NQ,839649.NQ,839657.NQ,839659.NQ,839666.NQ,839667.NQ,839668.NQ,839672.NQ,839675.NQ,839679.NQ,839681.NQ,839683.NQ,839684.NQ,839686.NQ,839692.NQ,839694.NQ,839696.NQ,839697.NQ,839698.NQ,839707.NQ,839710.NQ,839713.NQ,839717.NQ,839718.NQ,839719.NQ,839720.NQ,839723.NQ,839726.NQ,839731.NQ,839732.NQ,839733.NQ,839740.NQ,839742.NQ,839745.NQ,839746.NQ,839750.NQ,839751.NQ,839754.NQ,839755.NQ,839756.NQ,839759.NQ,839762.NQ,839765.NQ,839768.NQ,839769.NQ,839770.NQ,839773.NQ,839774.NQ,839775.NQ,839777.NQ,839778.NQ,839780.NQ,839782.NQ,839783.NQ,839785.NQ,839786.NQ,839788.NQ,839792.NQ,839796.NQ,839799.NQ,839801.NQ,839805.NQ,839806.NQ,839808.NQ,839809.NQ,839813.NQ,839817.NQ,839820.NQ,839821.NQ,839822.NQ,839825.NQ,839826.NQ,839830.NQ,839833.NQ,839834.NQ,839836.NQ,839837.NQ,839838.NQ,839839.NQ,839840.NQ,839841.NQ,839842.NQ,839845.NQ,839847.NQ,839853.NQ,839856.NQ,839857.NQ,839865.NQ,839867.NQ,839870.NQ,839871.NQ,839874.NQ,839879.NQ,839880.NQ,839881.NQ,839884.NQ,839888.NQ,839890.NQ,839893.NQ,839894.NQ,839898.NQ,839901.NQ,839904.NQ,839907.NQ,839909.NQ,839911.NQ,839913.NQ,839914.NQ,839915.NQ,839917.NQ,839920.NQ,839922.NQ,839926.NQ,839929.NQ,839934.NQ,839938.NQ,839939.NQ,839942.NQ,839943.NQ,839944.NQ,839951.NQ,839953.NQ,839954.NQ,839962.NQ,839964.NQ,839965.NQ,839966.NQ,839967.NQ,839969.NQ,839977.NQ,839979.NQ,839980.NQ,839981.NQ,839982.NQ,839983.NQ,839984.NQ,839985.NQ,839986.NQ,839987.NQ,839990.NQ,839993.NQ,839994.NQ,839998.NQ,839999.NQ,870000.NQ,870001.NQ,870003.NQ,870005.NQ,870009.NQ,870010.NQ,870014.NQ,870016.NQ,870020.NQ,870021.NQ,870024.NQ,870026.NQ,870029.NQ,870030.NQ,870031.NQ,870037.NQ,870044.NQ,870046.NQ,870049.NQ,870050.NQ,870052.NQ,870053.NQ,870055.NQ,870058.NQ,870061.NQ,870065.NQ,870066.NQ,870070.NQ,870071.NQ,870072.NQ,870073.NQ,870079.NQ,870080.NQ,870086.NQ,870092.NQ,870095.NQ,870096.NQ,870098.NQ,870099.NQ,870100.NQ,870101.NQ,870102.NQ,870104.NQ,870110.NQ,870111.NQ,870115.NQ,870116.NQ,870119.NQ,870122.NQ,870123.NQ,870125.NQ,870126.NQ,870130.NQ,870131.NQ,870134.NQ,870139.NQ,870140.NQ,870141.NQ,870144.NQ,870145.NQ,870149.NQ,870150.NQ,870154.NQ,870157.NQ,870158.NQ,870160.NQ,870164.NQ,870166.NQ,870167.NQ,870170.NQ,870172.NQ,870175.NQ,870176.NQ,870177.NQ,870178.NQ,870182.NQ,870183.NQ,870186.NQ,870188.NQ,870191.NQ,870195.NQ,870198.NQ,870202.NQ,870208.NQ,870212.NQ,870213.NQ,870215.NQ,870219.NQ,870220.NQ,870221.NQ,870223.NQ,870225.NQ,870227.NQ,870229.NQ,870231.NQ,870232.NQ,870233.NQ,870235.NQ,870236.NQ,870238.NQ,870250.NQ,870251.NQ,870253.NQ,870256.NQ,870259.NQ,870260.NQ,870262.NQ,870263.NQ,870264.NQ,870268.NQ,870270.NQ,870271.NQ,870275.NQ,870277.NQ,870278.NQ,870281.NQ,870282.NQ,870283.NQ,870284.NQ,870286.NQ,870287.NQ,870288.NQ,870292.NQ,870293.NQ,870294.NQ,870296.NQ,870298.NQ,870301.NQ,870302.NQ,870304.NQ,870305.NQ,870307.NQ,870308.NQ,870309.NQ,870316.NQ,870317.NQ,870318.NQ,870319.NQ,870323.NQ,870324.NQ,870328.NQ,870331.NQ,870334.NQ,870335.NQ,870336.NQ,870337.NQ,870343.NQ,870345.NQ,870350.NQ,870353.NQ,870355.NQ,870356.NQ,870358.NQ,870359.NQ,870361.NQ,870362.NQ,870364.NQ,870365.NQ,870366.NQ,870368.NQ,870369.NQ,870370.NQ,870371.NQ,870372.NQ,870374.NQ,870376.NQ,870382.NQ,870383.NQ,870386.NQ,870388.NQ,870389.NQ,870390.NQ,870395.NQ,870398.NQ,870405.NQ,870406.NQ,870409.NQ,870411.NQ,870412.NQ,870415.NQ,870417.NQ,870418.NQ,870419.NQ,870420.NQ,870421.NQ,870426.NQ,870427.NQ,870428.NQ,870429.NQ,870431.NQ,870432.NQ,870437.NQ,870438.NQ,870440.NQ,870441.NQ,870443.NQ,870446.NQ,870448.NQ,870450.NQ,870451.NQ,870452.NQ,870453.NQ,870457.NQ,870459.NQ,870470.NQ,870472.NQ,870475.NQ,870480.NQ,870482.NQ,870483.NQ,870487.NQ,870488.NQ,870489.NQ,870491.NQ,870492.NQ,870494.NQ,870496.NQ,870500.NQ,870502.NQ,870505.NQ,870506.NQ,870507.NQ,870511.NQ,870512.NQ,870519.NQ,870523.NQ,870524.NQ,870529.NQ,870530.NQ,870531.NQ,870532.NQ,870534.NQ,870535.NQ,870536.NQ,870538.NQ,870539.NQ,870542.NQ,870543.NQ,870544.NQ,870548.NQ,870549.NQ,870550.NQ,870551.NQ,870552.NQ,870553.NQ,870555.NQ,870556.NQ,870557.NQ,870558.NQ,870559.NQ,870566.NQ,870570.NQ,870571.NQ,870572.NQ,870573.NQ,870579.NQ,870581.NQ,870583.NQ,870586.NQ,870592.NQ,870593.NQ,870600.NQ,870601.NQ,870603.NQ,870605.NQ,870608.NQ,870614.NQ,870615.NQ,870618.NQ,870619.NQ,870620.NQ,870621.NQ,870622.NQ,870623.NQ,870624.NQ,870625.NQ,870626.NQ,870627.NQ,870629.NQ,870632.NQ,870633.NQ,870635.NQ,870637.NQ,870643.NQ,870646.NQ,870647.NQ,870652.NQ,870654.NQ,870656.NQ,870659.NQ,870662.NQ,870667.NQ,870669.NQ,870678.NQ,870679.NQ,870682.NQ,870686.NQ,870689.NQ,870690.NQ,870691.NQ,870693.NQ,870696.NQ,870697.NQ,870702.NQ,870706.NQ,870707.NQ,870710.NQ,870711.NQ,870712.NQ,870714.NQ,870716.NQ,870717.NQ,870719.NQ,870724.NQ,870725.NQ,870726.NQ,870727.NQ,870730.NQ,870733.NQ,870736.NQ,870737.NQ,870746.NQ,870749.NQ,870752.NQ,870753.NQ,870754.NQ,870756.NQ,870757.NQ,870760.NQ,870762.NQ,870764.NQ,870768.NQ,870772.NQ,870774.NQ,870775.NQ,870777.NQ,870779.NQ,870780.NQ,870781.NQ,870783.NQ,870784.NQ,870787.NQ,870788.NQ,870790.NQ,870792.NQ,870796.NQ,870797.NQ,870799.NQ,870800.NQ,870801.NQ,870802.NQ,870804.NQ,870805.NQ,870806.NQ,870810.NQ,870811.NQ,870812.NQ,870814.NQ,870815.NQ,870818.NQ,870819.NQ,870820.NQ,870821.NQ,870822.NQ,870825.NQ,870832.NQ,870834.NQ,870838.NQ,870840.NQ,870841.NQ,870844.NQ,870847.NQ,870848.NQ,870849.NQ,870851.NQ,870853.NQ,870855.NQ,870856.NQ,870859.NQ,870861.NQ,870869.NQ,870870.NQ,870874.NQ,870875.NQ,870876.NQ,870877.NQ,870879.NQ,870880.NQ,870881.NQ,870884.NQ,870886.NQ,870887.NQ,870888.NQ,870889.NQ,870890.NQ,870891.NQ,870892.NQ,870893.NQ,870895.NQ,870897.NQ,870899.NQ,870906.NQ,870909.NQ,870911.NQ,870912.NQ,870914.NQ,870916.NQ,870917.NQ,870924.NQ,870925.NQ,870931.NQ,870935.NQ,870936.NQ,870952.NQ,870953.NQ,870955.NQ,870957.NQ,870958.NQ,870959.NQ,870961.NQ,870962.NQ,870966.NQ,870968.NQ,870970.NQ,870972.NQ,870976.NQ,870978.NQ,870979.NQ,870981.NQ,870984.NQ,870986.NQ,870990.NQ,870991.NQ,870992.NQ,870996.NQ,870997.NQ,871002.NQ,871003.NQ,871005.NQ,871008.NQ,871012.NQ,871013.NQ,871014.NQ,871019.NQ,871022.NQ,871023.NQ,871028.NQ,871037.NQ,871039.NQ,871044.NQ,871047.NQ,871048.NQ,871056.NQ,871060.NQ,871064.NQ,871073.NQ,871074.NQ,871075.NQ,871076.NQ,871080.NQ,871081.NQ,871082.NQ,871084.NQ,871086.NQ,871088.NQ,871090.NQ,871091.NQ,871093.NQ,871095.NQ,871097.NQ,871098.NQ,871101.NQ,871102.NQ,871107.NQ,871108.NQ,871109.NQ,871122.NQ,871124.NQ,871125.NQ,871126.NQ,871132.NQ,871135.NQ,871136.NQ,871137.NQ,871138.NQ,871140.NQ,871147.NQ,871148.NQ,871149.NQ,871158.NQ,871163.NQ,871166.NQ,871167.NQ,871168.NQ,871169.NQ,871171.NQ,871172.NQ,871174.NQ,871176.NQ,871177.NQ,871178.NQ,871181.NQ,871183.NQ,871185.NQ,871186.NQ,871190.NQ,871192.NQ,871194.NQ,871195.NQ,871197.NQ,871198.NQ,871200.NQ,871201.NQ,871206.NQ,871215.NQ,871222.NQ,871223.NQ,871225.NQ,871226.NQ,871228.NQ,871229.NQ,871230.NQ,871231.NQ,871232.NQ,871235.NQ,871239.NQ,871241.NQ,871242.NQ,871244.NQ,871248.NQ,871252.NQ,871255.NQ,871258.NQ,871260.NQ,871261.NQ,871262.NQ,871263.NQ,871264.NQ,871271.NQ,871272.NQ,871274.NQ,871275.NQ,871278.NQ,871280.NQ,871281.NQ,871282.NQ,871283.NQ,871289.NQ,871291.NQ,871294.NQ,871295.NQ,871296.NQ,871297.NQ,871298.NQ,871301.NQ,871304.NQ,871305.NQ,871308.NQ,871310.NQ,871311.NQ,871312.NQ,871313.NQ,871314.NQ,871315.NQ,871318.NQ,871320.NQ,871323.NQ,871324.NQ,871325.NQ,871329.NQ,871330.NQ,871332.NQ,871334.NQ,871335.NQ,871336.NQ,871338.NQ,871339.NQ,871340.NQ,871342.NQ,871343.NQ,871346.NQ,871348.NQ,871349.NQ,871350.NQ,871351.NQ,871352.NQ,871356.NQ,871357.NQ,871358.NQ,871359.NQ,871360.NQ,871361.NQ,871364.NQ,871365.NQ,871366.NQ,871368.NQ,871369.NQ,871371.NQ,871373.NQ,871380.NQ,871381.NQ,871382.NQ,871386.NQ,871392.NQ,871394.NQ,871395.NQ,871398.NQ,871402.NQ,871403.NQ,871404.NQ,871406.NQ,871408.NQ,871409.NQ,871410.NQ,871412.NQ,871413.NQ,871415.NQ,871417.NQ,871418.NQ,871421.NQ,871424.NQ,871425.NQ,871427.NQ,871429.NQ,871430.NQ,871433.NQ,871436.NQ,871444.NQ,871446.NQ,871447.NQ,871449.NQ,871450.NQ,871451.NQ,871453.NQ,871455.NQ,871457.NQ,871458.NQ,871459.NQ,871461.NQ,871465.NQ,871466.NQ,871469.NQ,871470.NQ,871471.NQ,871472.NQ,871475.NQ,871478.NQ,871480.NQ,871481.NQ,871482.NQ,871484.NQ,871485.NQ,871486.NQ,871487.NQ,871491.NQ,871494.NQ,871495.NQ,871500.NQ,871501.NQ,871504.NQ,871507.NQ,871508.NQ,871511.NQ,871512.NQ,871515.NQ,871516.NQ,871518.NQ,871519.NQ,871522.NQ,871527.NQ,871528.NQ,871529.NQ,871536.NQ,871539.NQ,871540.NQ,871547.NQ,871548.NQ,871549.NQ,871550.NQ,871551.NQ,871554.NQ,871555.NQ,871556.NQ,871559.NQ,871561.NQ,871562.NQ,871563.NQ,871564.NQ,871566.NQ,871567.NQ,871572.NQ,871574.NQ,871575.NQ,871580.NQ,871582.NQ,871583.NQ,871584.NQ,871589.NQ,871592.NQ,871594.NQ,871597.NQ,871598.NQ,871599.NQ,871601.NQ,871604.NQ,871605.NQ,871607.NQ,871613.NQ,871615.NQ,871617.NQ,871618.NQ,871620.NQ,871621.NQ,871626.NQ,871629.NQ,871633.NQ,871635.NQ,871636.NQ,871641.NQ,871643.NQ,871644.NQ,871645.NQ,871646.NQ,871647.NQ,871648.NQ,871651.NQ,871652.NQ,871653.NQ,871654.NQ,871655.NQ,871656.NQ,871664.NQ,871670.NQ,871672.NQ,871674.NQ,871675.NQ,871676.NQ,871677.NQ,871685.NQ,871688.NQ,871690.NQ,871691.NQ,871692.NQ,871693.NQ,871694.NQ,871695.NQ,871696.NQ,871699.NQ,871700.NQ,871701.NQ,871702.NQ,871703.NQ,871705.NQ,871707.NQ,871710.NQ,871713.NQ,871714.NQ,871715.NQ,871717.NQ,871721.NQ,871725.NQ,871727.NQ,871728.NQ,871729.NQ,871732.NQ,871733.NQ,871734.NQ,871736.NQ,871738.NQ,871739.NQ,871740.NQ,871741.NQ,871744.NQ,871748.NQ,871751.NQ,871756.NQ,871757.NQ,871760.NQ,871761.NQ,871765.NQ,871767.NQ,871768.NQ,871769.NQ,871771.NQ,871772.NQ,871774.NQ,871775.NQ,871781.NQ,871782.NQ,871783.NQ,871787.NQ,871791.NQ,871792.NQ,871794.NQ,871797.NQ,871799.NQ,871803.NQ,871805.NQ,871806.NQ,871807.NQ,871811.NQ,871813.NQ,871816.NQ,871819.NQ,871821.NQ,871822.NQ,871823.NQ,871824.NQ,871827.NQ,871828.NQ,871830.NQ,871833.NQ,871834.NQ,871835.NQ,871837.NQ,871839.NQ,871840.NQ,871841.NQ,871842.NQ,871843.NQ,871845.NQ,871846.NQ,871847.NQ,871848.NQ,871849.NQ,871855.NQ,871856.NQ,871858.NQ,871859.NQ,871862.NQ,871866.NQ,871868.NQ,871870.NQ,871874.NQ,871875.NQ,871878.NQ,871879.NQ,871880.NQ,871881.NQ,871882.NQ,871883.NQ,871884.NQ,871885.NQ,871888.NQ,871892.NQ,871894.NQ,871897.NQ,871902.NQ,871907.NQ,871908.NQ,871909.NQ,871910.NQ,871912.NQ,871920.NQ,871923.NQ,871927.NQ,871930.NQ,871931.NQ,871932.NQ,871933.NQ,871934.NQ,871935.NQ,871936.NQ,871937.NQ,871939.NQ,871941.NQ,871943.NQ,871945.NQ,871948.NQ,871950.NQ,871951.NQ,871952.NQ,871953.NQ,871954.NQ,871955.NQ,871957.NQ,871958.NQ,871962.NQ,871968.NQ,871972.NQ,871975.NQ,871979.NQ,871982.NQ,871986.NQ,871987.NQ,871988.NQ,871990.NQ,871992.NQ,871994.NQ,871995.NQ,872001.NQ,872002.NQ,872006.NQ,872007.NQ,872009.NQ,872013.NQ,872014.NQ,872017.NQ,872018.NQ,872019.NQ,872021.NQ,872022.NQ,872024.NQ,872032.NQ,872037.NQ,872038.NQ,872042.NQ,872044.NQ,872049.NQ,872050.NQ,872055.NQ,872058.NQ,872061.NQ,872063.NQ,872065.NQ,872066.NQ,872068.NQ,872069.NQ,872070.NQ,872076.NQ,872077.NQ,872082.NQ,872085.NQ,872089.NQ,872090.NQ,872092.NQ,872093.NQ,872094.NQ,872098.NQ,872099.NQ,872100.NQ,872101.NQ,872102.NQ,872103.NQ,872104.NQ,872106.NQ,872107.NQ,872109.NQ,872111.NQ,872112.NQ,872117.NQ,872118.NQ,872121.NQ,872122.NQ,872123.NQ,872124.NQ,872127.NQ,872128.NQ,872129.NQ,872131.NQ,872135.NQ,872137.NQ,872138.NQ,872139.NQ,872143.NQ,872144.NQ,872145.NQ,872149.NQ,872150.NQ,872152.NQ,872153.NQ,872154.NQ,872155.NQ,872156.NQ,872161.NQ,872162.NQ,872164.NQ,872165.NQ,872167.NQ,872168.NQ,872170.NQ,872172.NQ,872174.NQ,872175.NQ,872178.NQ,872180.NQ,872183.NQ,872184.NQ,872186.NQ,872188.NQ,872189.NQ,872191.NQ,872192.NQ,872197.NQ,872198.NQ,872199.NQ,872200.NQ,872202.NQ,872203.NQ,872204.NQ,872205.NQ,872206.NQ,872209.NQ,872211.NQ,872212.NQ,872213.NQ,872216.NQ,872217.NQ,872220.NQ,872223.NQ,872224.NQ,872226.NQ,872227.NQ,872230.NQ,872232.NQ,872233.NQ,872235.NQ,872238.NQ,872240.NQ,872242.NQ,872245.NQ,872247.NQ,872248.NQ,872249.NQ,872250.NQ,872255.NQ,872256.NQ,872258.NQ,872259.NQ,872260.NQ,872261.NQ,872262.NQ,872264.NQ,872266.NQ,872267.NQ,872271.NQ,872272.NQ,872279.NQ,872281.NQ,872284.NQ,872285.NQ,872286.NQ,872287.NQ,872288.NQ,872289.NQ,872290.NQ,872291.NQ,872293.NQ,872294.NQ,872297.NQ,872298.NQ,872299.NQ,872300.NQ,872301.NQ,872303.NQ,872304.NQ,872305.NQ,872307.NQ,872308.NQ,872310.NQ,872313.NQ,872315.NQ,872318.NQ,872319.NQ,872320.NQ,872322.NQ,872324.NQ,872325.NQ,872327.NQ,872328.NQ,872330.NQ,872333.NQ,872334.NQ,872336.NQ,872337.NQ,872338.NQ,872339.NQ,872340.NQ,872341.NQ,872342.NQ,872344.NQ,872345.NQ,872346.NQ,872347.NQ,872349.NQ,872352.NQ,872355.NQ,872357.NQ,872358.NQ,872361.NQ,872362.NQ,872363.NQ,872369.NQ,872371.NQ,872373.NQ,872375.NQ,872376.NQ,872377.NQ,872379.NQ,872380.NQ,872381.NQ,872383.NQ,872386.NQ,872388.NQ,872389.NQ,872391.NQ,872393.NQ,872396.NQ,872397.NQ,872398.NQ,872399.NQ,872400.NQ,872401.NQ,872402.NQ,872404.NQ,872405.NQ,872406.NQ,872408.NQ,872409.NQ,872411.NQ,872415.NQ,872416.NQ,872417.NQ,872418.NQ,872419.NQ,872420.NQ,872421.NQ,872422.NQ,872423.NQ,872424.NQ,872425.NQ,872426.NQ,872430.NQ,872432.NQ,872434.NQ,872435.NQ,872439.NQ,872440.NQ,872443.NQ,872444.NQ,872449.NQ,872450.NQ,872451.NQ,872453.NQ,872458.NQ,872460.NQ,872461.NQ,872464.NQ,872471.NQ,872473.NQ,872475.NQ,872476.NQ,872477.NQ,872478.NQ,872480.NQ,872481.NQ,872482.NQ,872483.NQ,872486.NQ,872488.NQ,872489.NQ,872490.NQ,872491.NQ,872492.NQ,872493.NQ,872494.NQ,872495.NQ,872496.NQ,872497.NQ,872498.NQ,872500.NQ,872502.NQ,872504.NQ,872505.NQ,872506.NQ,872507.NQ,872510.NQ,872511.NQ,872512.NQ,872515.NQ,872516.NQ,872518.NQ,872519.NQ,872520.NQ,872521.NQ,872522.NQ,872523.NQ,872525.NQ,872528.NQ,872530.NQ,872532.NQ,872534.NQ,872536.NQ,872538.NQ,872539.NQ,872544.NQ,872545.NQ,872546.NQ,872547.NQ,872549.NQ,872550.NQ,872551.NQ,872552.NQ,872553.NQ,872554.NQ,872555.NQ,872556.NQ,872558.NQ,872559.NQ,872560.NQ,872562.NQ,872564.NQ,872565.NQ,872567.NQ,872568.NQ,872570.NQ,872572.NQ,872573.NQ,872574.NQ,872575.NQ,872577.NQ,872578.NQ,872579.NQ,872580.NQ,872581.NQ,872582.NQ,872583.NQ,872584.NQ,872585.NQ,872586.NQ,872588.NQ,872593.NQ,872594.NQ,872595.NQ,872597.NQ,872598.NQ,872599.NQ,872602.NQ,872606.NQ,872609.NQ,872610.NQ,872611.NQ,872612.NQ,872616.NQ,872617.NQ,872618.NQ,872621.NQ,872622.NQ,872623.NQ,872624.NQ,872626.NQ,872628.NQ,872629.NQ,872631.NQ,872632.NQ,872633.NQ,872634.NQ,872635.NQ,872636.NQ,872638.NQ,872640.NQ,872641.NQ,872642.NQ,872643.NQ,872644.NQ,872645.NQ,872646.NQ,872655.NQ,872656.NQ,872658.NQ,872660.NQ,872662.NQ,872663.NQ,872665.NQ,872666.NQ,872667.NQ,872668.NQ,872669.NQ,872670.NQ,872672.NQ,872673.NQ,872675.NQ,872676.NQ,872677.NQ,872678.NQ,872679.NQ,872680.NQ,872681.NQ,872682.NQ,872684.NQ,872685.NQ,872687.NQ,872689.NQ,872691.NQ,872696.NQ,872698.NQ,872702.NQ,872704.NQ,872706.NQ,872707.NQ,872709.NQ,872710.NQ,872711.NQ,872712.NQ,872714.NQ,872715.NQ,872716.NQ,872720.NQ,872721.NQ,872723.NQ,872727.NQ,872728.NQ,872729.NQ,872732.NQ,872733.NQ,872735.NQ,872739.NQ,872740.NQ,872741.NQ,872742.NQ,872744.NQ,872745.NQ,872747.NQ,872748.NQ,872750.NQ,872752.NQ,872753.NQ,872757.NQ,872758.NQ,872759.NQ,872760.NQ,872762.NQ,872765.NQ,872766.NQ,872767.NQ,872773.NQ,872775.NQ,872776.NQ,872779.NQ,872780.NQ,872782.NQ,872783.NQ,872784.NQ,872787.NQ,872788.NQ,872791.NQ,872792.NQ,872793.NQ,872799.NQ,872801.NQ,872802.NQ,872804.NQ,872806.NQ,872807.NQ,872809.NQ,872810.NQ,872811.NQ,872813.NQ,872816.NQ,872817.NQ,872818.NQ,872819.NQ,872820.NQ,872821.NQ,872822.NQ,872823.NQ,872827.NQ,872828.NQ,872829.NQ,872831.NQ,872833.NQ,872836.NQ,872839.NQ,872840.NQ,872842.NQ,872843.NQ,872844.NQ,872845.NQ,872850.NQ,872851.NQ,872856.NQ,872857.NQ,872858.NQ,872860.NQ,872861.NQ,872862.NQ,872863.NQ,872864.NQ,872865.NQ,872868.NQ,872869.NQ,872870.NQ,872873.NQ,872874.NQ,872875.NQ,872878.NQ,872879.NQ,872880.NQ,872881.NQ,872882.NQ,872883.NQ,872884.NQ,872886.NQ,872887.NQ,872888.NQ,872890.NQ,872891.NQ,872892.NQ,872893.NQ,872894.NQ,872895.NQ,872896.NQ,872899.NQ,872900.NQ,872901.NQ,872902.NQ,872906.NQ,872907.NQ,872908.NQ,872910.NQ,872911.NQ,872915.NQ,872916.NQ,872917.NQ,872918.NQ,872919.NQ,872920.NQ,872921.NQ,872922.NQ,872926.NQ,872928.NQ,872930.NQ,872931.NQ,872935.NQ,872936.NQ,872937.NQ,872940.NQ,872941.NQ,872946.NQ,872948.NQ,872950.NQ,872951.NQ,872952.NQ,872953.NQ,872955.NQ,872956.NQ,872957.NQ,872958.NQ,872960.NQ,872961.NQ,872962.NQ,872963.NQ,872965.NQ,872967.NQ,872968.NQ,872969.NQ,872970.NQ,872971.NQ,872975.NQ,872978.NQ,872980.NQ,872981.NQ,872982.NQ,872984.NQ,872985.NQ,872986.NQ,872987.NQ,872988.NQ,872990.NQ,872991.NQ,872992.NQ,872996.NQ,872997.NQ,872998.NQ,872999.NQ,873000.NQ,873002.NQ,873003.NQ,873004.NQ,873005.NQ,873006.NQ,873007.NQ,873008.NQ,873009.NQ,873010.NQ,873011.NQ,873015.NQ,873016.NQ,873017.NQ,873018.NQ,873019.NQ,873020.NQ,873021.NQ,873025.NQ,873026.NQ,873027.NQ,873030.NQ,873031.NQ,873033.NQ,873036.NQ,873037.NQ,873038.NQ,873041.NQ,873045.NQ,873046.NQ,873047.NQ,873048.NQ,873049.NQ,873050.NQ,873051.NQ,873052.NQ,873053.NQ,873055.NQ,873056.NQ,873057.NQ,873058.NQ,873059.NQ,873060.NQ,873061.NQ,873062.NQ,873063.NQ,873065.NQ,873069.NQ,873070.NQ,873071.NQ,873073.NQ,873078.NQ,873079.NQ,873081.NQ,873082.NQ,873083.NQ,873085.NQ,873088.NQ,873089.NQ,873092.NQ,873093.NQ,873095.NQ,873097.NQ,873098.NQ,873099.NQ,873100.NQ,873101.NQ,873102.NQ,873107.NQ,873108.NQ,873110.NQ,873111.NQ,873114.NQ,873116.NQ,873117.NQ,873118.NQ,873119.NQ,873120.NQ,873125.NQ,873126.NQ,873128.NQ,873129.NQ,873130.NQ,873131.NQ,873132.NQ,873133.NQ,873135.NQ,873136.NQ,873137.NQ,873138.NQ,873139.NQ,873140.NQ,873141.NQ,873142.NQ,873143.NQ,873145.NQ,873146.NQ,873147.NQ,873148.NQ,873150.NQ,873151.NQ,873153.NQ,873155.NQ,873157.NQ,873161.NQ,873162.NQ,873163.NQ,873165.NQ,873170.NQ,873172.NQ,873173.NQ,873175.NQ,873176.NQ,873177.NQ,873180.NQ,873182.NQ,873183.NQ,873184.NQ,873185.NQ,873186.NQ,873187.NQ,873188.NQ,873189.NQ,873190.NQ,873192.NQ,873193.NQ,873195.NQ,873196.NQ,873197.NQ,873198.NQ,873199.NQ,873202.NQ,873203.NQ,873205.NQ,873206.NQ,873207.NQ,873208.NQ,873210.NQ,873212.NQ,873215.NQ,873217.NQ,873218.NQ,873219.NQ,873222.NQ,873224.NQ,873225.NQ,873226.NQ,873227.NQ,873228.NQ,873229.NQ,873230.NQ,873231.NQ,873232.NQ,873233.NQ,873234.NQ,873235.NQ,873236.NQ,873237.NQ,873238.NQ,873239.NQ,873240.NQ,873241.NQ,873243.NQ,873245.NQ,873246.NQ,873247.NQ,873248.NQ,873249.NQ,873250.NQ,873251.NQ,873252.NQ,873253.NQ,873255.NQ,873256.NQ,873258.NQ,873260.NQ,873261.NQ,873263.NQ,873266.NQ,873267.NQ,873268.NQ,873269.NQ,873270.NQ,873272.NQ,873273.NQ,873275.NQ,873277.NQ,873278.NQ,873279.NQ,873280.NQ,873283.NQ,873285.NQ,873286.NQ,873288.NQ,873289.NQ,873290.NQ,873291.NQ,873293.NQ,873295.NQ,873296.NQ,873297.NQ,873300.NQ,873301.NQ,873302.NQ,873306.NQ,873307.NQ,873308.NQ,873309.NQ,873311.NQ,873312.NQ,873316.NQ,873318.NQ,873319.NQ,873320.NQ,873321.NQ,873322.NQ,873323.NQ,873324.NQ,873325.NQ,873326.NQ,873327.NQ,873328.NQ,873329.NQ,873330.NQ,873331.NQ,873332.NQ,873333.NQ,873334.NQ,873335.NQ,873336.NQ,873337.NQ,873338.NQ,873340.NQ,873341.NQ,873343.NQ,873344.NQ,873346.NQ,873347.NQ,873348.NQ,873349.NQ,873350.NQ,873351.NQ,873352.NQ,873353.NQ,873355.NQ,873356.NQ,873357.NQ,873358.NQ,873359.NQ,873360.NQ,873361.NQ,873362.NQ,873363.NQ,873366.NQ,873367.NQ,873369.NQ,873370.NQ,873373.NQ,873375.NQ,873376.NQ,873377.NQ,873378.NQ,873380.NQ,873381.NQ,873382.NQ,873383.NQ,873384.NQ,873385.NQ,873386.NQ,873387.NQ,873389.NQ,873390.NQ,873391.NQ,873392.NQ,873393.NQ,873395.NQ,873396.NQ,873397.NQ,873398.NQ,873399.NQ,873400.NQ,873401.NQ,873402.NQ,873403.NQ,873406.NQ,873407.NQ,873408.NQ,873410.NQ,873411.NQ,873412.NQ,873415.NQ,873416.NQ,873417.NQ,873418.NQ,873419.NQ,873420.NQ,873421.NQ,873423.NQ,873425.NQ,873426.NQ,873427.NQ,873428.NQ,873429.NQ,873430.NQ,873432.NQ,873433.NQ,873434.NQ,873435.NQ,873436.NQ,873437.NQ,873438.NQ,873440.NQ,873441.NQ,873442.NQ,873443.NQ,873444.NQ,873446.NQ,873447.NQ,873448.NQ,873449.NQ,873450.NQ,873451.NQ,873452.NQ,873453.NQ,873455.NQ,873456.NQ,873458.NQ,873459.NQ,873460.NQ,873461.NQ,873462.NQ,873463.NQ,873465.NQ,873467.NQ,873468.NQ,873469.NQ,873470.NQ,873471.NQ,873472.NQ,873473.NQ,873475.NQ,873476.NQ,873477.NQ,873478.NQ,873479.NQ,873480.NQ,873481.NQ,873482.NQ,873483.NQ,873485.NQ,873486.NQ,873487.NQ,873488.NQ,873490.NQ,873491.NQ,873492.NQ,873493.NQ,873495.NQ,873496.NQ,873498.NQ,873499.NQ,873500.NQ,873501.NQ,873502.NQ,873503.NQ,873505.NQ,873506.NQ,873507.NQ,873508.NQ,873509.NQ,873510.NQ,873511.NQ,873512.NQ,873513.NQ,873515.NQ,873516.NQ,873517.NQ,873518.NQ,873519.NQ,873520.NQ,873521.NQ,873522.NQ,873523.NQ,873525.NQ,873526.NQ,873528.NQ,873529.NQ,873530.NQ,873531.NQ,873533.NQ,873535.NQ,873536.NQ,873537.NQ,873538.NQ,873539.NQ,873540.NQ,873541.NQ,873542.NQ,873543.NQ,873546.NQ,873547.NQ,873548.NQ,873549.NQ,873550.NQ,873551.NQ,873552.NQ,873553.NQ,873554.NQ,873556.NQ,873557.NQ,873559.NQ,873560.NQ,873561.NQ,873563.NQ,873564.NQ,873565.NQ,873566.NQ,873567.NQ,873568.NQ,873569.NQ,873570.NQ,873571.NQ,873572.NQ,873573.NQ,873575.NQ,873576.NQ,873577.NQ,873579.NQ,873580.NQ,873581.NQ,873583.NQ,873585.NQ,873586.NQ,873587.NQ,873588.NQ,873589.NQ,873590.NQ,873591.NQ,873592.NQ,873593.NQ,873595.NQ,873596.NQ,873597.NQ,873598.NQ,873599.NQ,873600.NQ,873601.NQ,873602.NQ,873603.NQ,873605.NQ,873606.NQ,873607.NQ,873608.NQ,873609.NQ,873610.NQ,873611.NQ,873612.NQ,873613.NQ,873615.NQ,873616.NQ,873617.NQ,873618.NQ,873619.NQ,873620.NQ,873621.NQ,873623.NQ,873625.NQ,873626.NQ,873627.NQ,873628.NQ,873629.NQ,873630.NQ,873631.NQ,873632.NQ,873633.NQ,873635.NQ,873636.NQ,873637.NQ,873638.NQ,873639.NQ,873640.NQ,873641.NQ,873642.NQ,873643.NQ,873644.NQ,873645.NQ,873646.NQ,873647.NQ,873648.NQ,873649.NQ,873650.NQ,873651.NQ,873652.NQ,873653.NQ,873655.NQ,873656.NQ,873657.NQ,873658.NQ,873659.NQ,873660.NQ,873661.NQ,873662.NQ,873663.NQ,873664.NQ,873665.NQ,873666.NQ,873667.NQ,873668.NQ,873669.NQ,873670.NQ,873671.NQ,873672.NQ,873673.NQ,873675.NQ,873676.NQ,873678.NQ,873679.NQ,873680.NQ,873681.NQ,873682.NQ,873683.NQ,873684.NQ,873685.NQ,873686.NQ,873687.NQ,873688.NQ,873689.NQ,873690.NQ,873691.NQ,873692.NQ,873693.NQ,873694.NQ,873695.NQ,873696.NQ,873697.NQ,873698.NQ,873699.NQ,873700.NQ,873701.NQ,873702.NQ,873703.NQ,873705.NQ,873706.NQ,873707.NQ,873708.NQ,873709.NQ,873710.NQ,873711.NQ,873712.NQ,873713.NQ,873715.NQ,873716.NQ,873717.NQ,873718.NQ,873719.NQ,873720.NQ,873721.NQ,873722.NQ,873725.NQ,873726.NQ,873727.NQ,873728.NQ,873729.NQ,873730.NQ,873731.NQ,873732.NQ,873733.NQ,873735.NQ,873736.NQ,873737.NQ,873738.NQ,873739.NQ,873740.NQ,873741.NQ,873743.NQ,873744.NQ,873745.NQ,873746.NQ,873747.NQ,873748.NQ,873749.NQ,873750.NQ,873751.NQ,873752.NQ,873753.NQ,873755.NQ,873756.NQ,873757.NQ,873758.NQ,873759.NQ,873760.NQ,873762.NQ,873763.NQ,873765.NQ,873766.NQ,873767.NQ,873768.NQ,873769.NQ,873770.NQ,873771.NQ,873772.NQ,873773.NQ,873775.NQ,873776.NQ,873777.NQ,873778.NQ,873779.NQ,873780.NQ,873781.NQ,873782.NQ,873783.NQ,873784.NQ,873785.NQ,873786.NQ,873787.NQ,873788.NQ,873789.NQ,873790.NQ,873791.NQ,873792.NQ,873793.NQ,873795.NQ,873796.NQ,873797.NQ,873798.NQ,873799.NQ,873800.NQ,873801.NQ,873802.NQ,873803.NQ,873804.NQ,873805.NQ,873806.NQ,873807.NQ,873808.NQ,873809.NQ,873810.NQ,873811.NQ,873812.NQ,873813.NQ,873814.NQ,873815.NQ,873816.NQ,873817.NQ,873818.NQ,873820.NQ,873821.NQ,873822.NQ,873823.NQ,873825.NQ,873826.NQ,873827.NQ,873828.NQ,873829.NQ,873830.NQ,873831.NQ,873832.NQ,873833.NQ,873834.NQ,873835.NQ,873836.NQ,873837.NQ,873838.NQ,873839.NQ,873840.NQ,873841.NQ,873842.NQ,873843.NQ,873845.NQ,873847.NQ,873848.NQ,873849.NQ,873850.NQ,873851.NQ,873852.NQ,873853.NQ,873855.NQ,873856.NQ,873857.NQ,873858.NQ,873859.NQ,873860.NQ,873861.NQ,873862.NQ,873863.NQ,873864.NQ,873865.NQ,873866.NQ,873867.NQ,873868.NQ,873869.NQ,873870.NQ,873871.NQ,873872.NQ,873873.NQ,873874.NQ,873875.NQ,873876.NQ,873877.NQ,873878.NQ,873879.NQ,873880.NQ,873881.NQ,873882.NQ,873884.NQ,873885.NQ,873886.NQ,873887.NQ,873888.NQ,873889.NQ,873890.NQ,873891.NQ,873892.NQ,873893.NQ,873895.NQ,873896.NQ,873898.NQ,873899.NQ,873901.NQ,873902.NQ,873903.NQ,873905.NQ,873906.NQ,873907.NQ,873908.NQ,873909.NQ,873910.NQ,873911.NQ,873912.NQ,873913.NQ,873914.NQ,873915.NQ,873916.NQ,873917.NQ,873918.NQ,873919.NQ,873920.NQ,873921.NQ,873922.NQ,873923.NQ,873926.NQ,873927.NQ,873928.NQ,873929.NQ,873930.NQ,873931.NQ,873932.NQ,873933.NQ,873935.NQ,873936.NQ,873937.NQ,873938.NQ,873939.NQ,873943.NQ,873945.NQ,873946.NQ,873947.NQ,873948.NQ,873949.NQ,873950.NQ,873951.NQ,873953.NQ,873955.NQ,873956.NQ,873957.NQ,873958.NQ,873959.NQ,873960.NQ,873961.NQ,873962.NQ,873963.NQ,873964.NQ,873965.NQ,873966.NQ,873967.NQ,873968.NQ,873969.NQ,873970.NQ,873971.NQ,873972.NQ,873973.NQ,873975.NQ,873976.NQ,873977.NQ,873978.NQ,873979.NQ,873980.NQ,873981.NQ,873982.NQ,873983.NQ,873985.NQ,873987.NQ,873988.NQ,873989.NQ,873990.NQ,873991.NQ,873992.NQ,873993.NQ,873995.NQ,873996.NQ,873997.NQ,873998.NQ,873999.NQ,874000.NQ,874001.NQ,874002.NQ,874003.NQ,874005.NQ,874006.NQ,874007.NQ,874009.NQ,874010.NQ,874011.NQ,874016.NQ,874017.NQ,874018.NQ,874019.NQ,874020.NQ,874021.NQ,874022.NQ,874025.NQ,874026.NQ,874027.NQ,874028.NQ,874030.NQ,874031.NQ,874033.NQ,874035.NQ,874036.NQ,874037.NQ,874038.NQ,874039.NQ,874040.NQ,874041.NQ,874047.NQ,874048.NQ,874050.NQ,874055.NQ,874056.NQ,874057.NQ,874058.NQ,874062.NQ,874068.NQ,874070.NQ,874075.NQ,874095.NQ,874107.NQ,874109.NQ,874110.NQ", "close_per,trade_days_per,share_totaltradable,wgsd_assets","startDate="+date_b+";endDate="+date_a+";priceAdj=U;unit=1;tradeDate="+date_c+";rptDate="+date_c+";rptType=1;currencyType=")

In [34]:
df_n_c1 = pd.DataFrame(data1.Data,index = data1.Fields,columns = data1.Codes)

In [35]:
df_n_c1 = df_n_c1.T

In [36]:
df_n_c1.dropna(how = 'all',inplace = True)
df_n_c1.fillna(0)

,CLOSE_PER,TRADE_DAYS_PER,SHARE_TOTALTRADABLE,WGSD_ASSETS
430003.NQ,3.96,41.0,48717203.0,8.627768e+08
430004.NQ,1.54,67.0,95865281.0,1.353354e+08
430005.NQ,27.78,72.0,130631750.0,1.034773e+09
430009.NQ,5.02,43.0,49013173.0,2.069899e+08
430010.NQ,4.00,36.0,120000000.0,3.192366e+09
...,...,...,...,...
874038.NQ,0.00,0.0,0.0,3.590673e+08
874039.NQ,0.00,0.0,0.0,1.815759e+08
874047.NQ,0.00,0.0,0.0,1.356850e+08
874056.NQ,0.00,0.0,0.0,8.037016e+07


In [37]:
divide(df_n_c1)

,CLOSE_PER,TRADE_DAYS_PER,SHARE_TOTALTRADABLE,WGSD_ASSETS,Segment_a,Segment_c,Segment_d,Segment_s
430003.NQ,3.96,41.0,48717203.0,8.627768e+08,1,2,1,1
430004.NQ,1.54,67.0,95865281.0,1.353354e+08,1,3,1,1
430005.NQ,27.78,72.0,130631750.0,1.034773e+09,1,1,1,1
430009.NQ,5.02,43.0,49013173.0,2.069899e+08,1,2,1,1
430010.NQ,4.00,36.0,120000000.0,3.192366e+09,1,2,1,1
...,...,...,...,...,...,...,...,...
874038.NQ,NaN,NaN,NaN,3.590673e+08,1,4,4,4
874039.NQ,NaN,NaN,NaN,1.815759e+08,1,4,4,4
874047.NQ,NaN,NaN,NaN,1.356850e+08,1,4,4,4
874056.NQ,NaN,NaN,NaN,8.037016e+07,2,4,4,4


# 科创板